In [1]:
# CELL 01 - Bootstrap repo (clone/update)
# Colab bootstrap (no external notebook/script execution)
from pathlib import Path
import os
import shutil
import subprocess

REPO_URL = os.environ.get('CAFA_REPO_GIT_URL', 'https://github.com/PeterOla/cafa-6-protein-function-prediction.git')
REPO_DIR = Path(os.environ.get('CAFA_REPO_DIR', '/content/cafa-6-protein-function-prediction'))
SAFE_CWD = Path('/content') if Path('/content').exists() else Path('/')
def run(cmd: list[str]) -> None:
    cmd_str = ' '.join(cmd)
    print('+', cmd_str)
    p = subprocess.run(cmd, text=True, capture_output=True, cwd=str(SAFE_CWD))
    if p.stdout.strip():
        print(p.stdout)
    if p.stderr.strip():
        print(p.stderr)
    if p.returncode != 0:
        raise RuntimeError(f'Command failed (exit={p.returncode}): {cmd_str}')
os.chdir(SAFE_CWD)
if REPO_DIR.exists() and (REPO_DIR / '.git').is_dir():
    run(['git', '-C', str(REPO_DIR), 'fetch', '--depth', '1', 'origin'])
    run(['git', '-C', str(REPO_DIR), 'reset', '--hard', 'origin/HEAD'])
else:
    if REPO_DIR.exists():
        shutil.rmtree(REPO_DIR, ignore_errors=True)
    run(['git', 'clone', '--depth', '1', REPO_URL, str(REPO_DIR)])
os.chdir(REPO_DIR)
print('CWD:', Path.cwd())

+ git clone --depth 1 https://github.com/PeterOla/cafa-6-protein-function-prediction.git /content/cafa-6-protein-function-prediction
Cloning into '/content/cafa-6-protein-function-prediction'...

CWD: /content/cafa-6-protein-function-prediction


In [ ]:
# CELL 02 - Install dependencies
import importlib.util
import os
import subprocess
import sys

def _detect_kaggle() -> bool:
    return bool(os.environ.get('KAGGLE_KERNEL_RUN_TYPE') or os.environ.get('KAGGLE_URL_BASE') or os.environ.get('KAGGLE_DATA_PROXY_URL'))
def _detect_colab() -> bool:
    return bool(os.environ.get('COLAB_RELEASE_TAG') or os.environ.get('COLAB_GPU') or os.environ.get('COLAB_TPU_ADDR'))
IS_KAGGLE = _detect_kaggle()
IS_COLAB = (not IS_KAGGLE) and _detect_colab()
if IS_KAGGLE:
    print('Environment: Kaggle Detected')
elif IS_COLAB:
    print('Environment: Colab Detected')
else:
    print('Environment: Local Detected')
FORCE_PIP = os.environ.get('CAFA_FORCE_PIP', '0').strip() == '1'

def _pip_install(args: list[str]) -> None:
    print('+', sys.executable, '-m', 'pip', 'install', *args)
    subprocess.check_call([sys.executable, '-m', 'pip', '-q', 'install', *args])
# Kaggle has a heavily preinstalled environment; avoid upgrading core packages (pandas/notebook/requests/tornado/RAPIDS).
# Only install a small set if missing, unless CAFA_FORCE_PIP=1.
if IS_KAGGLE and not FORCE_PIP:
    minimal = ['obonet', 'biopython', 'pyyaml', 'py-boost']
    missing = [p for p in minimal if importlib.util.find_spec(p) is None]
    if missing:
        _pip_install(missing)
    else:
        print('Kaggle: skipping pip install (already satisfied). Set CAFA_FORCE_PIP=1 to force.')
else:
    _pip_install(['-r', 'requirements.txt'])


In [ ]:
# CELL 03 - Solution: 1. SETUP, CONFIG & DIAGNOSTICS
# 1. SETUP, CONFIG & DIAGNOSTICS
# ==========================================
# HARDWARE: CPU (Standard)
# ==========================================
import json
import os
import shutil
import subprocess
import sys
import time
import zipfile
from dataclasses import dataclass
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# ------------------------------------------
# Environment Detection & Paths
# ------------------------------------------
# Kaggle images can have `google-colab` installed; never use `import google.colab` as a signal.

def _detect_kaggle() -> bool:
    # Kaggle kernels reliably set at least one of these env vars.
    return bool(
        os.environ.get('KAGGLE_KERNEL_RUN_TYPE')
        or os.environ.get('KAGGLE_URL_BASE')
        or os.environ.get('KAGGLE_DATA_PROXY_URL')
    )


def _detect_colab() -> bool:
    # Colab sets these env vars; this avoids false positives on Kaggle.
    return bool(os.environ.get('COLAB_RELEASE_TAG') or os.environ.get('COLAB_GPU') or os.environ.get('COLAB_TPU_ADDR'))


IS_KAGGLE = _detect_kaggle()
IS_COLAB = (not IS_KAGGLE) and _detect_colab()

if IS_KAGGLE:
    print('Environment: Kaggle Detected')
    INPUT_ROOT = Path('/kaggle/input')
    WORKING_ROOT = Path('/kaggle/working')
    if INPUT_ROOT.exists():
        for dirname, _, filenames in os.walk(str(INPUT_ROOT)):
            for filename in filenames:
                print(os.path.join(dirname, filename))
elif IS_COLAB:
    print('Environment: Colab Detected')
    INPUT_ROOT = Path(os.environ.get('CAFA_INPUT_ROOT', str(Path('/content'))))
    WORKING_ROOT = Path(os.environ.get('CAFA_WORKING_ROOT', str(Path('/content'))))
else:
    print('Environment: Local Detected')
    CURRENT_DIR = Path.cwd()
    if CURRENT_DIR.name == 'notebooks':
        PROJECT_ROOT = CURRENT_DIR.parent
    else:
        PROJECT_ROOT = CURRENT_DIR
    INPUT_ROOT = Path(os.environ.get('CAFA_INPUT_ROOT', str(PROJECT_ROOT)))
    WORKING_ROOT = Path(os.environ.get('CAFA_WORKING_ROOT', str(PROJECT_ROOT / 'artefacts_local')))
    WORKING_ROOT.mkdir(exist_ok=True)

# ------------------------------------------
# Local cache roots (ephemeral) + published artefacts root
# ------------------------------------------
# IMPORTANT: we always write locally first (every runtime needs a write path),
# but the Kaggle Dataset is the *single source of truth* for resumability.

WORK_ROOT = Path(os.environ.get('CAFA_WORK_ROOT', str(WORKING_ROOT / 'work')))
WORK_ROOT.mkdir(parents=True, exist_ok=True)
(WORK_ROOT / 'parsed').mkdir(parents=True, exist_ok=True)
(WORK_ROOT / 'features').mkdir(parents=True, exist_ok=True)
(WORK_ROOT / 'external').mkdir(parents=True, exist_ok=True)

# Keep caches OUT of WORK_ROOT so we never accidentally publish them.
CACHE_ROOT = Path(os.environ.get('CAFA_CACHE_ROOT', str(WORKING_ROOT / 'cache')))
CACHE_ROOT.mkdir(parents=True, exist_ok=True)
os.environ.setdefault('HF_HOME', str(CACHE_ROOT / 'hf_home'))
os.environ.setdefault('TRANSFORMERS_CACHE', str(CACHE_ROOT / 'hf_home'))
os.environ.setdefault('HF_HUB_CACHE', str(CACHE_ROOT / 'hf_hub'))
os.environ.setdefault('TORCH_HOME', str(CACHE_ROOT / 'torch_home'))

# ------------------------------------------
# Dataset Discovery (competition data)
# ------------------------------------------

DATASET_SLUG = 'cafa-6-protein-function-prediction'


def _score_dataset_dir(p: Path) -> int:
    return (
        int((p / 'Train').exists())
        + int((p / 'Test').exists())
        + int((p / 'IA.tsv').exists())
        + int((p / 'sample_submission.tsv').exists())
    )


def find_dataset_root(input_root: Path, dataset_slug: str) -> Path:
    override = (os.environ.get('CAFA_DATASET_ROOT') or '').strip()
    if override:
        p = Path(override)
        if p.exists():
            return p
        raise FileNotFoundError(f'CAFA_DATASET_ROOT is set but does not exist: {p}')

    candidate = input_root / dataset_slug
    if candidate.exists() and _score_dataset_dir(candidate) >= 2:
        return candidate

    if _score_dataset_dir(input_root) >= 2:
        return input_root

    # Shallow scan: input_root/* and input_root/*/* (helps Colab + GitHub clones)
    candidates: list[Path] = []
    if input_root.exists():
        for p in input_root.iterdir():
            if not p.is_dir():
                continue
            candidates.append(p)
            try:
                for q in p.iterdir():
                    if q.is_dir():
                        candidates.append(q)
            except Exception:
                pass

    candidates = sorted(set(candidates), key=_score_dataset_dir, reverse=True)
    if candidates and _score_dataset_dir(candidates[0]) >= 2:
        return candidates[0]

    auto_flag = (os.environ.get('CAFA_COLAB_AUTO_DOWNLOAD') or '').strip() or '<unset>'
    has_user = bool((os.environ.get('KAGGLE_USERNAME') or '').strip())
    has_key = bool((os.environ.get('KAGGLE_KEY') or '').strip())

    raise FileNotFoundError(
        f'Dataset not found under {input_root}. '
        'Note: a GitHub clone does not include the CAFA competition files.\n'
        f'CAFA_COLAB_AUTO_DOWNLOAD={auto_flag}; KAGGLE_USERNAME set={has_user}; KAGGLE_KEY set={has_key}.\n'
        'Fix options:\n'
        '  1) Set CAFA_DATASET_ROOT to a folder containing Train/ and Test/ (e.g. Drive).\n'
        '  2) (Colab) Provide Kaggle creds and set CAFA_COLAB_AUTO_DOWNLOAD=1 to fetch via Kaggle API.'
    )


def _maybe_colab_auto_download_competition() -> None:
    if not IS_COLAB:
        return

    # Default-on in Colab.
    # - Disable by explicitly setting CAFA_COLAB_AUTO_DOWNLOAD=0.
    os.environ.setdefault('CAFA_COLAB_AUTO_DOWNLOAD', '1')

    flag = (os.environ.get('CAFA_COLAB_AUTO_DOWNLOAD') or '').strip()
    if flag == '0':
        print('Colab auto-download disabled (CAFA_COLAB_AUTO_DOWNLOAD=0).')
        return

    target_dir = Path(os.environ.get('CAFA_COLAB_DATA_DIR', str(Path('/content/cafa6_data'))))
    target_dir.mkdir(parents=True, exist_ok=True)
    if _score_dataset_dir(target_dir) >= 2:
        os.environ['CAFA_DATASET_ROOT'] = str(target_dir)
        return

    # Install kaggle CLI if missing
    try:
        subprocess.run(['kaggle', '--version'], check=True, capture_output=True, text=True)
    except Exception:
        subprocess.check_call([sys.executable, '-m', 'pip', '-q', 'install', 'kaggle'])

    # Minimal secret resolution: env vars then Colab userdata
    username = (os.environ.get('KAGGLE_USERNAME') or '').strip()
    key = (os.environ.get('KAGGLE_KEY') or '').strip()
    if (not username) or (not key):
        try:
            from google.colab import userdata  # type: ignore

            username = username or (userdata.get('KAGGLE_USERNAME') or '').strip()
            key = key or (userdata.get('KAGGLE_KEY') or '').strip()
        except Exception:
            pass

    if (not username) or (not key):
        raise RuntimeError(
            'CAFA_COLAB_AUTO_DOWNLOAD=1 but Kaggle API auth is missing. '
            'Set Secrets/env vars KAGGLE_USERNAME and KAGGLE_KEY.'
        )

    env = os.environ.copy()
    env['KAGGLE_USERNAME'] = username
    env['KAGGLE_KEY'] = key

    print(f'Downloading competition data to {target_dir} via Kaggle API...')
    # Do NOT rely on kaggle --unzip; unzip ourselves.
    p = subprocess.run(
        ['kaggle', 'competitions', 'download', '-c', DATASET_SLUG, '-p', str(target_dir)],
        text=True,
        capture_output=True,
        env=env,
    )
    if p.returncode != 0:
        print(p.stdout)
        print(p.stderr)
        raise RuntimeError(
            'Failed to download competition data. See logs above (you may need to accept the competition rules).'
        )

    for z in target_dir.glob('*.zip'):
        try:
            print(f'Unzipping {z.name}...')
            with zipfile.ZipFile(z, 'r') as zf:
                zf.extractall(target_dir)
            z.unlink()
        except Exception as e:
            print(f'Warning: failed to unzip {z}: {e}')

    os.environ['CAFA_DATASET_ROOT'] = str(target_dir)


_maybe_colab_auto_download_competition()

DATASET_ROOT = find_dataset_root(INPUT_ROOT, DATASET_SLUG)
print(f'DATASET_ROOT: {DATASET_ROOT}')

# Define input paths
PATH_IA = DATASET_ROOT / 'IA.tsv'
PATH_SAMPLE_SUB = DATASET_ROOT / 'sample_submission.tsv'
PATH_TRAIN_FASTA = DATASET_ROOT / 'Train' / 'train_sequences.fasta'
PATH_TRAIN_TERMS = DATASET_ROOT / 'Train' / 'train_terms.tsv'
PATH_TRAIN_TAXON = DATASET_ROOT / 'Train' / 'train_taxonomy.tsv'
PATH_GO_OBO = DATASET_ROOT / 'Train' / 'go-basic.obo'
PATH_TEST_FASTA = DATASET_ROOT / 'Test' / 'testsuperset.fasta'
PATH_TEST_TAXON = DATASET_ROOT / 'Test' / 'testsuperset-taxon-list.tsv'

# ------------------------------------------
# Sanity Checks
# ------------------------------------------

required = {
    'IA.tsv': PATH_IA,
    'Train/train_sequences.fasta': PATH_TRAIN_FASTA,
    'Train/train_terms.tsv': PATH_TRAIN_TERMS,
    'Train/go-basic.obo': PATH_GO_OBO,
}
missing = {k: v for k, v in required.items() if not v.exists()}
if missing:
    raise FileNotFoundError(f'Missing files: {missing}')
print('All required inputs found.')

# ------------------------------------------
# Fail-fast: FASTA readability checks (path issues)
# ------------------------------------------

def _fasta_smoke_test(path: Path, label: str, max_lines: int = 20000) -> None:
    path = Path(path)

    if not path.exists():
        raise FileNotFoundError(
            f"{label} FASTA not found: {path}\n"
            f"DATASET_ROOT={DATASET_ROOT}\n"
            "If you're on Colab, ensure the competition data downloaded/unzipped correctly,\n"
            "or set CAFA_DATASET_ROOT to the folder containing Train/ and Test/."
        )

    try:
        size = path.stat().st_size
    except Exception:
        size = None

    if size is not None and size == 0:
        raise RuntimeError(f"{label} FASTA file is empty (0 bytes): {path}")

    headers = 0
    first_nonempty = None
    try:
        with path.open('r', encoding='utf-8', errors='ignore') as f:
            for i, line in enumerate(f):
                s = line.strip()
                if not s:
                    continue
                if first_nonempty is None:
                    first_nonempty = s
                if s.startswith('>'):
                    headers += 1
                if i >= max_lines:
                    break
    except Exception as e:
        raise RuntimeError(f"Failed reading {label} FASTA at {path}: {type(e).__name__}: {e}")

    if first_nonempty is None:
        raise RuntimeError(f"{label} FASTA appears empty/unreadable (no non-empty lines): {path}")

    if not first_nonempty.startswith('>'):
        raise RuntimeError(
            f"{label} FASTA does not look like FASTA (first content line doesn't start with '>'): {path}\n"
            f"First line was: {first_nonempty[:120]!r}"
        )

    if headers == 0:
        raise RuntimeError(f"{label} FASTA had zero headers in the first {max_lines} lines: {path}")


print('FASTA smoke tests:')
_fasta_smoke_test(PATH_TRAIN_FASTA, 'Train')
print('  Train FASTA: OK')

if PATH_TEST_FASTA.exists():
    _fasta_smoke_test(PATH_TEST_FASTA, 'Test')
    print('  Test FASTA: OK')
else:
    print(f'  Test FASTA: MISSING at {PATH_TEST_FASTA} (continuing; some steps may fail later)')

# ------------------------------------------
# Checkpoint store (Kaggle Dataset = single source of truth)
# ------------------------------------------

def _get_secret(name: str) -> str:
    # Resolution order: env var -> Kaggle Secrets -> Colab userdata
    v = (os.environ.get(name, '') or '').strip()
    if v:
        return v

    try:
        from kaggle_secrets import UserSecretsClient  # type: ignore

        v = (UserSecretsClient().get_secret(name) or '').strip()
        if v:
            return v
    except Exception:
        pass

    try:
        from google.colab import userdata  # type: ignore

        v = (userdata.get(name) or '').strip()
        if v:
            return v
    except Exception:
        pass

    return ''


# Kaggle Secrets are NOT automatically environment variables.
# Resolve CAFA_FORCE_REBUILD via the same secret lookup and export it so later cells
# (which use os.getenv) behave consistently.
_raw_force_rebuild = (_get_secret('CAFA_FORCE_REBUILD') or '').strip()
if _raw_force_rebuild:
    os.environ.setdefault('CAFA_FORCE_REBUILD', _raw_force_rebuild)


def _truthy(v: str) -> bool:
    return str(v).strip().lower() in {'1', 'true', 'yes', 'y'}


print('CAFA_FORCE_REBUILD (env):', repr(os.environ.get('CAFA_FORCE_REBUILD', '')))
print('CAFA_FORCE_REBUILD (effective):', int(_truthy(os.environ.get('CAFA_FORCE_REBUILD', '0'))))


CHECKPOINT_DATASET_ID = (
    _get_secret('CAFA_CHECKPOINT_DATASET_ID')
    or _get_secret('CAFA_KAGGLE_DATASET_ID')
)
CHECKPOINT_DATASET_TITLE = os.environ.get('CAFA_CHECKPOINT_DATASET_TITLE', 'CAFA6 Checkpoints').strip()
CHECKPOINT_PULL = os.environ.get('CAFA_CHECKPOINT_PULL', '1').strip() == '1'
CHECKPOINT_PUSH = os.environ.get('CAFA_CHECKPOINT_PUSH', '1').strip() == '1'
MANIFEST_PATH = WORK_ROOT / 'manifest.json'


def _get_kaggle_token() -> str:
    return _get_secret('KAGGLE_API_TOKEN')


def _get_kaggle_user_key() -> tuple[str, str]:
    # Kaggle CLI expects Kaggle API credentials: username + key.
    username = _get_secret('KAGGLE_USERNAME')
    key = _get_secret('KAGGLE_KEY')
    if username and key:
        return username, key

    # Back-compat: allow KAGGLE_API_TOKEN to carry either JSON ({username,key}) or 'username:key'.
    tok = _get_kaggle_token()
    if tok:
        try:
            obj = json.loads(tok)
            username = (obj.get('username') or '').strip()
            key = (obj.get('key') or '').strip()
            if username and key:
                return username, key
        except Exception:
            pass

        if ':' in tok:
            u, k = tok.split(':', 1)
            username = u.strip()
            key = k.strip()
            if username and key:
                return username, key

    return '', ''


def _kaggle_env(require: bool = False) -> dict[str, str]:
    env = os.environ.copy()
    username, key = _get_kaggle_user_key()
    if username and key:
        env['KAGGLE_USERNAME'] = username
        env['KAGGLE_KEY'] = key

    if require and (not env.get('KAGGLE_USERNAME') or not env.get('KAGGLE_KEY')):
        raise RuntimeError(
            'Kaggle API auth missing. The `kaggle` CLI requires `KAGGLE_USERNAME` + `KAGGLE_KEY` '
            '(set them as env vars or Kaggle/Colab secrets). Or attach the '
            'checkpoint dataset as an Input so `STORE.pull()` can use the mounted copy.'
        )

    return env


def _ensure_kaggle_cli() -> None:
    try:
        subprocess.run(['kaggle', '--version'], check=True, capture_output=True, text=True)
    except Exception:
        subprocess.check_call([sys.executable, '-m', 'pip', '-q', 'install', 'kaggle'])
        subprocess.run(['kaggle', '--version'], check=True)


def _copy_merge(src: Path, dst: Path) -> None:
    src = Path(src)
    dst = Path(dst)

    for p in src.rglob('*'):
        if p.is_dir():
            continue
        rel = p.relative_to(src)
        out = dst / rel
        out.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(p, out)


def _maybe_unpack_dir_mode_zips(work_root: Path) -> None:
    # Kaggle CLI with `--dir-mode zip` uploads directories as `parsed.zip`, `external.zip`, etc.
    # We unpack them back into folders so the pipeline can resume from `WORK_ROOT/{parsed,external,features}/...`.
    work_root = Path(work_root)
    for name in ['parsed', 'external', 'features']:
        zpath = work_root / f'{name}.zip'
        if not zpath.exists():
            continue

        target_dir = work_root / name
        target_dir.mkdir(parents=True, exist_ok=True)
        needs_unpack = (not any(target_dir.rglob('*')))
        if not needs_unpack:
            # Folder already populated; keep the zip as-is (it might be a newer version).
            continue

        print(f'Unpacking checkpoint archive: {zpath} -> {target_dir}')
        with zipfile.ZipFile(zpath, 'r') as zf:
            zf.extractall(target_dir)
        try:
            zpath.unlink()
        except Exception:
            pass


def _load_manifest() -> dict:
    if MANIFEST_PATH.exists():
        try:
            return json.loads(MANIFEST_PATH.read_text(encoding='utf-8'))
        except Exception:
            return {}
    return {}


def _update_manifest(stage: str, required_paths: list[Path], note: str = '') -> None:
    m = _load_manifest()
    stages = m.get('stages', {})
    files = []

    for p in required_paths:
        p = Path(p)
        rel = str(p.relative_to(WORK_ROOT)) if str(p).startswith(str(WORK_ROOT)) else str(p)
        files.append({'path': rel, 'bytes': int(p.stat().st_size) if p.exists() else None})

    stages[stage] = {
        'ts_utc': time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime()),
        'note': note,
        'files': files,
    }

    m['stages'] = stages
    MANIFEST_PATH.write_text(json.dumps(m, indent=2), encoding='utf-8')


def _stage_files_signature(required_paths: list[Path]) -> list[dict]:
    sig = []
    for p in required_paths:
        p = Path(p)
        rel = str(p.relative_to(WORK_ROOT)) if str(p).startswith(str(WORK_ROOT)) else str(p)
        sig.append({'path': rel, 'bytes': int(p.stat().st_size) if p.exists() else None})
    return sorted(sig, key=lambda x: x['path'])


@dataclass
class KaggleCheckpointStore:
    work_root: Path
    dataset_id: str
    dataset_title: str
    pull_enabled: bool
    push_enabled: bool
    input_root: Path
    is_kaggle: bool

    @property
    def mount_dir(self) -> Path | None:
        if not self.is_kaggle or not self.dataset_id:
            return None
        slug = self.dataset_id.split('/')[-1]
        p = self.input_root / slug
        return p if p.exists() else None

    def pull(self) -> None:
        if not self.pull_enabled:
            print('Checkpoint pull disabled (CAFA_CHECKPOINT_PULL=0).')
            return

        checkpoint_required = str(os.environ.get('CAFA_CHECKPOINT_REQUIRED', '1')).strip().lower() in {'1', 'true', 'yes'}
        if not self.dataset_id:
            msg = 'Missing CAFA_CHECKPOINT_DATASET_ID=<user>/<slug>; cannot resume.'
            if checkpoint_required:
                raise ValueError(msg)
            print('WARNING: ' + msg)
            return

        if self.mount_dir is not None:
            print(f'Pulling checkpoints from Kaggle mounted dataset: {self.mount_dir}')
            _copy_merge(self.mount_dir, self.work_root)
            _maybe_unpack_dir_mode_zips(self.work_root)
            return

        print(f'Downloading checkpoints from Kaggle API: {self.dataset_id}')
        _ensure_kaggle_cli()
        env = _kaggle_env(require=checkpoint_required)

        if not env.get('KAGGLE_USERNAME') or not env.get('KAGGLE_KEY'):
            msg = (
                'Kaggle API auth missing. The `kaggle` CLI requires `KAGGLE_USERNAME` + `KAGGLE_KEY` '
                '(set them as env vars or Kaggle/Colab secrets). Either set them, or attach the '
                'checkpoint dataset as a Notebook Input so `STORE.pull()` can use the mounted copy.'
            )
            if checkpoint_required:
                raise RuntimeError(msg)
            print('WARNING: ' + msg)
            return

        tmp = self.work_root / '_tmp_kaggle_download'
        if tmp.exists():
            shutil.rmtree(tmp)
        tmp.mkdir(parents=True, exist_ok=True)

        p = subprocess.run(
            ['kaggle', 'datasets', 'download', '-d', self.dataset_id, '-p', str(tmp), '--unzip'],
            text=True,
            capture_output=True,
            env=env,
        )

        if p.returncode != 0:
            print(p.stdout)
            print(p.stderr)
            err = (p.stderr or '') + '\n' + (p.stdout or '')
            if '403' in err or 'Forbidden' in err:
                msg = (
                    'Checkpoint download was forbidden (HTTP 403). This almost always means the dataset is private '
                    'or not accessible from the current Kaggle account/runtime. Fix options:\n'
                    '  1) Attach the checkpoint dataset as a Kaggle notebook Input (fastest; no API call).\n'
                    '  2) Make the checkpoint dataset public (or share it with the account running the notebook).\n'
                    '  3) Ensure Secrets `KAGGLE_USERNAME`/`KAGGLE_KEY` belong to a user with access.'
                )
                if checkpoint_required:
                    raise RuntimeError(msg)
                print('WARNING: ' + msg)
                shutil.rmtree(tmp)
                return

            msg = 'Failed to download checkpoints from Kaggle. Check auth/network (see logs above).'
            if checkpoint_required:
                raise RuntimeError(msg)
            print('WARNING: ' + msg)
            shutil.rmtree(tmp)
            return

        _copy_merge(tmp, self.work_root)
        _maybe_unpack_dir_mode_zips(self.work_root)
        shutil.rmtree(tmp)

    def push(self, stage: str, required_paths: list[Path], note: str = '') -> None:
        if not self.push_enabled:
            print('Checkpoint push disabled (CAFA_CHECKPOINT_PUSH=0).')
            return

        if not self.dataset_id:
            raise ValueError('Missing CAFA_CHECKPOINT_DATASET_ID=<user>/<slug>; cannot checkpoint.')

        missing = [p for p in required_paths if not Path(p).exists()]
        if missing:
            raise FileNotFoundError(
                'Cannot checkpoint; missing required artefacts:\n' + '\n'.join([f' - {m}' for m in missing])
            )

        force_push = os.environ.get('CAFA_CHECKPOINT_FORCE_PUSH', '0').strip() == '1'
        if not force_push:
            m = _load_manifest()
            existing = (m.get('stages', {}) or {}).get(stage) if isinstance(m, dict) else None
            if isinstance(existing, dict):
                prev_files = existing.get('files', [])
                if isinstance(prev_files, list):
                    prev_sig = sorted(
                        [{'path': f.get('path'), 'bytes': f.get('bytes')} for f in prev_files if isinstance(f, dict)],
                        key=lambda x: str(x.get('path')),
                    )
                    cur_sig = _stage_files_signature(required_paths)
                    if prev_sig == cur_sig:
                        print(
                            f'Checkpoint stage {stage} unchanged; skipping publish '
                            '(set CAFA_CHECKPOINT_FORCE_PUSH=1 to force).'
                        )
                        return

        _update_manifest(stage, required_paths, note=note)

        # Publish WORK_ROOT directly (must not contain caches).
        (self.work_root / 'dataset-metadata.json').write_text(
            json.dumps({'title': self.dataset_title, 'id': self.dataset_id, 'licenses': [{'name': 'CC0-1.0'}]}, indent=2),
            encoding='utf-8',
        )
        (self.work_root / 'README.md').write_text(
            f'# {self.dataset_title}\n\nAuto-published checkpoint dataset for CAFA6.\n\nLatest stage: {stage}\n',
            encoding='utf-8',
        )

        _ensure_kaggle_cli()
        env = _kaggle_env(require=True)
        msg = f'{stage}: {note}'.strip() if note else stage

        # IMPORTANT: Kaggle CLI skips directories unless --dir-mode is set.
        p = subprocess.run(
            ['kaggle', 'datasets', 'version', '-p', str(self.work_root), '--dir-mode', 'zip', '-m', msg],
            text=True,
            capture_output=True,
            env=env,
        )

        if p.returncode != 0:
            # If dataset does not exist yet, create it.
            p2 = subprocess.run(
                ['kaggle', 'datasets', 'create', '-p', str(self.work_root), '--dir-mode', 'zip'],
                text=True,
                capture_output=True,
                env=env,
            )
            if p2.returncode != 0:
                print(p.stdout)
                print(p.stderr)
                print(p2.stdout)
                print(p2.stderr)
                raise RuntimeError('Kaggle dataset publish failed. See logs above.')
            print(p2.stdout)
            print(p2.stderr)
        else:
            print(p.stdout)
            print(p.stderr)
            print('Published new checkpoint dataset version:', self.dataset_id)


STORE = KaggleCheckpointStore(
    work_root=WORK_ROOT,
    dataset_id=CHECKPOINT_DATASET_ID,
    dataset_title=CHECKPOINT_DATASET_TITLE,
    pull_enabled=CHECKPOINT_PULL,
    push_enabled=CHECKPOINT_PUSH,
    input_root=INPUT_ROOT,
    is_kaggle=IS_KAGGLE,
)

# Pull once at startup (fresh runtimes resume here)
STORE.pull()


# Post-pull diagnostics: make it obvious whether artefacts are present (and whether dir-mode zips were unpacked).

def _p(path: Path) -> str:
    return str(path)


def _exists_bytes(path: Path) -> str:
    if not path.exists():
        return 'MISSING'
    try:
        return f'OK ({path.stat().st_size / (1024**2):.1f} MB)'
    except Exception:
        return 'OK'


print('Checkpoint status (after pull):')
print('  WORK_ROOT:', _p(WORK_ROOT))
print('  parsed/:', _exists_bytes(WORK_ROOT / 'parsed'))
print('  external/:', _exists_bytes(WORK_ROOT / 'external'))
print('  features/:', _exists_bytes(WORK_ROOT / 'features'))
print('  parsed.zip:', _exists_bytes(WORK_ROOT / 'parsed.zip'))
print('  external.zip:', _exists_bytes(WORK_ROOT / 'external.zip'))
print('  features.zip:', _exists_bytes(WORK_ROOT / 'features.zip'))
print('  external/entryid_text.tsv:', _exists_bytes(WORK_ROOT / 'external' / 'entryid_text.tsv'))
print('  parsed/train_seq.feather:', _exists_bytes(WORK_ROOT / 'parsed' / 'train_seq.feather'))


def stage_present(required_paths: list[Path]) -> bool:
    return all(Path(p).exists() for p in required_paths)


# ------------------------------------------
# Initial Diagnostics (Sequence Lengths)
# ------------------------------------------

%matplotlib inline
plt.rcParams.update({'font.size': 10})


def read_fasta_lengths(path: Path, max_records=20000):
    lengths = []
    current = 0
    n = 0
    with path.open('r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if n > 0:
                    lengths.append(current)
                n += 1
                current = 0
                if max_records and n > max_records:
                    break
            else:
                current += len(line)
        if n > 0:
            lengths.append(current)
    return np.array(lengths)


plt.figure(figsize=(10, 3))
plt.hist(read_fasta_lengths(PATH_TRAIN_FASTA), bins=50, alpha=0.5, label='Train')
if PATH_TEST_FASTA.exists():
    plt.hist(read_fasta_lengths(PATH_TEST_FASTA), bins=50, alpha=0.5, label='Test')
plt.title('Sequence Length Distribution (First 20k)')
plt.legend()
plt.show()


In [ ]:
# CELL 04 - Solution: 2. PHASE 1: DATA STRUCTURING & HIERARCHY
# 2. PHASE 1: DATA STRUCTURING & HIERARCHY
# ==========================================
# HARDWARE: CPU (Standard)
# ==========================================
# ------------------------------------------
# B. Parse OBO & Terms (needed in-memory downstream)
# ------------------------------------------

def parse_obo(path: Path):
    parents = {}
    namespaces = {}
    cur_id, cur_ns = None, None
    with path.open('r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == '[Term]':
                if cur_id and cur_ns:
                    namespaces[cur_id] = cur_ns
                cur_id, cur_ns = None, None
            elif line.startswith('id: GO:'):
                cur_id = line.split('id: ', 1)[1]
            elif line.startswith('namespace:'):
                cur_ns = line.split('namespace: ', 1)[1]
            elif line.startswith('is_a:') and cur_id:
                parent = line.split('is_a: ', 1)[1].split(' ! ')[0]
                parents.setdefault(cur_id, set()).add(parent)
        if cur_id and cur_ns:
            namespaces[cur_id] = cur_ns
    return parents, namespaces
print("Parsing OBO...")
go_parents, go_namespaces = parse_obo(PATH_GO_OBO)
print(f"GO Graph: {len(go_parents)} nodes with parents, {len(go_namespaces)} terms with namespace.")
# ------------------------------------------
# Milestone checkpoint: stage_01_parsed
# ------------------------------------------
parsed_dir = WORK_ROOT / 'parsed'
parsed_dir.mkdir(parents=True, exist_ok=True)
out_train_seq = parsed_dir / 'train_seq.feather'
out_test_seq = parsed_dir / 'test_seq.feather'
out_train_terms = parsed_dir / 'train_terms.parquet'
out_term_counts = parsed_dir / 'term_counts.parquet'
out_term_priors = parsed_dir / 'term_priors.parquet'
out_train_taxa = parsed_dir / 'train_taxa.feather'
out_test_taxa = parsed_dir / 'test_taxa.feather'
expected = [out_train_seq, out_train_terms, out_term_counts, out_term_priors, out_train_taxa]
if PATH_TEST_FASTA.exists():
    expected += [out_test_seq, out_test_taxa]
missing = [p for p in expected if not p.exists()]
if not missing:
    print("Parsed artefacts already exist; skipping Phase 1 writes.")
else:
    # ------------------------------------------
    # A. Parse FASTA to Feather
    # ------------------------------------------
    def parse_fasta(path: Path) -> pd.DataFrame:
        ids, seqs = [], []
        cur_id, cur_seq = None, []
        with path.open('r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line.startswith('>'):
                    if cur_id:
                        ids.append(cur_id)
                        seqs.append(''.join(cur_seq))
                    cur_id = line[1:].split()[0]
                    cur_seq = []
                else:
                    cur_seq.append(line)
            if cur_id:
                ids.append(cur_id)
                seqs.append(''.join(cur_seq))
        return pd.DataFrame({'id': ids, 'sequence': seqs})
    print("Parsing FASTA...")
    parse_fasta(PATH_TRAIN_FASTA).to_feather(out_train_seq)
    if PATH_TEST_FASTA.exists():
        parse_fasta(PATH_TEST_FASTA).to_feather(out_test_seq)
    print("FASTA parsed and saved to artefacts.")
    # ------------------------------------------
    # C. Process Terms & Priors
    # ------------------------------------------
    terms = pd.read_csv(PATH_TRAIN_TERMS, sep='\t')
    col_term = terms.columns[1]
    terms['aspect'] = terms[col_term].map(lambda x: go_namespaces.get(x, 'UNK'))
    # Plot Aspects
    plt.figure(figsize=(6, 3))
    terms['aspect'].value_counts().plot(kind='bar', title='Annotations by Namespace')
    plt.show()
    # Save Priors
    priors = (terms[col_term].value_counts() / terms.iloc[:, 0].nunique()).reset_index()
    priors.columns = ['term', 'prior']
    if PATH_IA.exists():
        ia = pd.read_csv(PATH_IA, sep='\t', names=['term', 'ia'])
        priors = priors.merge(ia, on='term', how='left').fillna(0)
    priors.to_parquet(out_term_priors)
    print("Terms processed and priors saved.")
    # ------------------------------------------
    # D. Process Taxonomy
    # ------------------------------------------
    print("Processing Taxonomy...")
    # Train Taxonomy
    tax_train = pd.read_csv(PATH_TRAIN_TAXON, sep='\t', header=None, names=['id', 'taxon_id'])
    tax_train['taxon_id'] = tax_train['taxon_id'].astype(int)
    tax_train.to_feather(out_train_taxa)
    # Test Taxonomy (Extract from FASTA headers)
    if PATH_TEST_FASTA.exists():
        ids, taxons = [], []
        with PATH_TEST_FASTA.open('r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line.startswith('>'):
                    parts = line[1:].split()
                    ids.append(parts[0])
                    # Assume second part is taxon if present
                    if len(parts) > 1:
                        try:
                            taxons.append(int(parts[1]))
                        except ValueError:
                            taxons.append(0)
                    else:
                        taxons.append(0)
        tax_test = pd.DataFrame({'id': ids, 'taxon_id': taxons})
        tax_test.to_feather(out_test_taxa)
        print(f"Taxonomy processed. Train: {len(tax_train)}, Test: {len(tax_test)}")
    else:
        print(f"Taxonomy processed. Train: {len(tax_train)}")
    # ------------------------------------------
    # E. Save Targets & Term List
    # ------------------------------------------
    print("Saving Targets & Term List...")
    # Save full terms list (long format)
    terms.to_parquet(out_train_terms)
    # Save unique term list with counts
    term_counts = terms['term'].value_counts().reset_index()
    term_counts.columns = ['term', 'count']
    term_counts.to_parquet(out_term_counts)
    print("Targets saved.")
    if 'STORE' in globals() and STORE is not None:
        STORE.push('stage_01_parsed', [p for p in expected if p.exists()], note='parsed FASTA/taxa/terms/priors')

In [ ]:
# CELL 05 - Inline: EntryID->text corpus builder
import argparse
import csv
import os
import re
import time
import xml.etree.ElementTree as ET
from dataclasses import dataclass
from pathlib import Path
import pandas as pd
import requests
from tqdm import tqdm

def _norm_uniprot_accession(raw_id: str) -> str:
    s = str(raw_id).strip()
    parts = s.split("|")
    # Common UniProt FASTA header: sp|P12345|NAME_HUMAN
    if len(parts) >= 2 and parts[0] in {"sp", "tr"}:
        return parts[1]
    if len(parts) >= 3:
        return parts[1]
    return s
def _detect_id_col(df: pd.DataFrame) -> str:
    for cand in ["id", "EntryID", "entry_id", "accession"]:
        if cand in df.columns:
            return cand
    return df.columns[0]
def _read_ids(feather_path: Path) -> list[str]:
    df = pd.read_feather(feather_path)
    col = _detect_id_col(df)
    return df[col].astype(str).tolist()
def _strip_go_leakage(text: str) -> str:
    # Remove explicit GO identifiers and obvious "GO:" tokens.
    text = re.sub(r"\bGO:\d{7}\b", " ", text)
    text = text.replace("GO:", " ")
    return re.sub(r"\s+", " ", text).strip()
def _make_session(total_retries: int = 5) -> requests.Session:
    from requests.adapters import HTTPAdapter
    from urllib3.util.retry import Retry
    retry = Retry(
        total=total_retries,
        backoff_factor=0.5,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"],
        raise_on_status=False,
    )
    s = requests.Session()
    s.mount("https://", HTTPAdapter(max_retries=retry))
    return s
@dataclass

class UniProtRow:
    accession: str
    protein_name: str
    organism_name: str
    keywords: str
    protein_families: str
    cc_function: str
    cc_subcellular_location: str
    pubmed_ids: list[str]
def fetch_uniprot_tsv(
    session: requests.Session,
    accessions: list[str],
    sleep_s: float,
    fields: str,
) -> dict[str, UniProtRow]:
    """Fetch UniProt rows for a small batch of accessions using the search endpoint (TSV)."""
    # Query is a disjunction of accessions.
    # Keep the batch modest to avoid URL length limits.
    query = " OR ".join([f"accession:{a}" for a in accessions])
    url = "https://rest.uniprot.org/uniprotkb/search"
    params = {
        "query": f"({query})",
        "format": "tsv",
        "fields": fields,
        "size": 500,
    }
    resp = session.get(url, params=params, timeout=60)
    # UniProt will return HTTP 400 for overly long/complex queries (often URL-length related).
    # Fall back to a recursive split so large runs complete reliably.
    if resp.status_code in {400, 414} and len(accessions) > 1:
        mid = len(accessions) // 2
        left = fetch_uniprot_tsv(session, accessions[:mid], sleep_s=sleep_s, fields=fields)
        right = fetch_uniprot_tsv(session, accessions[mid:], sleep_s=sleep_s, fields=fields)
        left.update(right)
        return left
    try:
        resp.raise_for_status()
    except requests.HTTPError as e:
        detail = (resp.text or '')[:1000] if resp is not None else ''
        raise requests.HTTPError(f'{e} :: UniProt response: {detail}') from e
    if sleep_s:
        time.sleep(sleep_s)
    lines = resp.text.splitlines()
    if not lines:
        return {}
    reader = csv.DictReader(lines, delimiter="\t")
    out: dict[str, UniProtRow] = {}
    for row in reader:
        acc = (row.get("Entry") or row.get("accession") or "").strip()
        if not acc:
            continue
        def _get(k: str) -> str:
            v = row.get(k)
            return str(v).strip() if v is not None else ""
        pmids_raw = (_get("PubMed ID") or _get("lit_pubmed_id")).strip()
        pmids: list[str] = []
        if pmids_raw:
            pmids = [p.strip() for p in re.split(r"[;\s]+", pmids_raw) if p.strip().isdigit()]
        out[acc] = UniProtRow(
            accession=acc,
            protein_name=_get("Protein names"),
            organism_name=_get("Organism"),
            keywords=_get("Keywords"),
            protein_families=_get("Protein families"),
            cc_function=_get("Function [CC]"),
            cc_subcellular_location=_get("Subcellular location [CC]"),
            pubmed_ids=pmids,
        )
    return out
def fetch_pubmed_abstracts(
    session: requests.Session,
    pmids: list[str],
    sleep_s: float,
    email: str,
    api_key: str,
    _depth: int = 0,
) -> dict[str, tuple[str, str]]:
    """Fetch PubMed title+abstract via NCBI E-utilities (XML)."""
    if not pmids:
        return {}
    pmids = [str(p).strip() for p in pmids if str(p).strip().isdigit()]
    if not pmids:
        return {}
    pmids = list(dict.fromkeys(pmids))

    def _looks_like_html(text: str) -> bool:
        t = (text or "").lstrip().lower()
        return t.startswith("<!doctype html") or t.startswith("<html") or "<html" in t[:2000]

    def _sanitize_xml(text: str) -> str:
        # Remove illegal XML control chars (keep \t \n \r).
        return re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F]", " ", text or "")

    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {
        "db": "pubmed",
        "id": ",".join(pmids),
        "retmode": "xml",
    }
    if email:
        params["email"] = email
    if api_key:
        params["api_key"] = api_key

    root = None
    for attempt in range(3):
        try:
            resp = session.get(url, params=params, timeout=60)
            if resp.status_code in {429, 500, 502, 503, 504}:
                raise requests.HTTPError(f"PubMed transient HTTP {resp.status_code}")
            resp.raise_for_status()
            if sleep_s:
                time.sleep(sleep_s)
            text = resp.text or ""
            if _looks_like_html(text):
                snippet = text[:300].replace("\n", " " )
                raise ValueError(f"PubMed returned HTML instead of XML: {snippet}")
            text = _sanitize_xml(text)
            root = ET.fromstring(text)
            break
        except Exception:
            if attempt < 2:
                time.sleep(1.5 * (attempt + 1))
            continue

    if root is None:
        if len(pmids) > 1 and _depth < 10:
            mid = len(pmids) // 2
            left = fetch_pubmed_abstracts(session, pmids[:mid], sleep_s=sleep_s, email=email, api_key=api_key, _depth=_depth + 1)
            right = fetch_pubmed_abstracts(session, pmids[mid:], sleep_s=sleep_s, email=email, api_key=api_key, _depth=_depth + 1)
            left.update(right)
            return left
        return {}

    out: dict[str, tuple[str, str]] = {}
    for article in root.findall(".//PubmedArticle"):
        pmid_el = article.find(".//PMID")
        if pmid_el is None or pmid_el.text is None:
            continue
        pmid = pmid_el.text.strip()
        title_el = article.find(".//ArticleTitle")
        abs_el = article.find(".//Abstract")
        title = ("" if title_el is None or title_el.text is None else title_el.text.strip())
        abstract = ""
        if abs_el is not None:
            parts = []
            for t in abs_el.findall(".//AbstractText"):
                if t.text:
                    parts.append(t.text.strip())
            abstract = " ".join(parts).strip()
        out[pmid] = (title, abstract)
    return out
def main() -> int:
    p = argparse.ArgumentParser(description="Build EntryID->text corpus from UniProt (fields) + PubMed abstracts for --mode text.")
    p.add_argument("--artefacts-dir", default=Path("artefacts_local") / "artefacts", type=Path, help="Artefacts root")
    p.add_argument("--out-path", default=Path("artefacts_local") / "artefacts" / "external" / "entryid_text.tsv", type=Path, help="Output TSV path (default: artefacts_local/artefacts/external/entryid_text.tsv)")
    p.add_argument("--cache-dir", default=Path("artefacts_local") / "artefacts" / "external" / "uniprot_pubmed_cache", type=Path, help="Cache dir for UniProt+PubMed lookups")
    p.add_argument("--max-ids", type=int, default=0, help="If >0, cap number of proteins (debug)")
    p.add_argument("--uniprot-batch-size", type=int, default=80)
    p.add_argument("--pubmed-batch-size", type=int, default=50)
    p.add_argument("--max-pubmed-per-protein", type=int, default=3)
    p.add_argument("--max-abstract-chars", type=int, default=2000)
    p.add_argument("--sleep-uniprot", type=float, default=0.1)
    p.add_argument("--sleep-pubmed", type=float, default=0.34)
    p.add_argument("--email", type=str, default=os.environ.get("NCBI_EMAIL", ""))
    p.add_argument("--api-key", type=str, default=os.environ.get("NCBI_API_KEY", ""))
    p.add_argument("--strip-go", action="store_true")
    args = p.parse_args()
    args.artefacts_dir = Path(args.artefacts_dir)
    args.out_path = Path(args.out_path)
    args.cache_dir = Path(args.cache_dir)
    args.out_path.parent.mkdir(parents=True, exist_ok=True)
    args.cache_dir.mkdir(parents=True, exist_ok=True)
    train_feather = args.artefacts_dir / "parsed" / "train_seq.feather"
    test_feather = args.artefacts_dir / "parsed" / "test_seq.feather"
    ids = []
    if train_feather.exists():
        ids.extend(_read_ids(train_feather))
    if test_feather.exists():
        ids.extend(_read_ids(test_feather))
    ids = [_norm_uniprot_accession(i) for i in ids]
    uniq = sorted(set(ids))
    if args.max_ids and args.max_ids > 0:
        uniq = uniq[: int(args.max_ids)]
    print(f"Unique accessions: {len(uniq)}")
    fields = ",".join(
        [
            "accession",
            "protein_name",
            "organism_name",
            "keyword",
            "protein_families",
            "cc_function",
            "cc_subcellular_location",
            "lit_pubmed_id",
        ]
    )
    session = _make_session()
    # UniProt rows cache
    uniprot_cache = args.cache_dir / "uniprot_rows.tsv"
    uniprot_rows: dict[str, UniProtRow] = {}
    if uniprot_cache.exists():
        print(f"Loading UniProt cache: {uniprot_cache}")
        df_u = pd.read_csv(uniprot_cache, sep="\t", dtype=str)
        for _, r in df_u.iterrows():
            acc = str(r["accession"])
            pmids = [p for p in str(r.get("pubmed_ids", "")).split(";") if p]
            uniprot_rows[acc] = UniProtRow(
                accession=acc,
                protein_name=str(r.get("protein_name", "")) if r.get("protein_name") is not None else "",
                organism_name=str(r.get("organism_name", "")) if r.get("organism_name") is not None else "",
                keywords=str(r.get("keywords", "")) if r.get("keywords") is not None else "",
                protein_families=str(r.get("protein_families", "")) if r.get("protein_families") is not None else "",
                cc_function=str(r.get("cc_function", "")) if r.get("cc_function") is not None else "",
                cc_subcellular_location=str(r.get("cc_subcellular_location", "")) if r.get("cc_subcellular_location") is not None else "",
                pubmed_ids=pmids,
            )
    missing = [a for a in uniq if a not in uniprot_rows]
    if missing:
        print(f"Fetching UniProt rows (missing={len(missing)})...")
        write_header = not uniprot_cache.exists()
        with uniprot_cache.open("a", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(
                f,
                delimiter="\t",
                fieldnames=[
                    "accession",
                    "protein_name",
                    "organism_name",
                    "keywords",
                    "protein_families",
                    "cc_function",
                    "cc_subcellular_location",
                    "pubmed_ids",
                ],
            )
            if write_header:
                writer.writeheader()
            for i in tqdm(range(0, len(missing), args.uniprot_batch_size), desc="UniProt"):
                batch = missing[i : i + args.uniprot_batch_size]
                got = fetch_uniprot_tsv(
                    session=session,
                    accessions=batch,
                    sleep_s=args.sleep_uniprot,
                    fields=fields,
                )
                for acc, r in got.items():
                    uniprot_rows[acc] = r
                    writer.writerow(
                        {
                            "accession": r.accession,
                            "protein_name": r.protein_name,
                            "organism_name": r.organism_name,
                            "keywords": r.keywords,
                            "protein_families": r.protein_families,
                            "cc_function": r.cc_function,
                            "cc_subcellular_location": r.cc_subcellular_location,
                            "pubmed_ids": ";".join(r.pubmed_ids),
                        }
                    )
    # PubMed cache
    pubmed_cache = args.cache_dir / "pubmed_abstracts.tsv"
    pmid_sets: dict[str, list[str]] = {}
    for acc, r in uniprot_rows.items():
        pmids = [p for p in r.pubmed_ids if p]
        if args.max_pubmed_per_protein and len(pmids) > args.max_pubmed_per_protein:
            pmids = pmids[: int(args.max_pubmed_per_protein)]
        pmid_sets[acc] = pmids
    pmids_uniq = sorted({p for ps in pmid_sets.values() for p in ps})
    pubmed_map: dict[str, tuple[str, str]] = {}
    if pubmed_cache.exists():
        print(f"Loading PubMed cache: {pubmed_cache}")
        df_p = pd.read_csv(pubmed_cache, sep="\t", dtype=str)
        for _, r in df_p.iterrows():
            pmid = str(r["pmid"])
            pubmed_map[pmid] = (
                str(r.get("title", "")) if r.get("title") is not None else "",
                str(r.get("abstract", "")) if r.get("abstract") is not None else "",
            )
    missing_pmids = [p for p in pmids_uniq if p not in pubmed_map]
    if missing_pmids:
        print(f"Fetching PubMed abstracts (missing={len(missing_pmids)})...")
        write_header = not pubmed_cache.exists()
        with pubmed_cache.open("a", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, delimiter="\t", fieldnames=["pmid", "title", "abstract"])
            if write_header:
                writer.writeheader()
            for i in tqdm(range(0, len(missing_pmids), args.pubmed_batch_size), desc="PubMed"):
                batch = missing_pmids[i : i + args.pubmed_batch_size]
                got = fetch_pubmed_abstracts(
                    session=session,
                    pmids=batch,
                    sleep_s=args.sleep_pubmed,
                    email=args.email,
                    api_key=args.api_key,
                )
                for pmid, (title, abstract) in got.items():
                    if args.max_abstract_chars and abstract:
                        abstract = abstract[: int(args.max_abstract_chars)]
                    pubmed_map[pmid] = (title, abstract)
                    writer.writerow({"pmid": pmid, "title": title, "abstract": abstract})
    # Build final EntryID -> text
    print("Writing entryid_text.tsv...")
    with args.out_path.open("w", newline="", encoding="utf-8") as f:
        w = csv.writer(f, delimiter="\t")
        w.writerow(["EntryID", "text"])
        n_with_uniprot = 0
        n_with_pubmed = 0
        for acc in tqdm(uniq, desc="Assemble"):
            r = uniprot_rows.get(acc)
            if r is None:
                w.writerow([acc, ""])
                continue
            parts: list[str] = []
            # UniProt text (short, curated)
            uniprot_bits = [
                r.protein_name,
                r.organism_name,
                r.keywords,
                r.protein_families,
                r.cc_function,
                r.cc_subcellular_location,
            ]
            uniprot_text = " ".join([b for b in uniprot_bits if b])
            if uniprot_text:
                n_with_uniprot += 1
                parts.append(uniprot_text)
            # PubMed abstracts (richer)
            pmids = pmid_sets.get(acc, [])
            abs_parts: list[str] = []
            for pmid in pmids:
                title, abstract = pubmed_map.get(pmid, ("", ""))
                if title or abstract:
                    abs_parts.append(f"{title}. {abstract}".strip(" ."))
            if abs_parts:
                n_with_pubmed += 1
                parts.append(" ".join(abs_parts))
            text = " ".join(parts).strip()
            if args.strip_go and text:
                text = _strip_go_leakage(text)
            w.writerow([acc, text])
    print(f"Saved: {args.out_path}")
    print(f"Coverage: UniProt text {n_with_uniprot}/{len(uniq)} = {n_with_uniprot/ max(1,len(uniq)):.3f}")
    print(f"Coverage: PubMed text  {n_with_pubmed}/{len(uniq)} = {n_with_pubmed/ max(1,len(uniq)):.3f}")
    return 0
# Notebook safety:
# Jupyter/IPython runs with __name__ == '__main__', which would trigger argparse via main() and fail
# due to extra kernel arguments (e.g. '-f <kernel.json>').
# This cell is intentionally library-only; run the next notebook cell to execute it.
print('EntryID→text corpus builder loaded. Run Notebook Cell 7 to build WORK_ROOT/external/entryid_text.tsv.')

In [ ]:
# CELL 06 - Run: build EntryID->text corpus (UniProt+PubMed)
# Milestone checkpoint: stage_02_external_text
import os
import sys
from pathlib import Path

print('Stage stage_02_external_text: start')
out_path = WORK_ROOT / 'external' / 'entryid_text.tsv'
cache_dir = CACHE_ROOT / 'uniprot_pubmed_cache'

FORCE_REBUILD = os.getenv('CAFA_FORCE_REBUILD', '0').strip().lower() not in {'0', '', 'false', 'no'}
print(f'  out_path: {out_path} (exists={out_path.exists()})')
print(f'  CAFA_FORCE_REBUILD: {FORCE_REBUILD}')

if out_path.exists() and not FORCE_REBUILD:
    print(f'entryid_text.tsv already exists at {out_path}; skipping corpus build (set CAFA_FORCE_REBUILD=1 to rebuild).')
else:
    cache_dir.mkdir(parents=True, exist_ok=True)
    sys.argv = [
        '03_build_entryid_text_from_uniprot_pubmed.py',
        '--artefacts-dir', str(WORK_ROOT),
        '--out-path', str(out_path),
        '--cache-dir', str(cache_dir),
        '--max-ids', '0',
        '--uniprot-batch-size', '80',
        '--max-pubmed-per-protein', '3',
        '--strip-go',
        '--sleep-uniprot', '0.1',
        '--sleep-pubmed', '0.34',
    ]
    _ = main()

if out_path.exists():
    print('Stage stage_02_external_text: attempting checkpoint push (may be skipped if unchanged).')
    STORE.push('stage_02_external_text', [out_path], note='entryid->text corpus')


In [ ]:
# CELL 07 - Inline: embeddings generator (used by TF-IDF; also supports extras)
print('Cell 7: embeddings generator loaded')
import argparse
import os
from pathlib import Path
import numpy as np
import pandas as pd
import torch

def _assert_all_finite_np(arr: np.ndarray, *, name: str) -> None:
    if not np.isfinite(arr).all():
        nan = int(np.isnan(arr).sum())
        inf = int(np.isinf(arr).sum())
        raise ValueError(f'{name}: non-finite values detected (nan={nan}, inf={inf}). Refusing to save.')

def _read_sequences(feather_path: Path) -> tuple[list[str], list[str]]:
    df = pd.read_feather(feather_path)
    if "id" not in df.columns or "sequence" not in df.columns:
        raise ValueError(f"Expected columns id, sequence in {feather_path}; got {list(df.columns)}")
    ids = df["id"].astype(str).tolist()
    seqs = df["sequence"].astype(str).tolist()
    return ids, seqs
def _smart_order(seqs: list[str]) -> np.ndarray:
    lengths = np.fromiter((len(s) for s in seqs), dtype=np.int64)
    return np.argsort(lengths)[::-1]
def _mean_pool(last_hidden_state: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
    # last_hidden_state: (B, L, D), attention_mask: (B, L)
    mask = attention_mask.unsqueeze(-1).to(last_hidden_state.dtype)
    x = last_hidden_state * mask
    denom = mask.sum(dim=1).clamp(min=1.0)
    return x.sum(dim=1) / denom
@torch.no_grad()
def embed_esm2(
    seqs: list[str],
    model_name: str,
    batch_size: int,
    max_len: int,
    device: torch.device,
) -> np.ndarray:
    from transformers import EsmModel, EsmTokenizer
    tok = EsmTokenizer.from_pretrained(model_name)
    model = EsmModel.from_pretrained(model_name).to(device)
    model.eval()
    order = _smart_order(seqs)
    seqs_sorted = [seqs[i] for i in order]
    outs: list[np.ndarray] = []
    use_amp = device.type == "cuda"
    for i in range(0, len(seqs_sorted), batch_size):
        batch = seqs_sorted[i : i + batch_size]
        ids = tok.batch_encode_plus(
            batch,
            add_special_tokens=True,
            padding="longest",
            truncation=True,
            max_length=max_len,
            return_tensors="pt",
        )
        ids = {k: v.to(device) for k, v in ids.items()}
        ctx = torch.amp.autocast("cuda") if use_amp else torch.autocast("cpu", enabled=False)
        with ctx:
            out = model(**ids)
        pooled = _mean_pool(out.last_hidden_state.float(), ids["attention_mask"])
        outs.append(pooled.cpu().numpy())
        if device.type == "cuda":
            torch.cuda.empty_cache()
    embs_sorted = np.vstack(outs).astype(np.float32)
    embs = np.zeros_like(embs_sorted)
    embs[order] = embs_sorted
    return embs
@torch.no_grad()
def embed_ankh(
    seqs: list[str],
    model_name: str,
    batch_size: int,
    max_len: int,
    device: torch.device,
    trust_remote_code: bool,
) -> np.ndarray:
    # Ankh models are HuggingFace-hosted and may require trust_remote_code.
    # We treat them as encoder-style models and mean-pool hidden states over attention_mask.
    from transformers import AutoModel, AutoTokenizer
    tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=trust_remote_code)
    model = AutoModel.from_pretrained(model_name, trust_remote_code=trust_remote_code).to(device)
    model.eval()
    order = _smart_order(seqs)
    seqs_sorted = [seqs[i] for i in order]
    outs: list[np.ndarray] = []
    amp_enabled = device.type == "cuda"
    for i in range(0, len(seqs_sorted), batch_size):
        batch = seqs_sorted[i : i + batch_size]
        # Many protein LMs expect space-separated amino acids; if the tokenizer has a small vocab,
        # this typically helps. If it hurts, you can disable via --ankh-space-sep 0.
        batch = [" ".join(list(s.replace("U", "X").replace("Z", "X").replace("O", "X").replace("B", "X"))) for s in batch]
        ids = tok(
            batch,
            add_special_tokens=True,
            padding=True,
            truncation=True,
            max_length=max_len,
            return_tensors="pt",
        )
        ids = {k: v.to(device) for k, v in ids.items()}
        if "attention_mask" not in ids:
            raise RuntimeError(f"Tokenizer for {model_name} did not return attention_mask")
        if (ids["attention_mask"].sum(dim=1) == 0).any():
            raise RuntimeError(f"Ankh tokenisation produced an all-zero attention_mask (batch_start={i}).")

        def _forward(use_amp: bool):
            ctx = (
                torch.amp.autocast("cuda")
                if (use_amp and device.type == "cuda")
                else torch.autocast("cpu", enabled=False)
            )
            with ctx:
                return model(**ids)

        out = _forward(amp_enabled)
        # HuggingFace convention: encoder outputs `last_hidden_state`
        last = getattr(out, "last_hidden_state", None)
        if last is None:
            raise RuntimeError(f"Model {model_name} did not return last_hidden_state")
        pooled = _mean_pool(last.float(), ids["attention_mask"])
        if not bool(torch.isfinite(pooled).all().item()):
            if amp_enabled:
                print(f"WARNING: Non-finite Ankh embeddings under AMP at batch_start={i}; retrying without AMP.")
                amp_enabled = False
                out = _forward(False)
                last = getattr(out, "last_hidden_state", None)
                if last is None:
                    raise RuntimeError(f"Model {model_name} did not return last_hidden_state")
                pooled = _mean_pool(last.float(), ids["attention_mask"])
            if not bool(torch.isfinite(pooled).all().item()):
                raise RuntimeError(f"Ankh produced non-finite embeddings even without AMP (batch_start={i}).")
        outs.append(pooled.cpu().numpy())
        if device.type == "cuda":
            torch.cuda.empty_cache()
    embs_sorted = np.vstack(outs).astype(np.float32)
    _assert_all_finite_np(embs_sorted, name="ankh_embeds")
    embs = np.zeros_like(embs_sorted)
    embs[order] = embs_sorted
    return embs
def main() -> int:
    ap = argparse.ArgumentParser(description="Generate optional multimodal embeddings as .npy artefacts.")
    ap.add_argument(
        "--artefacts-dir",
        type=Path,
        default=Path("artefacts_local") / "artefacts",
        help="Path containing parsed/ and features/ (default: artefacts_local/artefacts)",
    )
    ap.add_argument(
        "--mode",
        choices=["esm2_3b", "ankh", "text"],
        required=True,
        help="Which optional embedding to generate.",
    )
    ap.add_argument(
        "--device",
        type=str,
        default="cuda",
        help="cuda|cpu (default: cuda)",
    )
    ap.add_argument("--batch-size", type=int, default=2)
    ap.add_argument("--max-len", type=int, default=1024)
    ap.add_argument(
        "--esm2-3b-model",
        type=str,
        default="facebook/esm2_t36_3B_UR50D",
        help="HF model id for ESM2-3B",
    )
    ap.add_argument(
        "--ankh-model",
        type=str,
        default="ElnaggarLab/ankh-large",
        help="HF model id for Ankh (large is typically 1536D)",
    )
    ap.add_argument(
        "--trust-remote-code",
        action="store_true",
        help="Pass trust_remote_code=True for Ankh loading (often required).",
    )
    # Text mode (fixed-width sparse->dense features; default dimension = 10279)
    ap.add_argument(
        "--text-path",
        type=Path,
        default=None,
        help="Path to a 2-column TSV/CSV with EntryID and text (required for --mode text).",
    )
    ap.add_argument(
        "--text-sep",
        type=str,
        default="\t",
        help="Separator for --text-path (default: tab).",
    )
    ap.add_argument(
        "--text-id-col",
        type=str,
        default="EntryID",
        help="ID column name in --text-path (default: EntryID).",
    )
    ap.add_argument(
        "--text-col",
        type=str,
        default="text",
        help="Text column name in --text-path (default: text).",
    )
    ap.add_argument(
        "--text-dim",
        type=int,
        default=10279,
        help="Output feature dimension for text TF-IDF (default: 10279).",
    )
    ap.add_argument(
        "--text-dtype",
        choices=["float16", "float32"],
        default="float16",
        help="On-disk dtype for text .npy (default: float16 to keep size sane).",
    )
    ap.add_argument(
        "--text-ngram-max",
        type=int,
        default=2,
        help="Max n-gram for TF-IDF (default: 2).",
    )
    args = ap.parse_args()
    artefacts_dir: Path = args.artefacts_dir
    parsed_dir = artefacts_dir / "parsed"
    feat_dir = artefacts_dir / "features"
    feat_dir.mkdir(parents=True, exist_ok=True)
    train_feather = parsed_dir / "train_seq.feather"
    test_feather = parsed_dir / "test_seq.feather"
    if not train_feather.exists() or not test_feather.exists():
        raise FileNotFoundError(
            f"Expected {train_feather} and {test_feather}. Run Phase 1 parsing first."
        )
    # Make HF caches relocatable (handy on Colab/Kaggle)
    os.environ.setdefault("HF_HOME", str(artefacts_dir / "hf_cache"))
    os.environ.setdefault("TRANSFORMERS_CACHE", str(artefacts_dir / "hf_cache"))
    os.environ.setdefault("TORCH_HOME", str(artefacts_dir / "torch_cache"))
    device = torch.device("cuda" if (args.device == "cuda" and torch.cuda.is_available()) else "cpu")
    print(f"Device: {device}")
    print("Loading sequences...")
    train_ids, train_seqs = _read_sequences(train_feather)
    test_ids, test_seqs = _read_sequences(test_feather)
    if args.mode == "esm2_3b":
        print(f"Embedding ESM2-3B: {args.esm2_3b_model}")
        train_emb = embed_esm2(train_seqs, args.esm2_3b_model, args.batch_size, args.max_len, device)
        test_emb = embed_esm2(test_seqs, args.esm2_3b_model, args.batch_size, args.max_len, device)
        np.save(feat_dir / "train_embeds_esm2_3b.npy", train_emb)
        np.save(feat_dir / "test_embeds_esm2_3b.npy", test_emb)
        print(f"Saved: {feat_dir / 'train_embeds_esm2_3b.npy'}")
        print(f"Saved: {feat_dir / 'test_embeds_esm2_3b.npy'}")
        return 0
    if args.mode == "ankh":
        print(f"Embedding Ankh: {args.ankh_model}")
        train_emb = embed_ankh(
            train_seqs,
            args.ankh_model,
            args.batch_size,
            args.max_len,
            device,
            trust_remote_code=args.trust_remote_code,
        )
        test_emb = embed_ankh(
            test_seqs,
            args.ankh_model,
            args.batch_size,
            args.max_len,
            device,
            trust_remote_code=args.trust_remote_code,
        )
        np.save(feat_dir / "train_embeds_ankh.npy", train_emb)
        np.save(feat_dir / "test_embeds_ankh.npy", test_emb)
        print(f"Saved: {feat_dir / 'train_embeds_ankh.npy'}")
        print(f"Saved: {feat_dir / 'test_embeds_ankh.npy'}")
        return 0
    if args.mode == "text":
        if args.text_path is None:
            raise ValueError("--text-path is required for --mode text")
        from sklearn.feature_extraction.text import TfidfVectorizer
        import joblib
        if not args.text_path.exists():
            raise FileNotFoundError(f"text-path not found: {args.text_path}")
        print(f"Loading text corpus: {args.text_path}")
        df = pd.read_csv(args.text_path, sep=args.text_sep, dtype=str)
        if args.text_id_col not in df.columns or args.text_col not in df.columns:
            raise ValueError(
                f"Expected columns {args.text_id_col!r}, {args.text_col!r} in {args.text_path}; got {list(df.columns)}"
            )
        # Many-to-one mapping is possible (multiple pubs per EntryID). Concatenate.
        df = df[[args.text_id_col, args.text_col]].dropna()
        df[args.text_id_col] = df[args.text_id_col].astype(str)
        df[args.text_col] = df[args.text_col].astype(str)
        grouped = df.groupby(args.text_id_col, sort=False)[args.text_col].apply(lambda x: " \n ".join(x.tolist()))
        text_map = grouped.to_dict()
        def _norm_uniprot_id(pid: str) -> str:
            # Common UniProt FASTA headers: sp|P12345|NAME_HUMAN ...
            parts = str(pid).split("|")
            if len(parts) >= 2 and parts[0] in {"sp", "tr"}:
                return parts[1]
            if len(parts) >= 3:
                return parts[1]
            return str(pid)
        # Support both raw IDs and normalised accessions for joining.
        text_map_norm: dict[str, str] = {}
        for k, v in text_map.items():
            text_map_norm[str(k)] = v
            text_map_norm[_norm_uniprot_id(k)] = v
        train_texts = [text_map_norm.get(str(pid), "") for pid in train_ids]
        test_texts = [text_map_norm.get(str(pid), "") for pid in test_ids]
        # TF-IDF gives a fixed-width, high-dimensional text modality with controllable size.
        # This is the most practical way to realise a 10279D "text embedding" without a bespoke LLM pipeline.
        print(f"Fitting TF-IDF (dim={args.text_dim}, ngram<= {args.text_ngram_max})...")
        vec = TfidfVectorizer(
            max_features=args.text_dim,
            ngram_range=(1, max(1, int(args.text_ngram_max))),
            min_df=2,
            strip_accents="unicode",
            lowercase=True,
        )
        vec.fit(train_texts + test_texts)
        n_features = len(vec.get_feature_names_out())
        print(f"TF-IDF vocab size: {n_features} (padded to {args.text_dim})")
        # Persist vectorizer for reproducibility
        joblib.dump(vec, feat_dir / "text_vectorizer.joblib")
        print(f"Saved: {feat_dir / 'text_vectorizer.joblib'}")
        out_dtype = np.float16 if args.text_dtype == "float16" else np.float32
        def _write_memmap(name: str, texts: list[str]):
            path = feat_dir / name
            mm = np.lib.format.open_memmap(path, mode="w+", dtype=out_dtype, shape=(len(texts), args.text_dim))
            bs = 2048
            for i in range(0, len(texts), bs):
                chunk = texts[i : i + bs]
                X = vec.transform(chunk)  # sparse
                # Convert per-chunk to dense to write to .npy
                arr = X.toarray().astype(out_dtype, copy=False)
                if arr.shape[1] == args.text_dim:
                    mm[i : i + arr.shape[0], :] = arr
                else:
                    dense = np.zeros((arr.shape[0], args.text_dim), dtype=out_dtype)
                    if arr.shape[1] > 0:
                        dense[:, : arr.shape[1]] = arr
                    mm[i : i + dense.shape[0], :] = dense
            mm.flush()
            return path
        train_path = _write_memmap("train_embeds_text.npy", train_texts)
        test_path = _write_memmap("test_embeds_text.npy", test_texts)
        print(f"Saved: {train_path}")
        print(f"Saved: {test_path}")
        return 0
    raise RuntimeError("unreachable")
# Notebook safety:
# This cell is intentionally library-only (no auto-run).
# Jupyter passes extra kernel args (e.g. '-f <kernel.json>') that break argparse-based mains.
# Use the dedicated run cell that sets sys.argv explicitly.

In [ ]:
# CELL 08 - Run: TF-IDF embeddings (10279D)
# Milestone checkpoint: stage_03_tfidf_text
# Option B strictness: TF-IDF is required, but we allow building it in-place when missing.
import sys
from pathlib import Path

STRICT_OPTION_B = True
USE_TFIDF_TEXT = True
feat_dir = WORK_ROOT / 'features'
text_vect = feat_dir / 'text_vectorizer.joblib'
train_text_npy = feat_dir / 'train_embeds_text.npy'
test_text_npy = feat_dir / 'test_embeds_text.npy'
FORCE_REBUILD = os.getenv('CAFA_FORCE_REBUILD', '0').strip().lower() not in {'0', '', 'false', 'no'}
generated = False
if USE_TFIDF_TEXT:
    have_all = text_vect.exists() and train_text_npy.exists() and test_text_npy.exists()
    if have_all and not FORCE_REBUILD:
        print('TF-IDF artefacts already present; skipping generation (set CAFA_FORCE_REBUILD=1 to refit).')
    else:
        text_path = WORK_ROOT / 'external' / 'entryid_text.tsv'
        if not text_path.exists():
            raise FileNotFoundError(f'Missing corpus file: {text_path}. Run the corpus milestone first.')
        sys.argv = [
            '02_generate_optional_embeddings.py',
            '--artefacts-dir', str(WORK_ROOT),
            '--mode', 'text',
            '--text-path', str(text_path),
            '--text-dim', '10279',
        ]
        _ = main()
        generated = True
if text_vect.exists() and train_text_npy.exists() and test_text_npy.exists():
    if generated:
        print('TF-IDF generated; attempting checkpoint push (may be skipped if unchanged).')
    else:
        print('TF-IDF present; attempting checkpoint push (may be skipped if unchanged).')
    STORE.push('stage_03_tfidf_text', [text_vect, train_text_npy, test_text_npy], note='tf-idf 10279D')

    # Diagnostics: TF-IDF sparsity + row norms (sampled)
    try:
        import numpy as np
        import matplotlib.pyplot as plt

        plt.rcParams.update({'font.size': 12})

        def _sample_rows(x: np.ndarray, n: int = 20000) -> np.ndarray:
            m = min(int(x.shape[0]), int(n))
            if m <= 0:
                return np.zeros((0, int(x.shape[1])), dtype=np.float32)
            idx = np.linspace(0, int(x.shape[0]) - 1, num=m, dtype=np.int64)
            return np.asarray(x[idx], dtype=np.float32)

        x_tr = np.load(train_text_npy, mmap_mode='r')
        x_te = np.load(test_text_npy, mmap_mode='r')
        tr = _sample_rows(x_tr)
        te = _sample_rows(x_te)

        tr_nnz = (tr > 0).sum(axis=1)
        te_nnz = (te > 0).sum(axis=1)
        plt.figure(figsize=(10, 4))
        plt.hist(tr_nnz, bins=60, alpha=0.6, label=f'train nnz/row (n={len(tr_nnz)})')
        plt.hist(te_nnz, bins=60, alpha=0.6, label=f'test nnz/row (n={len(te_nnz)})')
        plt.title('TF-IDF: non-zeros per row (sampled)')
        plt.xlabel('nnz per row')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()

        tr_norm = np.linalg.norm(tr, axis=1)
        te_norm = np.linalg.norm(te, axis=1)
        plt.figure(figsize=(10, 4))
        plt.hist(tr_norm, bins=60, alpha=0.6, label='train L2')
        plt.hist(te_norm, bins=60, alpha=0.6, label='test L2')
        plt.title('TF-IDF: L2 norm per row (sampled)')
        plt.xlabel('L2 norm')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()
    except Exception as e:
        print('TF-IDF diagnostics skipped:', repr(e))
elif STRICT_OPTION_B:
    missing = [p for p in [text_vect, train_text_npy, test_text_npy] if not p.exists()]
    raise FileNotFoundError('Option B requires TF-IDF artefacts, but these are missing:\n' + '\n'.join([f' - {m}' for m in missing]))

In [ ]:
# CELL 09 - Solution: 2.1 PHASE 1 (Step 3): EXTERNAL DATA & EVIDENCE CODES (ARTEFACT-FIRST)
# 2.1 PHASE 1 (Step 3): EXTERNAL DATA & EVIDENCE CODES (ARTEFACT-FIRST)
# ================================================================
# Option B strictness: external GOA artefacts are REQUIRED (no silent fallback).
from pathlib import Path
import os

STRICT_OPTION_B = True
# Option B: always enable external features
PROCESS_EXTERNAL = True
# Recommended (Kaggle): set CAFA_GOA_DATASET_DIR to your dataset folder,
# e.g. /kaggle/input/goa-filtered-all-tsv-gz
GOA_ENV_DIR = os.getenv('CAFA_GOA_DATASET_DIR', '').strip()
GOA_ARTEFACT_DIRS = [
    Path(GOA_ENV_DIR) if GOA_ENV_DIR else None,
    # Path('/kaggle/input/<dataset-folder>'),
]
if PROCESS_EXTERNAL:
    EXT_DIR = WORK_ROOT / 'external'
    EXT_DIR.mkdir(exist_ok=True)
    def _existing(dirs: list[Path]) -> list[Path]:
        return [Path(d) for d in dirs if d is not None and str(d).strip() and Path(d).exists()]
    def _discover(dirs: list[Path]) -> list[Path]:
        patterns = ('**/goa_filtered_*.tsv.gz', '**/goa_filtered_*.tsv')
        found: list[Path] = []
        for d in dirs:
            found += [
                d / 'goa_filtered_all.tsv.gz', d / 'goa_filtered_iea.tsv.gz',
                d / 'goa_filtered_all.tsv', d / 'goa_filtered_iea.tsv',
            ]
            for pat in patterns:
                found += sorted(Path(d).glob(pat))
        out: list[Path] = []
        seen = set()
        for p in found:
            p = Path(p)
            if p in seen:
                continue
            seen.add(p)
            if p.exists() and p.is_file():
                out.append(p)
        return out
    def _pick_best(paths: list[Path]) -> Path:
        def score(p: Path) -> tuple[int, int]:
            name = p.name.lower()
            ext_rank = 0 if name.endswith('.tsv.gz') else 1
            if 'goa_filtered_all' in name:
                return (0, ext_rank)
            if 'goa_filtered_iea' in name:
                return (1, ext_rank)
            if 'all' in name:
                return (2, ext_rank)
            if 'iea' in name:
                return (3, ext_rank)
            return (4, ext_rank)
        return sorted(paths, key=score)[0]
    roots = _existing(GOA_ARTEFACT_DIRS)
    if not roots and Path('/kaggle/input').exists():
        # fallback discovery, but still strict if nothing found
        roots = [Path('/kaggle/input')]
    elif not roots:
        roots = [EXT_DIR, Path('artefacts_local/artefacts/external')]
    goa_paths = _discover(roots)
    if not goa_paths:
        msg = (
            'Option B requires precomputed GOA artefacts, but none were found.\n'
            'Fix:\n'
            ' - Publish goa_filtered_all.tsv.gz (and/or goa_filtered_iea.tsv.gz) as a Kaggle Dataset\n'
            ' - Attach it to this notebook\n'
            ' - Set env var CAFA_GOA_DATASET_DIR=/kaggle/input/<dataset-folder> (recommended)\n'
            ' - Or place the file under artefacts/external locally'
        )
        if STRICT_OPTION_B:
            raise FileNotFoundError(msg)
        print(msg)
        PROCESS_EXTERNAL = False
    else:
        GOA_FEATURE_PATH = _pick_best(goa_paths)
        print('Using GOA artefact:', GOA_FEATURE_PATH)
        print('Format expected: EntryID<TAB>term<TAB>evidence (header included)')
else:
    raise RuntimeError('Option B requires PROCESS_EXTERNAL=True (unexpected).')

In [ ]:
# CELL 10 - Solution: 2.2 PHASE 1 (Step 4): HIERARCHY PROPAGATION FOR EXTERNAL GOA (NO-KAGGLE / IEA)
# Milestone checkpoint: stage_04_external_goa_priors
# Produces:
# - external/prop_train_no_kaggle.tsv.gz
# - external/prop_test_no_kaggle.tsv.gz
# Option B strictness: this step MUST be satisfied when PROCESS_EXTERNAL=True.
import gzip
import pandas as pd
from pathlib import Path

EXTERNAL_TOP_K = 1500
EXTERNAL_PRIOR_SCORE = 1.0  # binary prior (later we down-weight when injecting into models)

if PROCESS_EXTERNAL:
    if 'GOA_FEATURE_PATH' not in locals():
        raise RuntimeError('Option B requires GOA_FEATURE_PATH. Run the external GOA discovery cell first.')

    # Ensure we have GO parents (hierarchy)
    if 'go_parents' not in locals():
        def parse_obo(path: Path):
            parents = {}
            namespaces = {}
            cur_id, cur_ns = None, None
            with path.open('r', encoding='utf-8') as f:
                for line in f:
                    line = line.strip()
                    if line == '[Term]':
                        if cur_id and cur_ns:
                            namespaces[cur_id] = cur_ns
                        cur_id, cur_ns = None, None
                    elif line.startswith('id: GO:'):
                        cur_id = line.split('id: ', 1)[1]
                    elif line.startswith('namespace:'):
                        cur_ns = line.split('namespace: ', 1)[1]
                    elif line.startswith('is_a:') and cur_id:
                        parent = line.split('is_a: ', 1)[1].split(' ! ')[0]
                        parents.setdefault(cur_id, set()).add(parent)
                if cur_id and cur_ns:
                    namespaces[cur_id] = cur_ns
            return parents, namespaces

        go_parents, go_namespaces = parse_obo(PATH_GO_OBO)

    # Ensure train/test IDs are available
    train_seq_path = WORK_ROOT / 'parsed' / 'train_seq.feather'
    test_seq_path = WORK_ROOT / 'parsed' / 'test_seq.feather'
    if not train_seq_path.exists():
        raise FileNotFoundError('Missing train_seq.feather. Run Phase 1 Step 2 (FASTA parse) first.')
    if not test_seq_path.exists():
        raise FileNotFoundError('Missing test_seq.feather. Run Phase 1 Step 2 (FASTA parse) first.')

    # IMPORTANT: GOA artefacts usually use UniProt accessions, but FASTA IDs may be like sp|P12345|NAME.
    # Normalise for matching, but write the original IDs so downstream joins (reindex on train_ids/test_ids) still work.
    def _norm_entry_id(raw: str) -> str:
        s = str(raw).strip()
        parts = s.split('|')
        if len(parts) >= 2 and parts[0] in {'sp', 'tr'}:
            return parts[1]
        if len(parts) >= 3:
            return parts[1]
        return s

    train_ids_raw = pd.read_feather(train_seq_path)['id'].astype(str).tolist()
    test_ids_raw = pd.read_feather(test_seq_path)['id'].astype(str).tolist()
    train_id_map = {_norm_entry_id(i): i for i in train_ids_raw}
    test_id_map = {_norm_entry_id(i): i for i in test_ids_raw}
    train_ids = set(train_id_map.keys())
    test_ids = set(test_id_map.keys())

    # Get top-K train terms (defines the external feature space)
    train_terms_path = WORK_ROOT / 'parsed' / 'train_terms.parquet'
    if not train_terms_path.exists():
        raise FileNotFoundError('Missing train_terms.parquet. Run Phase 1 Step 2 (targets parse) first.')

    train_terms = pd.read_parquet(train_terms_path)
    top_terms = train_terms['term'].value_counts().head(EXTERNAL_TOP_K).index.tolist()
    top_terms_set = set(top_terms)
    print(f'External propagation restricted to top {len(top_terms)} train terms.')

    EXT_DIR = WORK_ROOT / 'external'
    EXT_DIR.mkdir(exist_ok=True)
    out_train = EXT_DIR / 'prop_train_no_kaggle.tsv.gz'
    out_test = EXT_DIR / 'prop_test_no_kaggle.tsv.gz'

    if out_train.exists() and out_test.exists():
        print('Propagated GOA priors already present; skipping propagation.')
    else:
        # Ancestor closure with memoisation
        _anc_cache: dict[str, set[str]] = {}

        def ancestors(term: str) -> set[str]:
            if term in _anc_cache:
                return _anc_cache[term]
            seen = {term}
            stack = [term]
            while stack:
                t = stack.pop()
                for p in go_parents.get(t, ()):
                    if p not in seen:
                        seen.add(p)
                        stack.append(p)
            _anc_cache[term] = seen
            return seen

        cols = ['EntryID', 'term', 'evidence']
        print('Streaming GOA artefact:', GOA_FEATURE_PATH)
        print('Writing:', out_train)
        print('Writing:', out_test)

        n_train = 0
        n_test = 0
        with gzip.open(out_train, 'wt', encoding='utf-8') as ftr, gzip.open(out_test, 'wt', encoding='utf-8') as fte:
            ftr.write('EntryID\tterm\tscore\n')
            fte.write('EntryID\tterm\tscore\n')

            for chunk in pd.read_csv(
                GOA_FEATURE_PATH,
                sep='\t',
                dtype=str,
                usecols=lambda c: c in cols,
                chunksize=500_000,
            ):
                missing_cols = [c for c in cols if c not in chunk.columns]
                if missing_cols:
                    raise ValueError(f'GOA artefact missing columns: {missing_cols}. Found: {list(chunk.columns)}')

                chunk = chunk[cols].dropna()
                chunk = chunk[chunk['evidence'] == 'IEA']
                if chunk.empty:
                    continue

                chunk = chunk.drop_duplicates(subset=['EntryID', 'term'])

                for entry_id_raw, term in zip(chunk['EntryID'].tolist(), chunk['term'].tolist()):
                    entry_norm = _norm_entry_id(entry_id_raw)

                    if entry_norm in train_ids:
                        target = ftr
                        out_entry = train_id_map[entry_norm]
                    elif entry_norm in test_ids:
                        target = fte
                        out_entry = test_id_map[entry_norm]
                    else:
                        continue

                    keep = ancestors(term) & top_terms_set
                    if not keep:
                        continue

                    for t in keep:
                        target.write(f'{out_entry}\t{t}\t{EXTERNAL_PRIOR_SCORE}\n')

                    if target is ftr:
                        n_train += len(keep)
                    else:
                        n_test += len(keep)

        print(f'Wrote propagated IEA edges: train={n_train:,} test={n_test:,}')
        print('Outputs are intentionally sparse priors (score=1.0) and will be down-weighted when injected.')

    if out_train.exists() and out_test.exists():
        STORE.push('stage_04_external_goa_priors', [out_train, out_test], note='propagated IEA priors')
else:
    raise RuntimeError('Option B requires PROCESS_EXTERNAL=True (unexpected).')


In [ ]:
# CELL 10b - Diagnostics: artefact manifest (existence + sizes)
%matplotlib inline
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams.update({'font.size': 12})

def _mb(p: Path) -> float:
    return p.stat().st_size / (1024**2)
WORK_ROOT = Path(WORK_ROOT)
# Minimal contract: Phase 1 outputs + Option B required artefacts + later model outputs
paths = {
    # Phase 1 parsed
    'parsed/train_seq.feather': WORK_ROOT / 'parsed' / 'train_seq.feather',
    'parsed/test_seq.feather': WORK_ROOT / 'parsed' / 'test_seq.feather',
    'parsed/train_terms.parquet': WORK_ROOT / 'parsed' / 'train_terms.parquet',
    'parsed/term_priors.parquet': WORK_ROOT / 'parsed' / 'term_priors.parquet',
    'parsed/train_taxa.feather': WORK_ROOT / 'parsed' / 'train_taxa.feather',
    'parsed/test_taxa.feather': WORK_ROOT / 'parsed' / 'test_taxa.feather',
    # Text pipeline
    'external/entryid_text.tsv': WORK_ROOT / 'external' / 'entryid_text.tsv',
    'features/text_vectorizer.joblib': WORK_ROOT / 'features' / 'text_vectorizer.joblib',
    'features/train_embeds_text.npy': WORK_ROOT / 'features' / 'train_embeds_text.npy',
    'features/test_embeds_text.npy': WORK_ROOT / 'features' / 'test_embeds_text.npy',
    # Sequence embeddings (core)
    'features/train_embeds_t5.npy': WORK_ROOT / 'features' / 'train_embeds_t5.npy',
    'features/test_embeds_t5.npy': WORK_ROOT / 'features' / 'test_embeds_t5.npy',
    'features/train_embeds_esm2.npy': WORK_ROOT / 'features' / 'train_embeds_esm2.npy',
    'features/test_embeds_esm2.npy': WORK_ROOT / 'features' / 'test_embeds_esm2.npy',
    'features/train_embeds_esm2_3b.npy': WORK_ROOT / 'features' / 'train_embeds_esm2_3b.npy',
    'features/test_embeds_esm2_3b.npy': WORK_ROOT / 'features' / 'test_embeds_esm2_3b.npy',
    'features/train_embeds_ankh.npy': WORK_ROOT / 'features' / 'train_embeds_ankh.npy',
    'features/test_embeds_ankh.npy': WORK_ROOT / 'features' / 'test_embeds_ankh.npy',
    # External priors
    'external/prop_train_no_kaggle.tsv.gz': WORK_ROOT / 'external' / 'prop_train_no_kaggle.tsv.gz',
    'external/prop_test_no_kaggle.tsv.gz': WORK_ROOT / 'external' / 'prop_test_no_kaggle.tsv.gz',
    # Downstream expectations
    'features/top_terms_1500.json': WORK_ROOT / 'features' / 'top_terms_1500.json',
    'features/oof_pred_logreg.npy': WORK_ROOT / 'features' / 'oof_pred_logreg.npy',
    'features/oof_pred_gbdt.npy': WORK_ROOT / 'features' / 'oof_pred_gbdt.npy',
    'features/oof_pred_dnn.npy': WORK_ROOT / 'features' / 'oof_pred_dnn.npy',
    'features/test_pred_logreg.npy': WORK_ROOT / 'features' / 'test_pred_logreg.npy',
    'features/test_pred_gbdt.npy': WORK_ROOT / 'features' / 'test_pred_gbdt.npy',
    'features/test_pred_dnn.npy': WORK_ROOT / 'features' / 'test_pred_dnn.npy',
    'features/test_pred_gcn.npy': WORK_ROOT / 'features' / 'test_pred_gcn.npy',
}
rows = []
for name, p in paths.items():
    rows.append({'artefact': name, 'exists': p.exists(), 'mb': _mb(p) if p.exists() else 0.0, 'path': str(p)})
df = pd.DataFrame(rows).sort_values(['exists', 'mb'], ascending=[True, False])
print('WORK_ROOT:', WORK_ROOT)
display(df)
# Visual: top 25 largest artefacts
df2 = df[df['exists']].sort_values('mb', ascending=False).head(25)
if len(df2) > 0:
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df2, y='artefact', x='mb')
    plt.title('Largest artefacts (MB)')
    plt.xlabel('MB')
    plt.ylabel('artefact')
    plt.tight_layout()
    plt.show()
# Option B strict check (what must exist before the stacker)
option_b_required = [
    'external/prop_train_no_kaggle.tsv.gz',
    'external/prop_test_no_kaggle.tsv.gz',
    'features/text_vectorizer.joblib',
    'features/train_embeds_text.npy',
    'features/test_embeds_text.npy',
    'features/train_embeds_t5.npy',
    'features/test_embeds_t5.npy',
    'features/train_embeds_esm2.npy',
    'features/test_embeds_esm2.npy',
    'features/train_embeds_esm2_3b.npy',
    'features/test_embeds_esm2_3b.npy',
    'features/train_embeds_ankh.npy',
    'features/test_embeds_ankh.npy',
    'parsed/train_taxa.feather',
    'parsed/test_taxa.feather',
    'features/top_terms_1500.json',
]
missing = [a for a in option_b_required if not paths[a].exists()]
if missing:
    print('\nOption B missing artefacts:')
    for m in missing:
        print(' -', m)
else:
    print('\nOption B artefacts OK: priors + embeddings + TF-IDF + taxonomy + top terms present.')

In [ ]:
# CELL 11 - Solution: 3a. PHASE 1: EMBEDDINGS GENERATION (T5 only)
# 3a. PHASE 1: EMBEDDINGS GENERATION (T5 only)
# ============================================
# HARDWARE: GPU recommended
# ============================================
# Split from ESM2 so you can run each independently on Kaggle.
import os

FORCE_REBUILD = os.getenv('CAFA_FORCE_REBUILD', '0').strip().lower() not in {'0', '', 'false', 'no'}
# IMPORTANT: pushing checkpoints republishes the whole Kaggle Dataset version (dir-mode zip),
# which can mean re-uploading multi-GB zips even when embeddings already exist.
PUSH_EXISTING_CHECKPOINTS = os.getenv('CAFA_CHECKPOINT_PUSH_EXISTING', '0').strip() == '1'
t5_train_path = WORK_ROOT / 'features' / 'train_embeds_t5.npy'
t5_test_path = WORK_ROOT / 'features' / 'test_embeds_t5.npy'

has_test = (WORK_ROOT / 'parsed' / 'test_seq.feather').exists()
train_ready = t5_train_path.exists()
test_ready = (not has_test) or t5_test_path.exists()

COMPUTE_T5 = True  # <--- enable/disable T5 run

train_needed = COMPUTE_T5 and (FORCE_REBUILD or (not train_ready))
test_needed = COMPUTE_T5 and has_test and (FORCE_REBUILD or (not t5_test_path.exists()))

# If artefacts already exist, skip compute unless forced.
# Checkpoint-push for existing artefacts is opt-in (CAFA_CHECKPOINT_PUSH_EXISTING=1) to avoid huge re-uploads.
if COMPUTE_T5 and (not train_needed) and (not test_needed):
    print(f'T5 embeddings already exist at {t5_train_path}; skipping compute (set CAFA_FORCE_REBUILD=1 to rebuild).')
    COMPUTE_T5 = False
    if not PUSH_EXISTING_CHECKPOINTS:
        print('Skipping checkpoint push for existing T5 artefacts (set CAFA_CHECKPOINT_PUSH_EXISTING=1 to enable).')
    if PUSH_EXISTING_CHECKPOINTS and 'STORE' in globals() and STORE is not None:
        # Granular checkpoints: allow resuming test without losing train.
        try:
            STORE.push('stage_05a_embeddings_t5_train', [t5_train_path], note='ProtT5 mean-pool embeddings (train only, existing)')
        except Exception as e:
            print(f'WARN: failed to push stage_05a_embeddings_t5_train: {e}')

        if t5_test_path.exists():
            try:
                STORE.push('stage_05b_embeddings_t5_test', [t5_test_path], note='ProtT5 mean-pool embeddings (test only, existing)')
            except Exception as e:
                print(f'WARN: failed to push stage_05b_embeddings_t5_test: {e}')

        # Backwards-compatible combined stage.
        paths = [t5_train_path]
        if t5_test_path.exists():
            paths.append(t5_test_path)
        try:
            STORE.push('stage_05_embeddings_t5', paths, note='ProtT5 mean-pool embeddings (existing)')
        except Exception as e:
            print(f'WARN: failed to push stage_05_embeddings_t5: {e}')

if COMPUTE_T5:
    import gc
    import numpy as np
    import pandas as pd
    import torch
    from transformers import T5Tokenizer, T5EncoderModel
    from tqdm.auto import tqdm
    import contextlib

    # Optimise CUDA memory allocation
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

    # Fix Protobuf 'GetPrototype' error
    os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    amp_ctx = torch.amp.autocast('cuda') if device.type == 'cuda' else contextlib.nullcontext()

    def get_t5_model():
        print("Loading T5 Model...")
        tokenizer = T5Tokenizer.from_pretrained(
            "Rostlab/prot_t5_xl_half_uniref50-enc", do_lower_case=False, legacy=True
        )
        model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc").to(device)
        model.eval()
        return tokenizer, model

    def generate_embeddings_t5(model, tokenizer, sequences, batch_size=4, max_len=1024):
        seq_lens = [len(s) for s in sequences]
        sort_idx = np.argsort(seq_lens)[::-1]
        sorted_seqs = [sequences[i] for i in sort_idx]
        embeddings_list = []
        for i in tqdm(range(0, len(sorted_seqs), batch_size), desc="Embedding T5 (Smart Batch)"):
            batch_seqs = sorted_seqs[i : i + batch_size]
            batch_seqs = [seq.replace('U', 'X').replace('Z', 'X').replace('O', 'X').replace('B', 'X') for seq in batch_seqs]
            batch_seqs = [" ".join(list(seq)) for seq in batch_seqs]
            ids = tokenizer.batch_encode_plus(
                batch_seqs,
                add_special_tokens=True,
                padding="longest",
                truncation=True,
                max_length=max_len,
                return_tensors="pt",
            ).to(device)
            with torch.no_grad():
                with amp_ctx:
                    embedding_repr = model(input_ids=ids['input_ids'], attention_mask=ids['attention_mask'])
            emb = embedding_repr.last_hidden_state.float().detach().cpu().numpy()
            mask = ids['attention_mask'].detach().cpu().numpy()
            for j in range(len(batch_seqs)):
                seq_len = int(mask[j].sum())
                valid_emb = emb[j, :seq_len]
                embeddings_list.append(valid_emb.mean(axis=0))
            del ids, embedding_repr, emb, mask
            if device.type == 'cuda':
                torch.cuda.empty_cache()
        sorted_embeddings = np.vstack(embeddings_list)
        original_order_embeddings = np.zeros_like(sorted_embeddings)
        original_order_embeddings[sort_idx] = sorted_embeddings
        return original_order_embeddings

    tokenizer, model = get_t5_model()

    if train_needed:
        print("Loading train sequences for T5 embedding...")
        train_df = pd.read_feather(WORK_ROOT / 'parsed' / 'train_seq.feather')

        print(f"Generating Train Embeddings T5 ({len(train_df)})...")
        train_emb = generate_embeddings_t5(model, tokenizer, train_df['sequence'].tolist())
        np.save(t5_train_path, train_emb)

        # Push immediately after train completes so a crash during test doesn't lose progress.
        if 'STORE' in globals() and STORE is not None:
            try:
                STORE.push('stage_05a_embeddings_t5_train', [t5_train_path], note='ProtT5 mean-pool embeddings (train only)')
            except Exception as e:
                print(f'WARN: failed to push stage_05a_embeddings_t5_train: {e}')

        del train_emb, train_df
        gc.collect()
    else:
        print(f"Train embeddings already exist at {t5_train_path}; skipping train compute.")

    if test_needed:
        print("Loading test sequences for T5 embedding...")
        test_df = pd.read_feather(WORK_ROOT / 'parsed' / 'test_seq.feather')

        print(f"Generating Test Embeddings T5 ({len(test_df)})...")
        test_emb = generate_embeddings_t5(model, tokenizer, test_df['sequence'].tolist())
        np.save(t5_test_path, test_emb)

        # Push immediately after test completes as well.
        if 'STORE' in globals() and STORE is not None:
            try:
                STORE.push('stage_05b_embeddings_t5_test', [t5_test_path], note='ProtT5 mean-pool embeddings (test only)')
            except Exception as e:
                print(f'WARN: failed to push stage_05b_embeddings_t5_test: {e}')

        del test_emb, test_df
        gc.collect()
    else:
        if has_test:
            print(f"Test embeddings already exist at {t5_test_path}; skipping test compute.")
        else:
            print("No test sequences found; skipping test embedding.")

    del model, tokenizer
    gc.collect()

    if device.type == 'cuda':
        torch.cuda.empty_cache()

    print("T5 embeddings ready.")

    # Backwards-compatible combined stage.
    if 'STORE' in globals() and STORE is not None:
        paths = [t5_train_path]
        if t5_test_path.exists():
            paths.append(t5_test_path)
        try:
            STORE.push('stage_05_embeddings_t5', paths, note='ProtT5 mean-pool embeddings')
        except Exception as e:
            print(f'WARN: failed to push stage_05_embeddings_t5: {e}')
else:
    print("Skipping T5 embedding generation (COMPUTE_T5=False).")

# Diagnostics: embedding norms (train vs test; sampled)
try:
    import numpy as np
    import matplotlib.pyplot as plt
    from pathlib import Path

    plt.rcParams.update({'font.size': 12})

    def _plot_embed_norms(name: str, train_path: Path, test_path: Path):
        if not Path(train_path).exists():
            print(f'[{name}] missing train embeddings: {train_path}')
            return
        x_tr = np.load(train_path, mmap_mode='r')
        x_te = np.load(test_path, mmap_mode='r') if Path(test_path).exists() else None

        def _sample_norms(x: np.ndarray, n: int = 20000) -> np.ndarray:
            m = min(int(x.shape[0]), int(n))
            if m <= 0:
                return np.zeros((0,), dtype=np.float32)
            idx = np.linspace(0, int(x.shape[0]) - 1, num=m, dtype=np.int64)
            return np.linalg.norm(np.asarray(x[idx], dtype=np.float32), axis=1).astype(np.float32)

        tr_norm = _sample_norms(x_tr)
        plt.figure(figsize=(10, 4))
        plt.hist(tr_norm, bins=60, alpha=0.6, label=f'train (n={len(tr_norm)})')
        if x_te is not None:
            te_norm = _sample_norms(x_te)
            plt.hist(te_norm, bins=60, alpha=0.6, label=f'test (n={len(te_norm)})')
        plt.title(f'{name}: L2 norm distribution (sampled)')
        plt.xlabel('L2 norm')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()

        def _stats(a: np.ndarray) -> dict:
            if a.size == 0:
                return {}
            return {
                'min': float(np.min(a)),
                'p50': float(np.median(a)),
                'p95': float(np.quantile(a, 0.95)),
                'max': float(np.max(a)),
            }
        print(f'[{name}] train norm stats:', _stats(tr_norm))
        if x_te is not None:
            print(f'[{name}] test norm stats:', _stats(te_norm))

    _plot_embed_norms('ProtT5', t5_train_path, t5_test_path)
except Exception as e:
    print('T5 embedding diagnostics skipped:', repr(e))

In [ ]:
# CELL 12 - Solution: 3b. PHASE 1: EMBEDDINGS GENERATION (ESM2 only)
# 3b. PHASE 1: EMBEDDINGS GENERATION (ESM2 only)
# ==============================================
# HARDWARE: GPU recommended
# ==============================================
# Split from T5 so you can run each independently on Kaggle.
import os

FORCE_REBUILD = os.getenv('CAFA_FORCE_REBUILD', '0').strip().lower() not in {'0', '', 'false', 'no'}
# IMPORTANT: pushing checkpoints republishes the whole Kaggle Dataset version (dir-mode zip),
# which can mean re-uploading multi-GB zips even when embeddings already exist.
PUSH_EXISTING_CHECKPOINTS = os.getenv('CAFA_CHECKPOINT_PUSH_EXISTING', '0').strip() == '1'

esm2_train_path = WORK_ROOT / 'features' / 'train_embeds_esm2.npy'
esm2_test_path = WORK_ROOT / 'features' / 'test_embeds_esm2.npy'
esm2_3b_train_path = WORK_ROOT / 'features' / 'train_embeds_esm2_3b.npy'
esm2_3b_test_path = WORK_ROOT / 'features' / 'test_embeds_esm2_3b.npy'

# Ankh model + output naming
# - Default model keeps backwards-compatible filenames so downstream cells still work.
# - Non-default model writes to a new filename (no overwriting of existing Ankh artefacts).
DEFAULT_ANKH_MODEL = 'ElnaggarLab/ankh-large'
ANKH_MODEL = os.getenv('CAFA_ANKH_MODEL', DEFAULT_ANKH_MODEL).strip()


def _safe_slug(s: str, max_len: int = 40) -> str:
    s = (s or '').strip().lower()
    out = []
    for ch in s:
        if ch.isalnum():
            out.append(ch)
        else:
            out.append('_')
    slug = ''.join(out).strip('_')
    while '__' in slug:
        slug = slug.replace('__', '_')
    return slug[:max_len] if slug else 'unk'


ankh_suffix = '' if ANKH_MODEL == DEFAULT_ANKH_MODEL else f'__{_safe_slug(ANKH_MODEL)}'
ankh_train_path = WORK_ROOT / 'features' / f'train_embeds_ankh{ankh_suffix}.npy'
ankh_test_path = WORK_ROOT / 'features' / f'test_embeds_ankh{ankh_suffix}.npy'

has_test = (WORK_ROOT / 'parsed' / 'test_seq.feather').exists()

# These are core modalities for this project. We still skip compute if artefacts exist (unless forced),
# but they are not treated as optional downstream.
COMPUTE_ESM2 = True  # ESM2-650M
COMPUTE_ESM2_3B = True  # ESM2-3B
COMPUTE_ANKH = True  # Ankh

# Granular "needed" flags so we can resume mid-cell without recomputing train.
esm2_train_needed = COMPUTE_ESM2 and (FORCE_REBUILD or (not esm2_train_path.exists()))
esm2_test_needed = COMPUTE_ESM2 and has_test and (FORCE_REBUILD or (not esm2_test_path.exists()))

esm2_3b_train_needed = COMPUTE_ESM2_3B and (FORCE_REBUILD or (not esm2_3b_train_path.exists()))
esm2_3b_test_needed = COMPUTE_ESM2_3B and has_test and (FORCE_REBUILD or (not esm2_3b_test_path.exists()))

ankh_train_needed = COMPUTE_ANKH and (FORCE_REBUILD or (not ankh_train_path.exists()))
ankh_test_needed = COMPUTE_ANKH and has_test and (FORCE_REBUILD or (not ankh_test_path.exists()))

# If artefacts already exist, skip compute unless forced.
# Checkpoint-push for existing artefacts is opt-in (CAFA_CHECKPOINT_PUSH_EXISTING=1) to avoid huge re-uploads.
if COMPUTE_ESM2 and (not esm2_train_needed) and (not esm2_test_needed):
    print(f'ESM2 embeddings already exist at {esm2_train_path}; skipping compute (set CAFA_FORCE_REBUILD=1 to rebuild).')
    COMPUTE_ESM2 = False
    if not PUSH_EXISTING_CHECKPOINTS:
        print('Skipping checkpoint push for existing ESM2 artefacts (set CAFA_CHECKPOINT_PUSH_EXISTING=1 to enable).')
    if PUSH_EXISTING_CHECKPOINTS and 'STORE' in globals() and STORE is not None:
        try:
            STORE.push('stage_06_embeddings_esm2_train', [esm2_train_path], note='ESM2-650M mean-pool embeddings (train only, existing)')
        except Exception as e:
            print(f'WARN: failed to push stage_06_embeddings_esm2_train: {e}')

        if esm2_test_path.exists():
            try:
                STORE.push('stage_06_embeddings_esm2_test', [esm2_test_path], note='ESM2-650M mean-pool embeddings (test only, existing)')
            except Exception as e:
                print(f'WARN: failed to push stage_06_embeddings_esm2_test: {e}')

        paths = [esm2_train_path]
        if esm2_test_path.exists():
            paths.append(esm2_test_path)
        try:
            STORE.push('stage_06_embeddings_esm2', paths, note='ESM2-650M mean-pool embeddings (existing)')
        except Exception as e:
            print(f'WARN: failed to push stage_06_embeddings_esm2: {e}')

if COMPUTE_ESM2_3B and (not esm2_3b_train_needed) and (not esm2_3b_test_needed):
    print(f'ESM2-3B embeddings already exist at {esm2_3b_train_path}; skipping compute (set CAFA_FORCE_REBUILD=1 to rebuild).')
    COMPUTE_ESM2_3B = False
    if not PUSH_EXISTING_CHECKPOINTS:
        print('Skipping checkpoint push for existing ESM2-3B artefacts (set CAFA_CHECKPOINT_PUSH_EXISTING=1 to enable).')
    if PUSH_EXISTING_CHECKPOINTS and 'STORE' in globals() and STORE is not None:
        try:
            STORE.push('stage_06b_embeddings_esm2_3b_train', [esm2_3b_train_path], note='ESM2-3B mean-pool embeddings (train only, existing)')
        except Exception as e:
            print(f'WARN: failed to push stage_06b_embeddings_esm2_3b_train: {e}')

        if esm2_3b_test_path.exists():
            try:
                STORE.push('stage_06b_embeddings_esm2_3b_test', [esm2_3b_test_path], note='ESM2-3B mean-pool embeddings (test only, existing)')
            except Exception as e:
                print(f'WARN: failed to push stage_06b_embeddings_esm2_3b_test: {e}')

        paths = [esm2_3b_train_path]
        if esm2_3b_test_path.exists():
            paths.append(esm2_3b_test_path)
        try:
            STORE.push('stage_06b_embeddings_esm2_3b', paths, note='ESM2-3B mean-pool embeddings (existing)')
        except Exception as e:
            print(f'WARN: failed to push stage_06b_embeddings_esm2_3b: {e}')

if COMPUTE_ANKH and (not ankh_train_needed) and (not ankh_test_needed):
    print(f'Ankh embeddings already exist for model {ANKH_MODEL} at {ankh_train_path}; skipping compute (set CAFA_FORCE_REBUILD=1 to rebuild).')
    COMPUTE_ANKH = False
    if not PUSH_EXISTING_CHECKPOINTS:
        print('Skipping checkpoint push for existing Ankh artefacts (set CAFA_CHECKPOINT_PUSH_EXISTING=1 to enable).')
    if PUSH_EXISTING_CHECKPOINTS and 'STORE' in globals() and STORE is not None:
        stage_tag = 'ankh' if ANKH_MODEL == DEFAULT_ANKH_MODEL else f"ankh_{_safe_slug(ANKH_MODEL)}"
        try:
            STORE.push(f'stage_06c_embeddings_{stage_tag}_train', [ankh_train_path], note=f'Ankh mean-pool embeddings ({ANKH_MODEL}) (train only, existing)')
        except Exception as e:
            print(f'WARN: failed to push stage_06c embeddings train: {e}')

        if ankh_test_path.exists():
            try:
                STORE.push(f'stage_06c_embeddings_{stage_tag}_test', [ankh_test_path], note=f'Ankh mean-pool embeddings ({ANKH_MODEL}) (test only, existing)')
            except Exception as e:
                print(f'WARN: failed to push stage_06c embeddings test: {e}')

        # Combined stage includes all Ankh embedding variants present so newly-created files can't be missed.
        feat_dir = (WORK_ROOT / 'features')
        paths = []
        paths += sorted(feat_dir.glob('train_embeds_ankh*.npy'))
        paths += sorted(feat_dir.glob('test_embeds_ankh*.npy'))
        if ankh_train_path not in paths:
            paths.append(ankh_train_path)
        if ankh_test_path.exists() and ankh_test_path not in paths:
            paths.append(ankh_test_path)
        try:
            STORE.push(f'stage_06c_embeddings_{stage_tag}', paths, note=f'Ankh mean-pool embeddings ({ANKH_MODEL}) (existing)')
        except Exception as e:
            print(f'WARN: failed to push stage_06c embeddings combined: {e}')

if COMPUTE_ESM2 or COMPUTE_ESM2_3B or COMPUTE_ANKH:
    import gc
    import numpy as np
    import pandas as pd
    import torch
    from transformers import EsmTokenizer, EsmModel, AutoTokenizer, AutoModel
    from tqdm.auto import tqdm
    import contextlib

    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    amp_ctx = torch.amp.autocast('cuda') if device.type == 'cuda' else contextlib.nullcontext()

    def _mean_pool(last_hidden_state: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        mask = attention_mask.unsqueeze(-1).to(last_hidden_state.dtype)
        x = last_hidden_state * mask
        denom = mask.sum(dim=1).clamp(min=1.0)
        return x.sum(dim=1) / denom

    def get_esm2_model(model_name: str):
        print(f"Loading ESM2 model: {model_name}")
        tokenizer = EsmTokenizer.from_pretrained(model_name)
        model = EsmModel.from_pretrained(model_name).to(device)
        model.eval()
        return tokenizer, model

    def generate_embeddings_esm2(model, tokenizer, sequences, batch_size=16, max_len=1024):
        seq_lens = [len(s) for s in sequences]
        sort_idx = np.argsort(seq_lens)[::-1]
        sorted_seqs = [sequences[i] for i in sort_idx]
        embeddings_list = []
        for i in tqdm(range(0, len(sorted_seqs), batch_size), desc="Embedding ESM2 (Smart Batch)"):
            batch_seqs = sorted_seqs[i : i + batch_size]
            ids = tokenizer.batch_encode_plus(
                batch_seqs,
                add_special_tokens=True,
                padding="longest",
                truncation=True,
                max_length=max_len,
                return_tensors="pt",
            ).to(device)
            with torch.no_grad():
                with amp_ctx:
                    output = model(input_ids=ids['input_ids'], attention_mask=ids['attention_mask'])
            pooled = _mean_pool(output.last_hidden_state.float(), ids['attention_mask']).detach().cpu().numpy().astype(np.float32)
            embeddings_list.append(pooled)
            del ids, output, pooled
            if device.type == 'cuda':
                torch.cuda.empty_cache()
        sorted_embeddings = np.vstack(embeddings_list)
        original_order_embeddings = np.zeros_like(sorted_embeddings)
        original_order_embeddings[sort_idx] = sorted_embeddings
        return original_order_embeddings

    def generate_embeddings_ankh(model_name: str, sequences, batch_size=2, max_len=1024, trust_remote_code=True):
        print(f"Loading Ankh model: {model_name} (trust_remote_code={trust_remote_code})")
        tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=trust_remote_code)
        model = AutoModel.from_pretrained(model_name, trust_remote_code=trust_remote_code).to(device)
        model.eval()
        # Some Ankh checkpoints load as encoder-decoder (T5-like). For embeddings we only need the encoder.
        is_enc_dec = bool(getattr(getattr(model, 'config', None), 'is_encoder_decoder', False))
        if is_enc_dec and not hasattr(model, 'encoder'):
            raise RuntimeError(f'Ankh model appears encoder-decoder but has no .encoder: {type(model)}')
        encoder = model.encoder if is_enc_dec else model
        seq_lens = [len(s) for s in sequences]
        sort_idx = np.argsort(seq_lens)[::-1]
        sorted_seqs = [sequences[i] for i in sort_idx]
        outs = []
        amp_enabled = device.type == 'cuda'
        for i in tqdm(range(0, len(sorted_seqs), batch_size), desc="Embedding Ankh (Smart Batch)"):
            batch = sorted_seqs[i : i + batch_size]
            batch = [s.replace('U','X').replace('Z','X').replace('O','X').replace('B','X') for s in batch]
            batch = [' '.join(list(s)) for s in batch]
            ids = tok(batch, add_special_tokens=True, padding=True, truncation=True, max_length=max_len, return_tensors='pt')
            ids = {k: v.to(device) for k, v in ids.items()}
            if ids.get('attention_mask') is None:
                raise RuntimeError('Tokenizer did not return attention_mask for Ankh')
            if (ids['attention_mask'].sum(dim=1) == 0).any():
                raise RuntimeError(f'Ankh tokenisation produced an all-zero attention_mask (batch_start={i}).')

            def _forward(use_amp: bool):
                ctx = (torch.amp.autocast('cuda') if (use_amp and device.type == 'cuda') else torch.autocast('cpu', enabled=False))
                with torch.no_grad():
                    with ctx:
                        return encoder(input_ids=ids['input_ids'], attention_mask=ids.get('attention_mask'))

            out = _forward(amp_enabled)
            last = getattr(out, 'last_hidden_state', None)
            if last is None and isinstance(out, (tuple, list)) and len(out) > 0:
                last = out[0]
            if last is None:
                raise RuntimeError(f'Model {model_name} did not return last_hidden_state')
            pooled_t = _mean_pool(last.float(), ids['attention_mask'])
            if not bool(torch.isfinite(pooled_t).all().item()):
                if amp_enabled:
                    print(f'WARNING: Non-finite Ankh embeddings under AMP at batch_start={i}; retrying without AMP.')
                    amp_enabled = False
                    out = _forward(False)
                    last = getattr(out, 'last_hidden_state', None)
                    if last is None and isinstance(out, (tuple, list)) and len(out) > 0:
                        last = out[0]
                    if last is None:
                        raise RuntimeError(f'Model {model_name} did not return last_hidden_state')
                    pooled_t = _mean_pool(last.float(), ids['attention_mask'])
                if not bool(torch.isfinite(pooled_t).all().item()):
                    raise RuntimeError(f'Ankh produced non-finite embeddings even without AMP (batch_start={i}).')

            pooled = pooled_t.detach().cpu().numpy().astype(np.float32)
            outs.append(pooled)
            del ids, out, last, pooled_t, pooled
            if device.type == 'cuda':
                torch.cuda.empty_cache()
        sorted_embeddings = np.vstack(outs)
        original_order_embeddings = np.zeros_like(sorted_embeddings)
        original_order_embeddings[sort_idx] = sorted_embeddings
        if not np.isfinite(original_order_embeddings).all():
            nan = int(np.isnan(original_order_embeddings).sum())
            inf = int(np.isinf(original_order_embeddings).sum())
            raise ValueError(f'Ankh embeddings are non-finite (nan={nan}, inf={inf}). Refusing to save.')
        del model, tok
        gc.collect()
        if device.type == 'cuda':
            torch.cuda.empty_cache()
        return original_order_embeddings

    # Only load sequences if we still need to compute something.
    if esm2_train_needed or esm2_test_needed or esm2_3b_train_needed or esm2_3b_test_needed or ankh_train_needed or ankh_test_needed:
        print("Loading sequences...")
        train_df = pd.read_feather(WORK_ROOT / 'parsed' / 'train_seq.feather')
        test_df = None
        if (WORK_ROOT / 'parsed' / 'test_seq.feather').exists():
            test_df = pd.read_feather(WORK_ROOT / 'parsed' / 'test_seq.feather')
        train_seqs = train_df['sequence'].tolist()
        test_seqs = test_df['sequence'].tolist() if test_df is not None else None
    else:
        train_df = None
        test_df = None
        train_seqs = None
        test_seqs = None

    if COMPUTE_ESM2:
        tokenizer, model = get_esm2_model('facebook/esm2_t33_650M_UR50D')

        if esm2_train_needed:
            print(f"Generating Train Embeddings ESM2-650M ({len(train_seqs)})...")
            train_emb = generate_embeddings_esm2(model, tokenizer, train_seqs, batch_size=16)
            np.save(esm2_train_path, train_emb)

            if 'STORE' in globals() and STORE is not None:
                try:
                    STORE.push('stage_06_embeddings_esm2_train', [esm2_train_path], note='ESM2-650M mean-pool embeddings (train only)')
                except Exception as e:
                    print(f'WARN: failed to push stage_06_embeddings_esm2_train: {e}')

            del train_emb
            gc.collect()
        else:
            print(f"Train embeddings already exist at {esm2_train_path}; skipping train compute.")

        if esm2_test_needed and test_seqs is not None:
            print(f"Generating Test Embeddings ESM2-650M ({len(test_seqs)})...")
            test_emb = generate_embeddings_esm2(model, tokenizer, test_seqs, batch_size=16)
            np.save(esm2_test_path, test_emb)

            if 'STORE' in globals() and STORE is not None:
                try:
                    STORE.push('stage_06_embeddings_esm2_test', [esm2_test_path], note='ESM2-650M mean-pool embeddings (test only)')
                except Exception as e:
                    print(f'WARN: failed to push stage_06_embeddings_esm2_test: {e}')

            del test_emb
        else:
            if has_test:
                print(f"Test embeddings already exist at {esm2_test_path}; skipping test compute.")
            else:
                print("No test sequences found; skipping test embedding.")

        del model, tokenizer
        gc.collect()
        if device.type == 'cuda':
            torch.cuda.empty_cache()

        print("ESM2-650M embeddings ready.")

        # Backwards-compatible combined stage.
        if 'STORE' in globals() and STORE is not None:
            paths = [esm2_train_path]
            if esm2_test_path.exists():
                paths.append(esm2_test_path)
            try:
                STORE.push('stage_06_embeddings_esm2', paths, note='ESM2-650M mean-pool embeddings')
            except Exception as e:
                print(f'WARN: failed to push stage_06_embeddings_esm2: {e}')
    else:
        print("Skipping ESM2-650M embedding generation (COMPUTE_ESM2=False).")

    if COMPUTE_ESM2_3B:
        tokenizer, model = get_esm2_model('facebook/esm2_t36_3B_UR50D')

        if esm2_3b_train_needed:
            print(f"Generating Train Embeddings ESM2-3B ({len(train_seqs)})...")
            train_emb = generate_embeddings_esm2(model, tokenizer, train_seqs, batch_size=2)
            np.save(esm2_3b_train_path, train_emb)

            if 'STORE' in globals() and STORE is not None:
                try:
                    STORE.push('stage_06b_embeddings_esm2_3b_train', [esm2_3b_train_path], note='ESM2-3B mean-pool embeddings (train only)')
                except Exception as e:
                    print(f'WARN: failed to push stage_06b_embeddings_esm2_3b_train: {e}')

            del train_emb
            gc.collect()
        else:
            print(f"Train embeddings already exist at {esm2_3b_train_path}; skipping train compute.")

        if esm2_3b_test_needed and test_seqs is not None:
            print(f"Generating Test Embeddings ESM2-3B ({len(test_seqs)})...")
            test_emb = generate_embeddings_esm2(model, tokenizer, test_seqs, batch_size=2)
            np.save(esm2_3b_test_path, test_emb)

            if 'STORE' in globals() and STORE is not None:
                try:
                    STORE.push('stage_06b_embeddings_esm2_3b_test', [esm2_3b_test_path], note='ESM2-3B mean-pool embeddings (test only)')
                except Exception as e:
                    print(f'WARN: failed to push stage_06b_embeddings_esm2_3b_test: {e}')

            del test_emb
        else:
            if has_test:
                print(f"Test embeddings already exist at {esm2_3b_test_path}; skipping test compute.")
            else:
                print("No test sequences found; skipping test embedding.")

        del model, tokenizer
        gc.collect()
        if device.type == 'cuda':
            torch.cuda.empty_cache()

        print("ESM2-3B embeddings ready.")

        # Backwards-compatible combined stage.
        if 'STORE' in globals() and STORE is not None:
            paths = [esm2_3b_train_path]
            if esm2_3b_test_path.exists():
                paths.append(esm2_3b_test_path)
            try:
                STORE.push('stage_06b_embeddings_esm2_3b', paths, note='ESM2-3B mean-pool embeddings')
            except Exception as e:
                print(f'WARN: failed to push stage_06b_embeddings_esm2_3b: {e}')
    else:
        print("Skipping ESM2-3B embedding generation (COMPUTE_ESM2_3B=False).")

    if COMPUTE_ANKH:
        stage_tag = 'ankh' if ANKH_MODEL == DEFAULT_ANKH_MODEL else f"ankh_{_safe_slug(ANKH_MODEL)}"

        if ankh_train_needed:
            print(f"Generating Train Embeddings Ankh ({ANKH_MODEL}) ({len(train_seqs)})...")
            train_emb = generate_embeddings_ankh(ANKH_MODEL, train_seqs, batch_size=2, trust_remote_code=True)
            np.save(ankh_train_path, train_emb)

            if 'STORE' in globals() and STORE is not None:
                try:
                    STORE.push(f'stage_06c_embeddings_{stage_tag}_train', [ankh_train_path], note=f'Ankh mean-pool embeddings ({ANKH_MODEL}) (train only)')
                except Exception as e:
                    print(f'WARN: failed to push stage_06c embeddings train: {e}')

            del train_emb
            gc.collect()
        else:
            print(f"Train embeddings already exist at {ankh_train_path}; skipping train compute.")

        if ankh_test_needed and test_seqs is not None:
            print(f"Generating Test Embeddings Ankh ({ANKH_MODEL}) ({len(test_seqs)})...")
            test_emb = generate_embeddings_ankh(ANKH_MODEL, test_seqs, batch_size=2, trust_remote_code=True)
            np.save(ankh_test_path, test_emb)

            if 'STORE' in globals() and STORE is not None:
                try:
                    STORE.push(f'stage_06c_embeddings_{stage_tag}_test', [ankh_test_path], note=f'Ankh mean-pool embeddings ({ANKH_MODEL}) (test only)')
                except Exception as e:
                    print(f'WARN: failed to push stage_06c embeddings test: {e}')

            del test_emb
            gc.collect()
        else:
            if has_test:
                print(f"Test embeddings already exist at {ankh_test_path}; skipping test compute.")
            else:
                print("No test sequences found; skipping test embedding.")

        if device.type == 'cuda':
            torch.cuda.empty_cache()

        print("Ankh embeddings ready.")

        # Combined stage includes all Ankh embedding variants present so newly-created files can't be missed.
        if 'STORE' in globals() and STORE is not None:
            feat_dir = (WORK_ROOT / 'features')
            paths = []
            paths += sorted(feat_dir.glob('train_embeds_ankh*.npy'))
            paths += sorted(feat_dir.glob('test_embeds_ankh*.npy'))
            if ankh_train_path not in paths:
                paths.append(ankh_train_path)
            if ankh_test_path.exists() and ankh_test_path not in paths:
                paths.append(ankh_test_path)
            try:
                STORE.push(f'stage_06c_embeddings_{stage_tag}', paths, note=f'Ankh mean-pool embeddings ({ANKH_MODEL})')
            except Exception as e:
                print(f'WARN: failed to push stage_06c embeddings combined: {e}')
    else:
        print("Skipping Ankh embedding generation (COMPUTE_ANKH=False).")

    del train_df, test_df, train_seqs, test_seqs
    gc.collect()
else:
    print("Skipping embeddings generation (all COMPUTE_* flags False).")

# Diagnostics: embedding norms (train vs test; sampled)
try:
    import numpy as np
    import matplotlib.pyplot as plt
    from pathlib import Path

    plt.rcParams.update({'font.size': 12})

    def _plot_embed_norms(name: str, train_path: Path, test_path: Path):
        if not Path(train_path).exists():
            print(f'[{name}] missing train embeddings: {train_path}')
            return
        x_tr = np.load(train_path, mmap_mode='r')
        x_te = np.load(test_path, mmap_mode='r') if Path(test_path).exists() else None

        def _sample_norms(x: np.ndarray, n: int = 20000) -> np.ndarray:
            m = min(int(x.shape[0]), int(n))
            if m <= 0:
                return np.zeros((0,), dtype=np.float32)
            idx = np.linspace(0, int(x.shape[0]) - 1, num=m, dtype=np.int64)
            return np.linalg.norm(np.asarray(x[idx], dtype=np.float32), axis=1).astype(np.float32)

        tr_norm = _sample_norms(x_tr)
        plt.figure(figsize=(10, 4))
        plt.hist(tr_norm, bins=60, alpha=0.6, label=f'train (n={len(tr_norm)})')
        if x_te is not None:
            te_norm = _sample_norms(x_te)
            plt.hist(te_norm, bins=60, alpha=0.6, label=f'test (n={len(te_norm)})')
        plt.title(f'{name}: L2 norm distribution (sampled)')
        plt.xlabel('L2 norm')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()

        def _stats(a: np.ndarray) -> dict:
            if a.size == 0:
                return {}
            return {
                'min': float(np.min(a)),
                'p50': float(np.median(a)),
                'p95': float(np.quantile(a, 0.95)),
                'max': float(np.max(a)),
            }
        print(f'[{name}] train norm stats:', _stats(tr_norm))
        if x_te is not None:
            print(f'[{name}] test norm stats:', _stats(te_norm))

    _plot_embed_norms('ESM2-650M', esm2_train_path, esm2_test_path)
    _plot_embed_norms('ESM2-3B', esm2_3b_train_path, esm2_3b_test_path)
    _plot_embed_norms('Ankh', ankh_train_path, ankh_test_path)
except Exception as e:
    print('Embedding diagnostics skipped:', repr(e))


In [ ]:
# CELL 13 - Solution: 4. PHASE 2: LEVEL-1 MODELS (DIVERSE ENSEMBLE)
# 4. PHASE 2: LEVEL-1 MODELS (DIVERSE ENSEMBLE)
# =============================================
# HARDWARE: GPU (32GB+ recommended for full run)
# =============================================
# We train a diverse set of models:
# 1. Logistic Regression (Baseline)
# 2. Py-Boost (GBDT) - Requires 'py-boost' package
# 3. DNN Ensemble (Deep Learning)
TRAIN_LEVEL1 = True
if TRAIN_LEVEL1:
    import joblib
    import json
    from sklearn.model_selection import KFold
    from sklearn.metrics import f1_score
    # -----------------------------
    # Load targets + ids
    # -----------------------------
    print("Loading targets...")
    train_terms = pd.read_parquet(WORK_ROOT / 'parsed' / 'train_terms.parquet')
    train_ids = pd.read_feather(WORK_ROOT / 'parsed' / 'train_seq.feather')['id'].astype(str)
    test_ids = pd.read_feather(WORK_ROOT / 'parsed' / 'test_seq.feather')['id'].astype(str)
    # Target Matrix Construction (Top K Terms)
    TOP_K = 1500
    top_terms = train_terms['term'].value_counts().head(TOP_K).index.tolist()
    train_terms_top = train_terms[train_terms['term'].isin(top_terms)]
    Y_df = train_terms_top.pivot_table(index='EntryID', columns='term', aggfunc='size', fill_value=0)
    Y_df = Y_df.reindex(train_ids, fill_value=0)
    Y = Y_df.values.astype(np.float32)
    print(f"Targets: Y={Y.shape}")
    # -----------------------------
    # -----------------------------
    # Feature loading (multimodal)
    # -----------------------------
    print("Loading multimodal features...")
    FEAT_DIR = WORK_ROOT / 'features'
    def _load_pair(stem):
        tr = FEAT_DIR / f'train_embeds_{stem}.npy'
        te = FEAT_DIR / f'test_embeds_{stem}.npy'
        if tr.exists() and te.exists():
            return np.load(tr).astype(np.float32), np.load(te).astype(np.float32)
        return None, None
    features_train = {}
    features_test = {}
    # Core modalities (NOT OPTIONAL): T5, ESM2-650M, ESM2-3B, Ankh, TF-IDF text, taxonomy
    for stem, key in [
        ('t5', 't5'),
        ('esm2', 'esm2_650m'),
        ('esm2_3b', 'esm2_3b'),
        ('ankh', 'ankh'),
        ('text', 'text'),
    ]:
        a_tr, a_te = _load_pair(stem)
        if a_tr is not None:
            features_train[key] = a_tr
            features_test[key] = a_te
    # Taxonomy (encode as one-hot / bag-of-taxa)
    taxa_train_path = WORK_ROOT / 'parsed' / 'train_taxa.feather'
    taxa_test_path = WORK_ROOT / 'parsed' / 'test_taxa.feather'
    if taxa_train_path.exists() and taxa_test_path.exists():
        from sklearn.preprocessing import OneHotEncoder
        tax_tr = pd.read_feather(taxa_train_path).astype({'id': str})
        tax_te = pd.read_feather(taxa_test_path).astype({'id': str})
        tax_tr = tax_tr.set_index('id').reindex(train_ids, fill_value=0).reset_index()
        tax_te = tax_te.set_index('id').reindex(test_ids, fill_value=0).reset_index()
        enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=np.float32)
        enc.fit(pd.concat([tax_tr[['taxon_id']], tax_te[['taxon_id']]], axis=0))
        X_tax_tr = enc.transform(tax_tr[['taxon_id']]).astype(np.float32)
        X_tax_te = enc.transform(tax_te[['taxon_id']]).astype(np.float32)
        features_train['taxa'] = X_tax_tr
        features_test['taxa'] = X_tax_te
        print(f"Taxa features: train={X_tax_tr.shape} test={X_tax_te.shape}")
    else:
        raise FileNotFoundError("Missing required taxonomy artefacts: parsed/train_taxa.feather and parsed/test_taxa.feather")
    # Sanity checks
    n_train = len(train_ids)
    n_test = len(test_ids)
    for k, v in features_train.items():
        if v.shape[0] != n_train:
            raise ValueError(f"Feature {k} train rows mismatch: {v.shape[0]} vs {n_train}")
    for k, v in features_test.items():
        if v.shape[0] != n_test:
            raise ValueError(f"Feature {k} test rows mismatch: {v.shape[0]} vs {n_test}")
    required_keys = ['t5', 'esm2_650m', 'esm2_3b', 'ankh', 'text', 'taxa']
    missing_keys = [k for k in required_keys if k not in features_train]
    if missing_keys:
        exp = {
            't5': ('features/train_embeds_t5.npy', 'features/test_embeds_t5.npy'),
            'esm2_650m': ('features/train_embeds_esm2.npy', 'features/test_embeds_esm2.npy'),
            'esm2_3b': ('features/train_embeds_esm2_3b.npy', 'features/test_embeds_esm2_3b.npy'),
            'ankh': ('features/train_embeds_ankh.npy', 'features/test_embeds_ankh.npy'),
            'text': ('features/train_embeds_text.npy', 'features/test_embeds_text.npy'),
            'taxa': ('parsed/train_taxa.feather', 'parsed/test_taxa.feather'),
        }
        msg = ['Missing required modalities (not optional):']
        for k in missing_keys:
            a, b = exp.get(k, ('?', '?'))
            msg.append(f' - {k}: expected {a} and {b}')
        raise FileNotFoundError('\n'.join(msg))
    # Flat concatenation for classical models (LR/GBDT)
    FLAT_KEYS = [k for k in ['t5', 'esm2_650m', 'esm2_3b', 'ankh', 'taxa', 'text'] if k in features_train]
    X = np.hstack([features_train[k] for k in FLAT_KEYS]).astype(np.float32)
    X_test = np.hstack([features_test[k] for k in FLAT_KEYS]).astype(np.float32)
    print(f"Flat X keys={FLAT_KEYS}")
    print(f"Flat shapes: X={X.shape}, X_test={X_test.shape}")
    # -----------------------------
    # CAFA-like IA-weighted diagnostic F1 (vectorised)
    # -----------------------------
    if 'ia' in locals():
        ia_df = ia[['term', 'ia']].copy()
    elif PATH_IA.exists():
        ia_df = pd.read_csv(PATH_IA, sep='\t', names=['term', 'ia'])
    else:
        ia_df = pd.DataFrame({'term': [], 'ia': []})
    ia_map = dict(zip(ia_df['term'], ia_df['ia']))
    def _ia_weight(term):
        v = ia_map.get(term, 0.0)
        if pd.isna(v):
            return 0.0
        return float(v)
    weights = np.array([_ia_weight(t) for t in top_terms], dtype=np.float32)
    ns_to_aspect = {
        'molecular_function': 'MF',
        'biological_process': 'BP',
        'cellular_component': 'CC',
    }
    if 'go_namespaces' in locals():
        term_aspects = np.array([ns_to_aspect.get(go_namespaces.get(t, ''), 'UNK') for t in top_terms])
    else:
        term_aspects = np.array(['UNK'] * len(top_terms))
    def ia_weighted_f1(y_true, y_score, thr=0.3):
        y_true = (y_true > 0).astype(np.int8)
        y_pred = (y_score >= thr).astype(np.int8)
        tp = (y_pred & y_true).sum(axis=0).astype(np.float64)
        pred = y_pred.sum(axis=0).astype(np.float64)
        true = y_true.sum(axis=0).astype(np.float64)
        def _score(mask=None):
            w = weights if mask is None else (weights * mask)
            w_tp = float((w * tp).sum())
            w_pred = float((w * pred).sum())
            w_true = float((w * true).sum())
            p = (w_tp / w_pred) if w_pred > 0 else 0.0
            r = (w_tp / w_true) if w_true > 0 else 0.0
            return (2 * p * r / (p + r)) if (p + r) > 0 else 0.0
        out = {'ALL': _score(None)}
        for asp in ['MF', 'BP', 'CC']:
            mask = (term_aspects == asp).astype(np.float32)
            out[asp] = _score(mask)
        return out
    # ------------------------------------------
    # A. Logistic Regression (Baseline)
    # ------------------------------------------
    print("\n--- Training Logistic Regression ---")
    from sklearn.linear_model import LogisticRegression
    from sklearn.multiclass import OneVsRestClassifier
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    oof_preds_logreg = np.zeros(Y.shape, dtype=np.float32)
    test_preds_logreg = np.zeros((len(test_ids), Y.shape[1]), dtype=np.float32)
    for fold, (idx_tr, idx_val) in enumerate(kf.split(X)):
        print(f"LogReg Fold {fold+1}/5")
        X_tr, X_val = X[idx_tr], X[idx_val]
        Y_tr, Y_val = Y[idx_tr], Y[idx_val]
        clf_logreg = OneVsRestClassifier(
            LogisticRegression(max_iter=500, solver='sag', n_jobs=1, C=1.0)
        )
        clf_logreg.n_jobs = -1
        clf_logreg.fit(X_tr, Y_tr)
        val_probs = clf_logreg.predict_proba(X_val)
        oof_preds_logreg[idx_val] = val_probs
        test_preds_logreg += clf_logreg.predict_proba(X_test) / kf.get_n_splits()
        val_preds = (val_probs > 0.3).astype(int)
        f1 = f1_score(Y_val, val_preds, average='micro')
        ia_f1 = ia_weighted_f1(Y_val, val_probs, thr=0.3)
        print(f"  >> Fold {fold+1} micro-F1@0.30: {f1:.4f}")
        print(f"  >> Fold {fold+1} IA-F1@0.30: ALL={ia_f1['ALL']:.4f} MF={ia_f1['MF']:.4f} BP={ia_f1['BP']:.4f} CC={ia_f1['CC']:.4f}")
        joblib.dump(clf_logreg, WORK_ROOT / 'features' / f'level1_logreg_fold{fold}.pkl')
    np.save(WORK_ROOT / 'features' / 'oof_pred_logreg.npy', oof_preds_logreg)
    np.save(WORK_ROOT / 'features' / 'test_pred_logreg.npy', test_preds_logreg)
    print("LogReg OOF + test preds saved.")
    # ------------------------------------------
    # B. Py-Boost (GBDT)
    # ------------------------------------------
    try:
        from py_boost import GradientBoosting
        HAS_PYBOOST = True
    except ImportError as e:
        raise RuntimeError('py_boost is REQUIRED for this pipeline (GBDT is mandatory). Install it with: pip install py-boost') from e
    if HAS_PYBOOST:
        print("\n--- Training Py-Boost GBDT ---")
        oof_preds_gbdt = np.zeros(Y.shape, dtype=np.float32)
        test_preds_gbdt = np.zeros((len(test_ids), Y.shape[1]), dtype=np.float32)
        for fold, (idx_tr, idx_val) in enumerate(kf.split(X)):
            print(f"GBDT Fold {fold+1}/5")
            X_tr, X_val = X[idx_tr], X[idx_val]
            Y_tr, Y_val = Y[idx_tr], Y[idx_val]
            model = GradientBoosting(
                loss='bce',
                ntrees=1000,
                lr=0.05,
                max_depth=6,
                verbose=100,
                es=50,
            )
            model.fit(X_tr, Y_tr, eval_sets=[{'X': X_val, 'y': Y_val}])
            val_probs = model.predict(X_val)
            oof_preds_gbdt[idx_val] = val_probs
            test_preds_gbdt += model.predict(X_test) / kf.get_n_splits()
            val_preds = (val_probs > 0.3).astype(int)
            f1 = f1_score(Y_val, val_preds, average='micro')
            ia_f1 = ia_weighted_f1(Y_val, val_probs, thr=0.3)
            print(f"  >> Fold {fold+1} micro-F1@0.30: {f1:.4f}")
            print(f"  >> Fold {fold+1} IA-F1@0.30: ALL={ia_f1['ALL']:.4f} MF={ia_f1['MF']:.4f} BP={ia_f1['BP']:.4f} CC={ia_f1['CC']:.4f}")
            model.save(str(WORK_ROOT / 'features' / f'level1_gbdt_fold{fold}.json'))
        np.save(WORK_ROOT / 'features' / 'oof_pred_gbdt.npy', oof_preds_gbdt)
        np.save(WORK_ROOT / 'features' / 'test_pred_gbdt.npy', test_preds_gbdt)
        print("GBDT OOF + test preds saved.")
    # ------------------------------------------
    # C. DNN Ensemble (PyTorch, IA-weighted, multi-input + multi-state)
    # ------------------------------------------
    print("\n--- Training DNN Ensemble (IA-weighted, multimodal, multi-state) ---")
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Build a stable per-label IA weight vector for the current TOP_K targets
    ia_w = weights.copy()
    ia_w = np.where(np.isfinite(ia_w) & (ia_w > 0), ia_w, 1.0).astype(np.float32)
    ia_w = ia_w / float(np.mean(ia_w))
    ia_w = np.clip(ia_w, 0.5, 5.0)
    ia_w_t = torch.tensor(ia_w, dtype=torch.float32, device=device).view(1, -1)
    # Optional: include other model predictions as an input stream (PB OOFs analogue)
    USE_BASE_OOFS_IN_DNN = True
    if USE_BASE_OOFS_IN_DNN and (WORK_ROOT / 'features' / 'oof_pred_logreg.npy').exists():
        oof_stream = [np.load(WORK_ROOT / 'features' / 'oof_pred_logreg.npy').astype(np.float32)]
        test_stream = [np.load(WORK_ROOT / 'features' / 'test_pred_logreg.npy').astype(np.float32)]
        if (WORK_ROOT / 'features' / 'oof_pred_gbdt.npy').exists():
            oof_stream.append(np.load(WORK_ROOT / 'features' / 'oof_pred_gbdt.npy').astype(np.float32))
            test_stream.append(np.load(WORK_ROOT / 'features' / 'test_pred_gbdt.npy').astype(np.float32))
        base_oof = np.hstack(oof_stream)
        base_test = np.hstack(test_stream)
        features_train['base_oof'] = base_oof
        features_test['base_oof'] = base_test
        print(f"Base OOF stream: train={base_oof.shape} test={base_test.shape}")
    # Select modality keys for the DNN (towers)
    DNN_KEYS = [k for k in ['t5', 'esm2_650m', 'esm2_3b', 'ankh', 'taxa', 'text', 'base_oof'] if k in features_train]
    print(f"DNN modality keys={DNN_KEYS}")
    class Tower(nn.Module):
        def __init__(self, in_dim, out_dim=512, dropout=0.1):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(in_dim, 1024),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(1024, out_dim),
                nn.ReLU(),
            )
        def forward(self, x):
            return self.net(x)
    class ColossalMultiModalDNN(nn.Module):
        def __init__(self, dims: dict, output_dim: int):
            super().__init__()
            self.keys = list(dims.keys())
            self.towers = nn.ModuleDict({k: Tower(dims[k]) for k in self.keys})
            fused_dim = 512 * len(self.keys)
            self.head = nn.Sequential(
                nn.Linear(fused_dim, 2048),
                nn.BatchNorm1d(2048),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(2048, 1024),
                nn.BatchNorm1d(1024),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(1024, output_dim),
            )
        def forward(self, batch: dict):
            hs = [self.towers[k](batch[k]) for k in self.keys]
            h = torch.cat(hs, dim=1)
            return self.head(h)
    # Prepare torch tensors per modality
    train_t = {k: torch.tensor(features_train[k], dtype=torch.float32, device=device) for k in DNN_KEYS}
    test_t = {k: torch.tensor(features_test[k], dtype=torch.float32, device=device) for k in DNN_KEYS}
    def _batch_dict(tensors: dict, idx):
        return {k: v[idx] for k, v in tensors.items()}
    # Multi-state ensembling
    DNN_SEEDS = [42, 43, 44, 45, 46]
    DNN_EPOCHS = 10
    BATCH_SIZE = 256
    oof_sum = np.zeros(Y.shape, dtype=np.float32)
    test_sum = np.zeros((len(test_ids), Y.shape[1]), dtype=np.float32)
    n_states = len(DNN_SEEDS)
    for state_i, seed in enumerate(DNN_SEEDS, 1):
        print(f"\n[DNN] Random state {state_i}/{n_states}: seed={seed}")
        torch.manual_seed(seed)
        np.random.seed(seed)
        kf_state = KFold(n_splits=5, shuffle=True, random_state=seed)
        oof_state = np.zeros(Y.shape, dtype=np.float32)
        test_state = np.zeros((len(test_ids), Y.shape[1]), dtype=np.float32)
        dims = {k: int(features_train[k].shape[1]) for k in DNN_KEYS}
        for fold, (idx_tr, idx_val) in enumerate(kf_state.split(train_ids)):
            print(f"DNN Fold {fold+1}/5")
            model = ColossalMultiModalDNN(dims=dims, output_dim=Y.shape[1]).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
            Y_full_t = torch.tensor(Y, dtype=torch.float32, device=device)
            n_samples = len(idx_tr)
            model.train()
            idx_tr_t = torch.tensor(idx_tr, dtype=torch.long, device=device)
            for _epoch in range(DNN_EPOCHS):
                perm = idx_tr_t[torch.randperm(n_samples, device=device)]
                for i in range(0, n_samples, BATCH_SIZE):
                    b = perm[i:i + BATCH_SIZE]
                    optimizer.zero_grad()
                    logits = model(_batch_dict(train_t, b))
                    yb = Y_full_t[b]
                    loss_el = F.binary_cross_entropy_with_logits(logits, yb, reduction='none')
                    loss = (loss_el * ia_w_t).mean()
                    loss.backward()
                    optimizer.step()
            model.eval()
            with torch.no_grad():
                idx_val_t = torch.tensor(idx_val, dtype=torch.long, device=device)
                val_probs = torch.sigmoid(model(_batch_dict(train_t, idx_val_t))).cpu().numpy()
                oof_state[idx_val] = val_probs
                # test prediction (average over folds)
                test_probs = torch.sigmoid(model(test_t)).cpu().numpy()
                test_state += test_probs / kf_state.get_n_splits()
            val_preds = (val_probs > 0.3).astype(int)
            f1 = f1_score(Y[idx_val], val_preds, average='micro')
            ia_f1 = ia_weighted_f1(Y[idx_val], val_probs, thr=0.3)
            print(f"  >> Fold {fold+1} micro-F1@0.30: {f1:.4f}")
            print(f"  >> Fold {fold+1} IA-F1@0.30: ALL={ia_f1['ALL']:.4f} MF={ia_f1['MF']:.4f} BP={ia_f1['BP']:.4f} CC={ia_f1['CC']:.4f}")
            torch.save(model.state_dict(), WORK_ROOT / 'features' / f'level1_dnn_seed{seed}_fold{fold}.pth')
        oof_sum += oof_state
        test_sum += test_state
    oof_preds_dnn = (oof_sum / n_states).astype(np.float32)
    test_preds_dnn = (test_sum / n_states).astype(np.float32)
    np.save(WORK_ROOT / 'features' / 'oof_pred_dnn.npy', oof_preds_dnn)
    np.save(WORK_ROOT / 'features' / 'test_pred_dnn.npy', test_preds_dnn)
    print("DNN OOF + test preds saved (multi-state averaged).")
    # Persist term list for downstream stacker/submission consistency
    with open(WORK_ROOT / 'features' / 'top_terms_1500.json', 'w') as f:
        json.dump(top_terms, f)
    if 'STORE' in globals() and STORE is not None:
        req = [WORK_ROOT / 'features' / 'top_terms_1500.json', WORK_ROOT / 'features' / 'oof_pred_logreg.npy', WORK_ROOT / 'features' / 'test_pred_logreg.npy', WORK_ROOT / 'features' / 'oof_pred_dnn.npy', WORK_ROOT / 'features' / 'test_pred_dnn.npy']
        req += [WORK_ROOT / 'features' / 'oof_pred_gbdt.npy', WORK_ROOT / 'features' / 'test_pred_gbdt.npy']
        STORE.push('stage_07_level1_preds', req, note='Level-1 OOF + test preds (LR/GBDT/DNN)')
    # Diagnostic: IA-F1 vs threshold curve (OOF)
    thrs = np.linspace(0.05, 0.60, 23)
    curves = {k: [] for k in ['ALL', 'MF', 'BP', 'CC']}
    for thr in thrs:
        s = ia_weighted_f1(Y, oof_preds_dnn, thr=float(thr))
        for k in curves.keys():
            curves[k].append(s[k])
    plt.figure(figsize=(10, 3))
    for k in ['ALL', 'MF', 'BP', 'CC']:
        plt.plot(thrs, curves[k], label=k)
    plt.title('DNN OOF: IA-weighted F1 vs threshold (multi-state)')
    plt.xlabel('threshold')
    plt.ylabel('IA-F1')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    print("Phase 2 Complete. OOF + test predictions generated.")
else:
    print("Skipping Phase 2.")


In [ ]:
# CELL 13B - Logistic Regression + checkpoint push
if not TRAIN_LEVEL1:
    print('Skipping LogReg (TRAIN_LEVEL1=False).')
else:
    from sklearn.linear_model import LogisticRegression
    from sklearn.multiclass import OneVsRestClassifier

    PRED_DIR = WORK_ROOT / 'features' / 'level1_preds'
    PRED_DIR.mkdir(parents=True, exist_ok=True)

    lr_oof_path = PRED_DIR / 'oof_pred_logreg.npy'
    lr_test_path = PRED_DIR / 'test_pred_logreg.npy'

    if lr_oof_path.exists() and lr_test_path.exists() and CAFA_FORCE_REBUILD != '1':
        print('LogReg preds exist; skipping training (set CAFA_FORCE_REBUILD=1 to force).')
        oof_pred_logreg = np.load(lr_oof_path)
        test_pred_logreg = np.load(lr_test_path)
    else:
        n_splits = 5
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        oof_pred_logreg = np.zeros((X.shape[0], Y.shape[1]), dtype=np.float32)
        test_pred_logreg = np.zeros((X_test.shape[0], Y.shape[1]), dtype=np.float32)

        for fold, (tr_idx, va_idx) in enumerate(kf.split(X), start=1):
            print(f'Fold {fold}/{n_splits}')
            base = LogisticRegression(
                C=2.0,
                solver='saga',
                penalty='l2',
                max_iter=200,
                n_jobs=-1,
                verbose=0,
            )
            clf = OneVsRestClassifier(base, n_jobs=-1)
            clf.fit(X[tr_idx], Y[tr_idx])
            va = clf.predict_proba(X[va_idx]).astype(np.float32)
            te = clf.predict_proba(X_test).astype(np.float32)
            oof_pred_logreg[va_idx] = va
            test_pred_logreg += te / float(n_splits)
            print('  IA-F1:', ia_weighted_f1(Y[va_idx], va, thr=0.3))

        np.save(lr_oof_path, oof_pred_logreg)
        np.save(lr_test_path, test_pred_logreg)
        print('Saved:', lr_oof_path)
        print('Saved:', lr_test_path)

    # Checkpoint push (always)
    STORE.push(
        stage='stage_07a_level1_logreg',
        required_paths=[
            str((WORK_ROOT / 'features' / 'top_terms_1500.json').as_posix()),
            str(lr_oof_path.as_posix()),
            str(lr_test_path.as_posix()),
        ],
        note='Level-1 Logistic Regression predictions (OOF + test).',
    )

    # Diagnostics: probability histograms + IA-F1 vs threshold (sampled)
    try:
        import os
        import numpy as np
        import matplotlib.pyplot as plt

        plt.rcParams.update({'font.size': 12})
        DIAG_N = int(os.getenv('CAFA_DIAG_N', '20000'))

        def _sub(y_true: np.ndarray, y_score: np.ndarray):
            n = int(y_true.shape[0])
            m = min(n, int(DIAG_N))
            if m <= 0:
                return y_true[:0], y_score[:0]
            idx = np.linspace(0, n - 1, num=m, dtype=np.int64)
            return y_true[idx], y_score[idx]

        y_t, y_s = _sub(Y, oof_pred_logreg)
        row_max_oof = y_s.max(axis=1)
        row_mean_oof = y_s.mean(axis=1)

        plt.figure(figsize=(10, 4))
        plt.hist(row_max_oof, bins=60, alpha=0.7)
        plt.title('LogReg OOF: max probability per protein (sampled)')
        plt.xlabel('max prob')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.show()

        plt.figure(figsize=(10, 4))
        plt.hist(row_mean_oof, bins=60, alpha=0.7)
        plt.title('LogReg OOF: mean probability per protein (sampled)')
        plt.xlabel('mean prob')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.show()

        if test_pred_logreg is not None:
            te_s = test_pred_logreg
            te_m = min(int(te_s.shape[0]), int(DIAG_N))
            te_idx = np.linspace(0, int(te_s.shape[0]) - 1, num=te_m, dtype=np.int64) if te_m > 0 else np.array([], dtype=np.int64)
            row_max_te = te_s[te_idx].max(axis=1) if te_m > 0 else np.array([], dtype=np.float32)
            plt.figure(figsize=(10, 4))
            plt.hist(row_max_oof, bins=60, alpha=0.5, label='OOF')
            plt.hist(row_max_te, bins=60, alpha=0.5, label='test')
            plt.title('LogReg: max probability per protein (OOF vs test; sampled)')
            plt.xlabel('max prob')
            plt.ylabel('count')
            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.show()

        if 'ia_weighted_f1' in globals():
            thrs = np.linspace(0.05, 0.60, 23)
            curves = {k: [] for k in ['ALL', 'MF', 'BP', 'CC']}
            for thr in thrs:
                s = ia_weighted_f1(y_t, y_s, thr=float(thr))
                for k in curves.keys():
                    curves[k].append(s[k])
            plt.figure(figsize=(10, 3))
            for k in ['ALL', 'MF', 'BP', 'CC']:
                plt.plot(thrs, curves[k], label=k)
            plt.title('LogReg OOF: IA-weighted F1 vs threshold (sampled)')
            plt.xlabel('threshold')
            plt.ylabel('IA-F1')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()
    except Exception as e:
        print('LogReg diagnostics skipped:', repr(e))


In [ ]:
# CELL 13C - GBDT (py-boost) + checkpoint push
if not TRAIN_LEVEL1:
    print('Skipping GBDT (TRAIN_LEVEL1=False).')
else:
    PRED_DIR = WORK_ROOT / 'features' / 'level1_preds'
    PRED_DIR.mkdir(parents=True, exist_ok=True)

    gbdt_oof_path = PRED_DIR / 'oof_pred_gbdt.npy'
    gbdt_test_path = PRED_DIR / 'test_pred_gbdt.npy'

    if gbdt_oof_path.exists() and gbdt_test_path.exists() and CAFA_FORCE_REBUILD != '1':
        print('GBDT preds exist; skipping training (set CAFA_FORCE_REBUILD=1 to force).')
        oof_pred_gbdt = np.load(gbdt_oof_path)
        test_pred_gbdt = np.load(gbdt_test_path)
    else:
        try:
            from py_boost import GradientBoosting
            from py_boost.multioutput import MultiOutputRegressor
        except Exception as e:
            raise RuntimeError('py_boost is REQUIRED for this pipeline (GBDT is mandatory). Install it with: pip install py-boost') from e

        # Note: Multi-label probabilities via regressor outputs squashed to [0,1]
        n_splits = 5
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        oof_pred_gbdt = np.zeros((X.shape[0], Y.shape[1]), dtype=np.float32)
        test_pred_gbdt = np.zeros((X_test.shape[0], Y.shape[1]), dtype=np.float32)

        for fold, (tr_idx, va_idx) in enumerate(kf.split(X), start=1):
            print(f'Fold {fold}/{n_splits}')
            base = GradientBoosting(loss='logloss', ntrees=150, lr=0.05, max_depth=6, subsample=0.8, colsample=0.8, random_state=42 + fold)
            model = MultiOutputRegressor(base)
            model.fit(X[tr_idx], Y[tr_idx])
            va = model.predict(X[va_idx]).astype(np.float32)
            te = model.predict(X_test).astype(np.float32)
            # safety clamp
            va = np.clip(va, 0.0, 1.0)
            te = np.clip(te, 0.0, 1.0)
            oof_pred_gbdt[va_idx] = va
            test_pred_gbdt += te / float(n_splits)
            print('  IA-F1:', ia_weighted_f1(Y[va_idx], va, thr=0.3))

        np.save(gbdt_oof_path, oof_pred_gbdt)
        np.save(gbdt_test_path, test_pred_gbdt)
        print('Saved:', gbdt_oof_path)
        print('Saved:', gbdt_test_path)

    # Checkpoint push (always)
    STORE.push(
        stage='stage_07b_level1_gbdt',
        required_paths=[
            str((WORK_ROOT / 'features' / 'top_terms_1500.json').as_posix()),
            str(gbdt_oof_path.as_posix()),
            str(gbdt_test_path.as_posix()),
        ],
        note='Level-1 GBDT predictions (OOF + test).',
    )

    # Diagnostics: probability histograms + IA-F1 vs threshold (sampled)
    try:
        import os
        import numpy as np
        import matplotlib.pyplot as plt

        plt.rcParams.update({'font.size': 12})
        DIAG_N = int(os.getenv('CAFA_DIAG_N', '20000'))

        def _sub(y_true: np.ndarray, y_score: np.ndarray):
            n = int(y_true.shape[0])
            m = min(n, int(DIAG_N))
            if m <= 0:
                return y_true[:0], y_score[:0]
            idx = np.linspace(0, n - 1, num=m, dtype=np.int64)
            return y_true[idx], y_score[idx]

        y_t, y_s = _sub(Y, oof_pred_gbdt)
        row_max_oof = y_s.max(axis=1)
        row_mean_oof = y_s.mean(axis=1)

        plt.figure(figsize=(10, 4))
        plt.hist(row_max_oof, bins=60, alpha=0.7)
        plt.title('GBDT OOF: max probability per protein (sampled)')
        plt.xlabel('max prob')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.show()

        plt.figure(figsize=(10, 4))
        plt.hist(row_mean_oof, bins=60, alpha=0.7)
        plt.title('GBDT OOF: mean probability per protein (sampled)')
        plt.xlabel('mean prob')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.show()

        if test_pred_gbdt is not None:
            te_s = test_pred_gbdt
            te_m = min(int(te_s.shape[0]), int(DIAG_N))
            te_idx = np.linspace(0, int(te_s.shape[0]) - 1, num=te_m, dtype=np.int64) if te_m > 0 else np.array([], dtype=np.int64)
            row_max_te = te_s[te_idx].max(axis=1) if te_m > 0 else np.array([], dtype=np.float32)
            plt.figure(figsize=(10, 4))
            plt.hist(row_max_oof, bins=60, alpha=0.5, label='OOF')
            plt.hist(row_max_te, bins=60, alpha=0.5, label='test')
            plt.title('GBDT: max probability per protein (OOF vs test; sampled)')
            plt.xlabel('max prob')
            plt.ylabel('count')
            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.show()

        if 'ia_weighted_f1' in globals():
            thrs = np.linspace(0.05, 0.60, 23)
            curves = {k: [] for k in ['ALL', 'MF', 'BP', 'CC']}
            for thr in thrs:
                s = ia_weighted_f1(y_t, y_s, thr=float(thr))
                for k in curves.keys():
                    curves[k].append(s[k])
            plt.figure(figsize=(10, 3))
            for k in ['ALL', 'MF', 'BP', 'CC']:
                plt.plot(thrs, curves[k], label=k)
            plt.title('GBDT OOF: IA-weighted F1 vs threshold (sampled)')
            plt.xlabel('threshold')
            plt.ylabel('IA-F1')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()
    except Exception as e:
        print('GBDT diagnostics skipped:', repr(e))


In [ ]:
# CELL 13D - DNN (PyTorch) + checkpoint push
if not TRAIN_LEVEL1:
    print('Skipping DNN (TRAIN_LEVEL1=False).')
else:
    import torch
    import torch.nn as nn
    from torch.utils.data import Dataset, DataLoader

    PRED_DIR = WORK_ROOT / 'features' / 'level1_preds'
    PRED_DIR.mkdir(parents=True, exist_ok=True)

    dnn_oof_path = PRED_DIR / 'oof_pred_dnn.npy'
    dnn_test_path = PRED_DIR / 'test_pred_dnn.npy'

    if dnn_oof_path.exists() and dnn_test_path.exists() and CAFA_FORCE_REBUILD != '1':
        print('DNN preds exist; skipping training (set CAFA_FORCE_REBUILD=1 to force).')
        oof_pred_dnn = np.load(dnn_oof_path)
        test_pred_dnn = np.load(dnn_test_path)
    else:
        torch.manual_seed(42)
        np.random.seed(42)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print('DNN device:', device)

        # Keep DNN input identical to flat concatenation (for now)
        X_dnn = X.astype(np.float32)
        X_dnn_test = X_test.astype(np.float32)
        y_dnn = Y.astype(np.float32)

        class NumpyDataset(Dataset):
            def __init__(self, X, y=None):
                self.X = torch.from_numpy(X)
                self.y = None if y is None else torch.from_numpy(y)
            def __len__(self):
                return self.X.shape[0]
            def __getitem__(self, idx):
                if self.y is None:
                    return self.X[idx]
                return self.X[idx], self.y[idx]

        class MLP(nn.Module):
            def __init__(self, in_dim, out_dim, hidden=2048, p=0.2):
                super().__init__()
                self.net = nn.Sequential(
                    nn.Linear(in_dim, hidden),
                    nn.BatchNorm1d(hidden),
                    nn.ReLU(),
                    nn.Dropout(p),
                    nn.Linear(hidden, hidden),
                    nn.BatchNorm1d(hidden),
                    nn.ReLU(),
                    nn.Dropout(p),
                    nn.Linear(hidden, out_dim),
                )
            def forward(self, x):
                return self.net(x)

        def train_one_fold(train_idx, val_idx, seed=0, epochs=3, batch_size=256, lr=1e-3):
            torch.manual_seed(42 + seed)
            model = MLP(X_dnn.shape[1], y_dnn.shape[1]).to(device)
            opt = torch.optim.AdamW(model.parameters(), lr=lr)
            # IA-weighted BCE (weights broadcast over classes)
            w = torch.from_numpy(weights).to(device)
            bce = nn.BCEWithLogitsLoss(reduction='none')

            dl_tr = DataLoader(NumpyDataset(X_dnn[train_idx], y_dnn[train_idx]), batch_size=batch_size, shuffle=True, drop_last=False)
            dl_va = DataLoader(NumpyDataset(X_dnn[val_idx], y_dnn[val_idx]), batch_size=batch_size, shuffle=False, drop_last=False)
            for ep in range(1, epochs + 1):
                model.train()
                for xb, yb in dl_tr:
                    xb = xb.to(device, non_blocking=True).float()
                    yb = yb.to(device, non_blocking=True).float()
                    logits = model(xb)
                    loss_per = bce(logits, yb)
                    loss = (loss_per * w).mean()
                    opt.zero_grad(set_to_none=True)
                    loss.backward()
                    opt.step()
                # quick val metric
                model.eval()
                with torch.no_grad():
                    va_scores = []
                    va_true = []
                    for xb, yb in dl_va:
                        xb = xb.to(device, non_blocking=True).float()
                        logits = model(xb)
                        va_scores.append(torch.sigmoid(logits).cpu().numpy())
                        va_true.append(yb.numpy())
                    va_scores = np.vstack(va_scores)
                    va_true = np.vstack(va_true)
                    s = ia_weighted_f1(va_true, va_scores, thr=0.3)
                print(f'  seed={seed} ep={ep}/{epochs} IA-F1={s}')
            return model

        n_splits = 5
        n_seeds = 2
        epochs = 3
        batch_size = 256
        lr = 1e-3

        oof_pred_dnn = np.zeros((X_dnn.shape[0], y_dnn.shape[1]), dtype=np.float32)
        test_pred_dnn = np.zeros((X_dnn_test.shape[0], y_dnn.shape[1]), dtype=np.float32)
        counts = np.zeros((X_dnn.shape[0], 1), dtype=np.float32)

        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        for fold, (tr_idx, va_idx) in enumerate(kf.split(X_dnn), start=1):
            print(f'Fold {fold}/{n_splits}')
            fold_test = np.zeros_like(test_pred_dnn)
            for seed in range(n_seeds):
                model = train_one_fold(tr_idx, va_idx, seed=seed, epochs=epochs, batch_size=batch_size, lr=lr)
                model.eval()
                # OOF
                dl_va = DataLoader(NumpyDataset(X_dnn[va_idx], None), batch_size=1024, shuffle=False)
                preds_va = []
                with torch.no_grad():
                    for xb in dl_va:
                        xb = xb.to(device, non_blocking=True).float()
                        preds_va.append(torch.sigmoid(model(xb)).cpu().numpy())
                preds_va = np.vstack(preds_va).astype(np.float32)
                oof_pred_dnn[va_idx] += preds_va
                counts[va_idx] += 1.0
                # TEST
                dl_te = DataLoader(NumpyDataset(X_dnn_test, None), batch_size=1024, shuffle=False)
                preds_te = []
                with torch.no_grad():
                    for xb in dl_te:
                        xb = xb.to(device, non_blocking=True).float()
                        preds_te.append(torch.sigmoid(model(xb)).cpu().numpy())
                preds_te = np.vstack(preds_te).astype(np.float32)
                fold_test += preds_te
            test_pred_dnn += (fold_test / float(n_seeds))
        oof_pred_dnn = (oof_pred_dnn / np.maximum(counts, 1.0)).astype(np.float32)
        test_pred_dnn = (test_pred_dnn / float(n_splits)).astype(np.float32)

        np.save(dnn_oof_path, oof_pred_dnn)
        np.save(dnn_test_path, test_pred_dnn)
        print('Saved:', dnn_oof_path)
        print('Saved:', dnn_test_path)

    # Checkpoint push (always)
    STORE.push(
        stage='stage_07c_level1_dnn',
        required_paths=[
            str((WORK_ROOT / 'features' / 'top_terms_1500.json').as_posix()),
            str(dnn_oof_path.as_posix()),
            str(dnn_test_path.as_posix()),
        ],
        note='Level-1 DNN predictions (OOF + test).',
    )

    # Diagnostics: probability histograms + IA-F1 vs threshold (sampled)
    try:
        import os
        import numpy as np
        import matplotlib.pyplot as plt

        plt.rcParams.update({'font.size': 12})
        DIAG_N = int(os.getenv('CAFA_DIAG_N', '20000'))

        def _sub(y_true: np.ndarray, y_score: np.ndarray):
            n = int(y_true.shape[0])
            m = min(n, int(DIAG_N))
            if m <= 0:
                return y_true[:0], y_score[:0]
            idx = np.linspace(0, n - 1, num=m, dtype=np.int64)
            return y_true[idx], y_score[idx]

        y_t, y_s = _sub(Y, oof_pred_dnn)
        row_max_oof = y_s.max(axis=1)
        row_mean_oof = y_s.mean(axis=1)

        plt.figure(figsize=(10, 4))
        plt.hist(row_max_oof, bins=60, alpha=0.7)
        plt.title('DNN OOF: max probability per protein (sampled)')
        plt.xlabel('max prob')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.show()

        plt.figure(figsize=(10, 4))
        plt.hist(row_mean_oof, bins=60, alpha=0.7)
        plt.title('DNN OOF: mean probability per protein (sampled)')
        plt.xlabel('mean prob')
        plt.ylabel('count')
        plt.grid(True, alpha=0.3)
        plt.show()

        if test_pred_dnn is not None:
            te_s = test_pred_dnn
            te_m = min(int(te_s.shape[0]), int(DIAG_N))
            te_idx = np.linspace(0, int(te_s.shape[0]) - 1, num=te_m, dtype=np.int64) if te_m > 0 else np.array([], dtype=np.int64)
            row_max_te = te_s[te_idx].max(axis=1) if te_m > 0 else np.array([], dtype=np.float32)
            plt.figure(figsize=(10, 4))
            plt.hist(row_max_oof, bins=60, alpha=0.5, label='OOF')
            plt.hist(row_max_te, bins=60, alpha=0.5, label='test')
            plt.title('DNN: max probability per protein (OOF vs test; sampled)')
            plt.xlabel('max prob')
            plt.ylabel('count')
            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.show()

        if 'ia_weighted_f1' in globals():
            thrs = np.linspace(0.05, 0.60, 23)
            curves = {k: [] for k in ['ALL', 'MF', 'BP', 'CC']}
            for thr in thrs:
                s = ia_weighted_f1(y_t, y_s, thr=float(thr))
                for k in curves.keys():
                    curves[k].append(s[k])
            plt.figure(figsize=(10, 3))
            for k in ['ALL', 'MF', 'BP', 'CC']:
                plt.plot(thrs, curves[k], label=k)
            plt.title('DNN OOF: IA-weighted F1 vs threshold (sampled)')
            plt.xlabel('threshold')
            plt.ylabel('IA-F1')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()
    except Exception as e:
        print('DNN diagnostics skipped:', repr(e))


In [ ]:
# CELL 13E - KNN (cosine; ESM2-3B) + checkpoint push
if not TRAIN_LEVEL1:
    print('Skipping KNN (TRAIN_LEVEL1=False).')
else:
    from sklearn.neighbors import NearestNeighbors
    from sklearn.model_selection import KFold
    import json

    PRED_DIR = WORK_ROOT / 'features' / 'level1_preds'
    PRED_DIR.mkdir(parents=True, exist_ok=True)

    knn_oof_path = PRED_DIR / 'oof_pred_knn.npy'
    knn_test_path = PRED_DIR / 'test_pred_knn.npy'

    # Backwards-compatible copies (some downstream code loads from WORK_ROOT/features)
    knn_oof_compat = WORK_ROOT / 'features' / 'oof_pred_knn.npy'
    knn_test_compat = WORK_ROOT / 'features' / 'test_pred_knn.npy'

    if knn_oof_path.exists() and knn_test_path.exists() and CAFA_FORCE_REBUILD != '1':
        print('KNN preds exist; skipping training (set CAFA_FORCE_REBUILD=1 to force).')
        oof_pred_knn = np.load(knn_oof_path)
        test_pred_knn = np.load(knn_test_path)
        oof_max_sim = None
    else:
        if 'features_train' not in globals() or 'features_test' not in globals():
            raise RuntimeError('Missing `features_train`/`features_test`. Run the Phase 2 feature load cell first.')
        if 'esm2_3b' not in features_train:
            raise FileNotFoundError("Missing required modality 'esm2_3b' in features_train. Ensure features/train_embeds_esm2_3b.npy exists.")

        X_knn = features_train['esm2_3b'].astype(np.float32)
        X_knn_test = features_test['esm2_3b'].astype(np.float32)
        # Enforce TOP_K alignment using the persisted term list.
        top_terms_path = WORK_ROOT / 'features' / 'top_terms_1500.json'
        if top_terms_path.exists():
            top_terms_knn = json.loads(top_terms_path.read_text())
            if Y.shape[1] != len(top_terms_knn):
                raise ValueError(f'KNN shape mismatch: Y has {Y.shape[1]} cols but top_terms_1500.json has {len(top_terms_knn)} terms.')
        else:
            # Fall back to in-memory top_terms if present (should be created in Phase 2 cell).
            if 'top_terms' not in globals():
                raise RuntimeError('Missing top_terms_1500.json and in-memory top_terms. Run the Phase 2 cell first.')
            top_terms_knn = list(top_terms)
            if Y.shape[1] != len(top_terms_knn):
                raise ValueError(f'KNN shape mismatch: Y has {Y.shape[1]} cols but top_terms has {len(top_terms_knn)} terms.')

        # KNN needs binary targets (presence/absence), not counts.
        Y_knn = (Y > 0).astype(np.float32)

        def _l2_norm(x: np.ndarray, eps: float = 1e-12) -> np.ndarray:
            n = np.linalg.norm(x, axis=1, keepdims=True)
            return x / np.maximum(n, eps)

        # Cosine distance is best-behaved on L2-normalised vectors
        X_knn = _l2_norm(X_knn)
        X_knn_test = _l2_norm(X_knn_test)

        KNN_K = int(globals().get('KNN_K', 50))
        KNN_BATCH = int(globals().get('KNN_BATCH', 256))

        n_splits = 5
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

        oof_pred_knn = np.zeros((X_knn.shape[0], Y_knn.shape[1]), dtype=np.float32)
        test_pred_knn = np.zeros((X_knn_test.shape[0], Y_knn.shape[1]), dtype=np.float32)
        oof_max_sim = np.zeros((X_knn.shape[0],), dtype=np.float32)

        for fold, (tr_idx, va_idx) in enumerate(kf.split(X_knn), start=1):
            print(f'Fold {fold}/{n_splits} (KNN)')
            knn = NearestNeighbors(n_neighbors=KNN_K, metric='cosine', n_jobs=-1)
            knn.fit(X_knn[tr_idx])

            dists, neigh = knn.kneighbors(X_knn[va_idx], return_distance=True)
            sims = np.clip((1.0 - dists).astype(np.float32), 0.0, 1.0)
            oof_max_sim[va_idx] = sims.max(axis=1)
            neigh_global = tr_idx[neigh]  # map to global row indices into Y_knn

            for i in range(0, len(va_idx), KNN_BATCH):
                j = min(i + KNN_BATCH, len(va_idx))
                neigh_b = neigh_global[i:j]
                sims_b = sims[i:j]
                denom = np.maximum(sims_b.sum(axis=1, keepdims=True), 1e-8)
                Y_nei = Y_knn[neigh_b]  # (B, K, L)
                scores = (np.einsum('bk,bkl->bl', sims_b, Y_nei) / denom).astype(np.float32)
                oof_pred_knn[va_idx[i:j]] = scores

            if 'ia_weighted_f1' in globals():
                print('  IA-F1:', ia_weighted_f1(Y_knn[va_idx], oof_pred_knn[va_idx], thr=0.3))

        # Final model on full train -> test
        knn_final = NearestNeighbors(n_neighbors=KNN_K, metric='cosine', n_jobs=-1)
        knn_final.fit(X_knn)
        dists_te, neigh_te = knn_final.kneighbors(X_knn_test, return_distance=True)
        sims_te = np.clip((1.0 - dists_te).astype(np.float32), 0.0, 1.0)
        denom_te = np.maximum(sims_te.sum(axis=1, keepdims=True), 1e-8)

        for i in range(0, X_knn_test.shape[0], KNN_BATCH):
            j = min(i + KNN_BATCH, X_knn_test.shape[0])
            neigh_b = neigh_te[i:j]
            sims_b = sims_te[i:j]
            Y_nei = Y_knn[neigh_b]
            scores = (np.einsum('bk,bkl->bl', sims_b, Y_nei) / denom_te[i:j]).astype(np.float32)
            test_pred_knn[i:j] = scores

        np.save(knn_oof_path, oof_pred_knn)
        np.save(knn_test_path, test_pred_knn)
        np.save(knn_oof_compat, oof_pred_knn)
        np.save(knn_test_compat, test_pred_knn)
        print('Saved:', knn_oof_path)
        print('Saved:', knn_test_path)
        print('Saved:', knn_oof_compat)
        print('Saved:', knn_test_compat)

    # Checkpoint push (always)
    STORE.push(
        stage='stage_07d_level1_knn',
        required_paths=[
            str((WORK_ROOT / 'features' / 'top_terms_1500.json').as_posix()),
            str(knn_oof_path.as_posix()),
            str(knn_test_path.as_posix()),
        ],
        note='Level-1 KNN (cosine) predictions using ESM2-3B embeddings (OOF + test).',
    )

    # Diagnostics: similarity distribution + IA-F1 vs threshold
    try:
        import matplotlib.pyplot as plt

        if oof_max_sim is not None:
            plt.figure(figsize=(10, 4))
            plt.hist(oof_max_sim, bins=50)
            plt.title('KNN OOF diagnostic: max cosine similarity to neighbours (per protein)')
            plt.xlabel('max similarity')
            plt.ylabel('count')
            plt.grid(True, alpha=0.3)
            plt.show()

        if 'ia_weighted_f1' in globals():
            thrs = np.linspace(0.05, 0.60, 23)
            curves = {k: [] for k in ['ALL', 'MF', 'BP', 'CC']}
            for thr in thrs:
                s = ia_weighted_f1(Y, oof_pred_knn, thr=float(thr))
                for k in curves.keys():
                    curves[k].append(s[k])
            plt.figure(figsize=(10, 3))
            for k in ['ALL', 'MF', 'BP', 'CC']:
                plt.plot(thrs, curves[k], label=k)
            plt.title('KNN OOF: IA-weighted F1 vs threshold')
            plt.xlabel('threshold')
            plt.ylabel('IA-F1')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()
    except Exception as e:
        print('KNN diagnostics skipped:', repr(e))


In [ ]:
# CELL 14 - Solution: 5. PHASE 3: HIERARCHY-AWARE STACKING (GCN)
# 5. PHASE 3: HIERARCHY-AWARE STACKING (GCN)
# ==========================================
# NOTE: This cell is kept as a lightweight alternative stacker.
# The main stacker used by Phase 4 is implemented in the next cell.
TRAIN_STACKER = False
if TRAIN_STACKER:
    import torch
    import torch.nn as nn
    from sklearn.metrics import f1_score
    print("Loading OOF Predictions for stacking...")
    oof_logreg = np.load(WORK_ROOT / 'features' / 'oof_pred_logreg.npy')
    try:
        oof_gbdt = np.load(WORK_ROOT / 'features' / 'oof_pred_gbdt.npy')
    except Exception:
        oof_gbdt = np.zeros_like(oof_logreg)
    try:
        oof_dnn = np.load(WORK_ROOT / 'features' / 'oof_pred_dnn.npy')
    except Exception:
        oof_dnn = np.zeros_like(oof_logreg)
    try:
        oof_knn = np.load(WORK_ROOT / 'features' / 'oof_pred_knn.npy')
    except Exception:
        oof_knn = np.zeros_like(oof_logreg)
    X_stack = np.hstack([oof_logreg, oof_gbdt, oof_dnn, oof_knn])
    Y_stack = Y
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    X_stack_t = torch.tensor(X_stack, dtype=torch.float32).to(device)
    Y_stack_t = torch.tensor(Y_stack, dtype=torch.float32).to(device)
    class Stacker(nn.Module):
        def __init__(self, input_dim, output_dim):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(input_dim, 2048),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(2048, output_dim),
                nn.Sigmoid(),
            )
        def forward(self, x):
            return self.net(x)
    stacker = Stacker(X_stack.shape[1], Y_stack.shape[1]).to(device)
    optimizer = torch.optim.Adam(stacker.parameters(), lr=1e-3)
    criterion = nn.BCELoss()
    stacker.train()
    for epoch in range(20):
        optimizer.zero_grad()
        out = stacker(X_stack_t)
        loss = criterion(out, Y_stack_t)
        loss.backward()
        optimizer.step()
        if epoch % 5 == 0:
            with torch.no_grad():
                preds = (out > 0.3).float()
                f1 = f1_score(Y_stack_t.cpu().numpy()[:1000], preds.cpu().numpy()[:1000], average='micro')
            print(f"Epoch {epoch}: Loss {loss.item():.4f}, Approx micro-F1@0.30 {f1:.4f}")
    torch.save(stacker.state_dict(), WORK_ROOT / 'features' / 'final_stacker.pth')
    print("Saved: final_stacker.pth")
    stacker.eval()
    with torch.no_grad():
        final_preds = stacker(X_stack_t).cpu().numpy()
        final_f1 = f1_score(Y_stack, (final_preds > 0.3).astype(int), average='micro')
    print(f"Final Stacker micro-F1@0.30: {final_f1:.4f}")
else:
    print("Skipping Phase 3 (lightweight stacker).")

In [ ]:
# CELL 15 - Solution: 5b. PHASE 3: HIERARCHY-AWARE STACKING (GRAPH SMOOTHING GCN)
# 5b. PHASE 3: HIERARCHY-AWARE STACKING (GRAPH SMOOTHING GCN)
# =========================================================
# Option B strictness: if PROCESS_EXTERNAL=True, we REQUIRE the propagated prior files to exist.
PROCESS_EXTERNAL = globals().get('PROCESS_EXTERNAL', True)
import numpy as np
import pandas as pd
import json
from pathlib import Path
from sklearn.metrics import f1_score
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
TOP_K = 1500

def _load_level1_pred(fname: str):
    """Load Level-1 prediction arrays from either features/level1_preds or features/."""
    cand = [
        WORK_ROOT / 'features' / 'level1_preds' / fname,
        WORK_ROOT / 'features' / fname,
    ]
    for p in cand:
        if p.exists():
            return np.load(p).astype(np.float32)
    return None

# 1. Load Level-1 predictions
train_feats = []
for fname in ['oof_pred_logreg.npy', 'oof_pred_gbdt.npy', 'oof_pred_dnn.npy', 'oof_pred_knn.npy']:
    arr = _load_level1_pred(fname)
    if arr is not None:
        train_feats.append(arr)
if not train_feats:
    raise FileNotFoundError('No Level-1 OOF predictions found. Run Phase 2 first.')
X_stack = np.mean(train_feats, axis=0).astype(np.float32)

# 2. Build Y label matrix for top-K terms
train_terms = pd.read_parquet(WORK_ROOT / 'parsed' / 'train_terms.parquet')
train_ids = pd.read_feather(WORK_ROOT / 'parsed' / 'train_seq.feather')['id'].astype(str)
top_terms = train_terms['term'].value_counts().head(TOP_K).index.tolist()
train_terms_top = train_terms[train_terms['term'].isin(top_terms)]
Y_df = train_terms_top.pivot_table(index='EntryID', columns='term', aggfunc='size', fill_value=0)
Y_df = Y_df.reindex(train_ids, fill_value=0)
Y = Y_df.values.astype(np.float32)

# 2b. External priors (Phase 1 Step 4 outputs) -> inject as *conservative* extra signal
EXTERNAL_PRIOR_WEIGHT = 0.25
ext_dir = WORK_ROOT / 'external'
prior_train_path = ext_dir / 'prop_train_no_kaggle.tsv.gz'
if PROCESS_EXTERNAL:
    if not prior_train_path.exists():
        raise FileNotFoundError(
            f'Option B requires external priors, but missing: {prior_train_path}. '
            'Run Phase 1 Step 4 propagation or ensure your checkpoint dataset contains these files (run setup: STORE.pull()).',
        )
    prior_train = pd.read_csv(prior_train_path, sep='\t')
    prior_train = prior_train[prior_train['term'].isin(top_terms)]
    prior_mat = prior_train.pivot_table(index='EntryID', columns='term', values='score', aggfunc='max', fill_value=0.0)
    prior_mat = prior_mat.reindex(train_ids.tolist(), fill_value=0.0)
    prior_mat = prior_mat.reindex(columns=top_terms, fill_value=0.0)
    prior_np = prior_mat.values.astype(np.float32)
    X_stack = np.maximum(X_stack, EXTERNAL_PRIOR_WEIGHT * prior_np)
    print(f'Injected external IEA prior into train stack (weight={EXTERNAL_PRIOR_WEIGHT}).')
(WORK_ROOT / 'features').mkdir(parents=True, exist_ok=True)
with open(WORK_ROOT / 'features' / 'top_terms_1500.json', 'w') as f:
    json.dump(top_terms, f)
print('Saved: top_terms_1500.json')

# 3. Graph adjacency from go-basic.obo (reload if needed)
if 'go_parents' not in locals() or 'go_namespaces' not in locals():
    print('Reloading GO graph (parse_obo)...')
    def parse_obo(path: Path):
        parents = {}
        namespaces = {}
        cur_id, cur_ns = None, None
        with path.open('r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line == '[Term]':
                    if cur_id and cur_ns:
                        namespaces[cur_id] = cur_ns
                    cur_id, cur_ns = None, None
                elif line.startswith('id: GO:'):
                    cur_id = line.split('id: ', 1)[1]
                elif line.startswith('namespace:'):
                    cur_ns = line.split('namespace: ', 1)[1]
                elif line.startswith('is_a:') and cur_id:
                    parent = line.split('is_a: ', 1)[1].split(' ! ')[0]
                    parents.setdefault(cur_id, set()).add(parent)
            if cur_id and cur_ns:
                namespaces[cur_id] = cur_ns
        return parents, namespaces
    go_parents, go_namespaces = parse_obo(PATH_GO_OBO)
def build_adjacency(terms_list, parents_dict):
    term_to_idx = {t: i for i, t in enumerate(terms_list)}
    n_terms = len(terms_list)
    src, dst = [], []
    for child in terms_list:
        parents = parents_dict.get(child, set())
        if not parents:
            continue
        child_idx = term_to_idx[child]
        for parent in parents:
            if parent in term_to_idx:
                parent_idx = term_to_idx[parent]
                src.append(child_idx); dst.append(parent_idx)
                src.append(parent_idx); dst.append(child_idx)
    src.extend(range(n_terms))
    dst.extend(range(n_terms))
    indices = torch.tensor([src, dst], dtype=torch.long)
    values = torch.ones(len(src), dtype=torch.float32)
    return torch.sparse_coo_tensor(indices, values, (n_terms, n_terms)).coalesce().to(device)
class SimpleGCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, adj_matrix):
        super().__init__()
        self.adj = adj_matrix
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = torch.sparse.mm(self.adj, x.t()).t()
        return torch.sigmoid(x)

# 4. Build test stack once (and inject external priors once), then split by ontology
print('\nPreparing test stack...')
test_feats = []
for fname in ['test_pred_logreg.npy', 'test_pred_gbdt.npy', 'test_pred_dnn.npy', 'test_pred_knn.npy']:
    arr = _load_level1_pred(fname)
    if arr is not None:
        test_feats.append(arr)
if not test_feats:
    raise FileNotFoundError('No Level-1 test predictions found. Run Phase 2 first.')
X_test_stack = np.mean(test_feats, axis=0).astype(np.float32)
prior_test_path = ext_dir / 'prop_test_no_kaggle.tsv.gz'
if PROCESS_EXTERNAL:
    if not prior_test_path.exists():
        raise FileNotFoundError(
            f'Option B requires external priors, but missing: {prior_test_path}. '
            'Run Phase 1 Step 4 propagation or ensure your checkpoint dataset contains these files (run setup: STORE.pull()).',
        )
    test_ids = pd.read_feather(WORK_ROOT / 'parsed' / 'test_seq.feather')['id'].astype(str)
    prior_test = pd.read_csv(prior_test_path, sep='\t')
    prior_test = prior_test[prior_test['term'].isin(top_terms)]
    prior_t = prior_test.pivot_table(index='EntryID', columns='term', values='score', aggfunc='max', fill_value=0.0)
    prior_t = prior_t.reindex(test_ids.tolist(), fill_value=0.0)
    prior_t = prior_t.reindex(columns=top_terms, fill_value=0.0)
    prior_test_np = prior_t.values.astype(np.float32)
    X_test_stack = np.maximum(X_test_stack, EXTERNAL_PRIOR_WEIGHT * prior_test_np)
    print(f'Injected external IEA prior into test stack (weight={EXTERNAL_PRIOR_WEIGHT}).')

# 5. Ontology split (BP/MF/CC)
ns_to_aspect = {
    'molecular_function': 'MF',
    'biological_process': 'BP',
    'cellular_component': 'CC',
}
aspects = []
for t in top_terms:
    asp = ns_to_aspect.get(go_namespaces.get(t, ''), 'BP')
    aspects.append(asp)
aspects = np.array(aspects)
aspect_to_idx = {
    'BP': np.where(aspects == 'BP')[0].tolist(),
    'MF': np.where(aspects == 'MF')[0].tolist(),
    'CC': np.where(aspects == 'CC')[0].tolist(),
}
for k in ['BP', 'MF', 'CC']:
    print(f'Terms[{k}]={len(aspect_to_idx[k])}')

# 6. Train 3 specialised GCNs and stitch outputs back
test_pred_gcn = np.zeros_like(X_test_stack, dtype=np.float32)
X_tensor_full = torch.tensor(X_stack, dtype=torch.float32, device=device)
Y_tensor_full = torch.tensor(Y, dtype=torch.float32, device=device)
X_test_full = torch.tensor(X_test_stack, dtype=torch.float32, device=device)
def train_one(aspect_name: str, idx_cols: list[int]):
    if not idx_cols:
        print(f'[{aspect_name}] No terms; skipping.')
        return None
    terms_sub = [top_terms[i] for i in idx_cols]
    adj = build_adjacency(terms_sub, go_parents)
    model = SimpleGCN(input_dim=len(idx_cols), hidden_dim=1024, output_dim=len(idx_cols), adj_matrix=adj).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.BCELoss()
    X_t = X_tensor_full[:, idx_cols]
    Y_t = Y_tensor_full[:, idx_cols]
    n_samples = X_t.shape[0]
    BS = 256
    EPOCHS = 5
    model.train()
    print(f'\n=== Training GCN[{aspect_name}] terms={len(idx_cols)} ===')
    for epoch in range(EPOCHS):
        total_loss = 0.0
        perm = torch.randperm(n_samples, device=device)
        for i in range(0, n_samples, BS):
            b = perm[i:i + BS]
            optimizer.zero_grad()
            out = model(X_t[b])
            loss = criterion(out, Y_t[b])
            loss.backward()
            optimizer.step()
            total_loss += float(loss.item())
        with torch.no_grad():
            pred = (model(X_t[:2000]) > 0.3).float().cpu().numpy()
            f1 = f1_score(Y_t[:2000].cpu().numpy(), pred, average='micro')
        print(f'Epoch {epoch+1}/{EPOCHS} Loss={total_loss:.4f} micro-F1@0.30={f1:.4f}')
    return model

In [ ]:
# CELL 16 - Solution: 6. PHASE 4: POST-PROCESSING & SUBMISSION
# 6. PHASE 4: POST-PROCESSING & SUBMISSION
# ========================================
# HARDWARE: CPU / GPU
# ========================================
# This phase applies the "Strict Post-Processing" rules (Max/Min Propagation)
# and generates the final submission file.
import json
from pathlib import Path
import numpy as np
import pandas as pd

# Check if submission already exists
if (WORK_ROOT / 'submission.tsv').exists():
    print("submission.tsv already exists. Skipping Phase 4.")
else:
    print("Starting Phase 4: Post-processing & submission...")
    # Ensure go_parents is available (from Phase 1)
    if 'go_parents' not in locals() or 'go_namespaces' not in locals():
        print("Reloading GO graph (parse_obo)...")
        def parse_obo(path: Path):
            parents = {}
            namespaces = {}
            cur_id, cur_ns = None, None
            with path.open('r', encoding='utf-8') as f:
                for line in f:
                    line = line.strip()
                    if line == '[Term]':
                        if cur_id and cur_ns:
                            namespaces[cur_id] = cur_ns
                        cur_id, cur_ns = None, None
                    elif line.startswith('id: GO:'):
                        cur_id = line.split('id: ', 1)[1]
                    elif line.startswith('namespace:'):
                        cur_ns = line.split('namespace: ', 1)[1]
                    elif line.startswith('is_a:') and cur_id:
                        parent = line.split('is_a: ', 1)[1].split(' ! ')[0]
                        parents.setdefault(cur_id, set()).add(parent)
                if cur_id and cur_ns:
                    namespaces[cur_id] = cur_ns
            return parents, namespaces
        go_parents, go_namespaces = parse_obo(PATH_GO_OBO)
    # Load test IDs
    test_ids = pd.read_feather(WORK_ROOT / 'parsed' / 'test_seq.feather')['id']
    # Load stacker predictions
    pred_path = WORK_ROOT / 'features' / 'test_pred_gcn.npy'
    if not pred_path.exists():
        raise FileNotFoundError("Missing `test_pred_gcn.npy`. Run Phase 3 (GCN stacker) first.")
    preds = np.load(pred_path)
    # Load term list (must match Phase 3)
    terms_path = WORK_ROOT / 'features' / 'top_terms_1500.json'
    if terms_path.exists():
        with open(terms_path, 'r') as f:
            top_terms = json.load(f)
    else:
        print("Warning: top_terms_1500.json missing; rebuilding from train_terms counts (may mismatch Phase 3).")
        train_terms = pd.read_parquet(WORK_ROOT / 'parsed' / 'train_terms.parquet')
        top_terms = train_terms['term'].value_counts().head(preds.shape[1]).index.tolist()
    if preds.shape[1] != len(top_terms):
        raise ValueError(f"Shape mismatch: preds has {preds.shape[1]} terms, top_terms has {len(top_terms)}.")
    # ------------------------------------------
    # Strict post-processing (Max/Min Propagation)
    # ------------------------------------------
    print(f"Applying hierarchy rules on {len(top_terms)} terms...")
    df_pred = pd.DataFrame(preds, columns=top_terms)
    term_set = set(top_terms)
    term_to_parents = {}
    term_to_children = {}
    for term in top_terms:
        parents = go_parents.get(term, set())
        if not parents:
            continue
        parents = parents.intersection(term_set)
        if not parents:
            continue
        term_to_parents[term] = list(parents)
        for p in parents:
            term_to_children.setdefault(p, []).append(term)
    # Max Propagation (Child -> Parent)
    for _ in range(2):
        for child, parents in term_to_parents.items():
            child_scores = df_pred[child].values
            for parent in parents:
                df_pred[parent] = np.maximum(df_pred[parent].values, child_scores)
    # Min Propagation (Parent -> Child)
    for _ in range(2):
        for parent, children in term_to_children.items():
            parent_scores = df_pred[parent].values
            for child in children:
                df_pred[child] = np.minimum(df_pred[child].values, parent_scores)
    # ------------------------------------------
    # Submission formatting (CAFA rules)
    # - tab-separated, no header
    # - score in (0, 1.000]
    # - up to 3 significant figures
    # - <= 1500 terms per target (MF/BP/CC combined)
    # ------------------------------------------
    df_pred['EntryID'] = test_ids.values
    submission = df_pred.melt(id_vars='EntryID', var_name='term', value_name='score')
    # Enforce score range + remove zeros
    submission['score'] = submission['score'].clip(lower=0.0, upper=1.0)
    submission = submission[submission['score'] > 0.0]
    # Light pruning (keeps file size sane; still rule-compliant)
    submission = submission[submission['score'] >= 0.001]
    # Keep top 1500 per protein (rule)
    submission = submission.sort_values(['EntryID', 'score'], ascending=[True, False])
    submission = submission.groupby('EntryID', sort=False).head(1500)
    # Write with <= 3 significant figures
    submission.to_csv(
        WORK_ROOT / 'submission.tsv',
        sep='\t',
        index=False,
        header=False,
        float_format='%.3g',
    )
    print(f"Done! Submission saved to {WORK_ROOT / 'submission.tsv'}")
    if 'STORE' in globals() and STORE is not None:
        STORE.push('stage_09_submission', [WORK_ROOT / 'submission.tsv'], note='final submission')

In [ ]:
# CELL 17 - Solution: 7. PHASE 5: FREE TEXT PREDICTION (EXTRA)
# 7. PHASE 5: FREE TEXT PREDICTION (EXTRA)
# ==========================================
# HARDWARE: CPU
# ==========================================
# Official CAFA constraints (summary):
# - Combined file (GO + Text) allowed
# - Text: up to 5 lines per protein; ASCII printable; no tabs; <=3000 chars per protein total
# - Scores should be in (0, 1.000] and up to 3 significant figures
if (WORK_ROOT / 'submission_with_text.tsv').exists():
    print("submission_with_text.tsv already exists. Skipping Phase 5.")
elif not (WORK_ROOT / 'submission.tsv').exists():
    print("submission.tsv not found. Please run Phase 4 first.")
else:
    print("Starting Phase 5: Text Generation...")
    # 1. Load Submission & GO Graph
    print("Loading submission and GO data...")
    submission = pd.read_csv(
        WORK_ROOT / 'submission.tsv',
        sep='\t',
        header=None,
        names=['EntryID', 'term', 'score'],
    )

    if 'graph' not in locals():
        import obonet
        graph = obonet.read_obo(PATH_GO_OBO)

    # 2. Generate Text Descriptions
    print("Generating descriptions...")

    # Pre-fetch term names to avoid graph lookups in loop
    term_names = {node: data.get('name', 'unknown function') for node, data in graph.nodes(data=True)}

    text_rows = []
    unique_ids = submission['EntryID'].unique()
    for protein_id in tqdm(unique_ids, desc="Generating Text"):
        prot_preds = submission[submission['EntryID'] == protein_id]
        top_go = prot_preds.sort_values('score', ascending=False).head(3)
        if top_go.empty:
            # If we have no GO lines for this protein, skip text (keeps score>0 rule clean)
            continue

        term_descs = []
        for _, row in top_go.iterrows():
            term_id = row['term']
            term_descs.append(term_names.get(term_id, term_id))

        joined_terms = ", ".join(term_descs)
        description = f"{protein_id} is predicted to be involved in: {joined_terms}."

        # Ensure no tabs in description
        description = description.replace('\t', ' ')

        # Score: strictly > 0 and <= 1
        score = float(top_go.iloc[0]['score'])
        score = min(max(score, 0.001), 1.0)

        # One line per protein (<=5 allowed)
        text_rows.append({
            'EntryID': protein_id,
            'term': 'Text',
            'score': score,
            'description': description,
        })

    df_text = pd.DataFrame(text_rows)

    print("Saving combined submission...")
    with open(WORK_ROOT / 'submission_with_text.tsv', 'w', encoding='utf-8') as f:
        # 1) GO preds (3 cols) already CAFA-formatted in Phase 4
        submission.to_csv(f, sep='\t', index=False, header=False, float_format='%.3g')

        # 2) Text preds (4 cols)
        for _, row in df_text.iterrows():
            # Up to 3 significant figures
            score_str = format(float(row['score']), '.3g')
            f.write(f"{row['EntryID']}\tText\t{score_str}\t{row['description']}\n")

    print(f"Done! Combined submission saved to {WORK_ROOT / 'submission_with_text.tsv'}")


In [ ]:
# CELL 18 - Diagnostics: what actually contributes (OOF ablations; read-only)
# Goal: identify which Level-1 predictors help/hurt an OOF mean-ensemble under IA-F1.
# This cell ONLY reads artefacts and plots; it does not affect training or saved outputs.

import os
from pathlib import Path
import numpy as np
import pandas as pd

try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    plt.rcParams.update({'font.size': 12})
except Exception as e:
    plt = None
    sns = None
    print('Plotting libs not available; will print tables only:', repr(e))

WORK_ROOT = Path(WORK_ROOT)
DIAG_N = int(os.getenv('CAFA_DIAG_N', '20000'))
THRS = np.linspace(0.05, 0.60, 23)

def _load_pred(stem: str) -> np.ndarray | None:
    # Prefer level1_preds folder, fall back to legacy root folder
    cands = [
        WORK_ROOT / 'features' / 'level1_preds' / f'{stem}.npy',
        WORK_ROOT / 'features' / f'{stem}.npy',
    ]
    for p in cands:
        if p.exists():
            return np.load(p)
    return None

def _load_targets_topk() -> tuple[np.ndarray, list[str]]:
    train_terms_path = WORK_ROOT / 'parsed' / 'train_terms.parquet'
    train_seq_path = WORK_ROOT / 'parsed' / 'train_seq.feather'
    top_terms_path = WORK_ROOT / 'features' / 'top_terms_1500.json'

    if not train_terms_path.exists() or not train_seq_path.exists():
        raise FileNotFoundError('Missing parsed targets; expected parsed/train_terms.parquet and parsed/train_seq.feather')

    train_terms = pd.read_parquet(train_terms_path)
    train_ids = pd.read_feather(train_seq_path)['id'].astype(str).tolist()

    if top_terms_path.exists():
        import json
        top_terms = json.loads(top_terms_path.read_text())
    else:
        # Fallback: recompute top terms from training labels
        top_terms = train_terms['term'].value_counts().head(1500).index.tolist()

    train_terms_top = train_terms[train_terms['term'].isin(top_terms)]
    y_df = train_terms_top.pivot_table(index='EntryID', columns='term', aggfunc='size', fill_value=0)
    y_df = y_df.reindex(train_ids, fill_value=0)
    y = y_df.values.astype(np.float32)
    return y, top_terms

def _load_ia_weights(top_terms: list[str]) -> np.ndarray:
    # IA.tsv is required in repo root; still guard for robustness
    ia_path = WORK_ROOT.parent / 'IA.tsv'
    if not ia_path.exists():
        ia_path = WORK_ROOT / 'IA.tsv'
    if not ia_path.exists():
        return np.ones((len(top_terms),), dtype=np.float32)
    ia_df = pd.read_csv(ia_path, sep='\t', names=['term', 'ia'])
    ia_map = dict(zip(ia_df['term'], ia_df['ia']))
    w = np.array([float(ia_map.get(t, 0.0) or 0.0) for t in top_terms], dtype=np.float32)
    # Avoid all-zeros (would collapse the metric)
    if float(w.sum()) <= 0:
        w = np.ones_like(w)
    return w

def _subsample_rows(y_true: np.ndarray, *ys: np.ndarray) -> tuple[np.ndarray, list[np.ndarray]]:
    n = int(y_true.shape[0])
    m = min(n, int(DIAG_N))
    if m <= 0:
        return y_true[:0], [a[:0] for a in ys]
    idx = np.linspace(0, n - 1, num=m, dtype=np.int64)
    out = [np.asarray(a[idx], dtype=np.float32) for a in ys]
    return np.asarray(y_true[idx], dtype=np.float32), out

def _ia_f1(y_true: np.ndarray, y_score: np.ndarray, weights: np.ndarray, thr: float) -> float:
    # IA-weighted F1 over ALL terms (no aspect split here; goal is model contribution ranking)
    y_true = (y_true > 0).astype(np.int8)
    y_pred = (y_score >= float(thr)).astype(np.int8)
    tp = (y_pred & y_true).sum(axis=0).astype(np.float64)
    pred = y_pred.sum(axis=0).astype(np.float64)
    true = y_true.sum(axis=0).astype(np.float64)
    w = weights.astype(np.float64)
    w_tp = float((w * tp).sum())
    w_pred = float((w * pred).sum())
    w_true = float((w * true).sum())
    p = (w_tp / w_pred) if w_pred > 0 else 0.0
    r = (w_tp / w_true) if w_true > 0 else 0.0
    return (2 * p * r / (p + r)) if (p + r) > 0 else 0.0

def _best_over_thrs(y_true: np.ndarray, y_score: np.ndarray, weights: np.ndarray) -> tuple[float, float, np.ndarray]:
    scores = np.array([_ia_f1(y_true, y_score, weights, thr=float(t)) for t in THRS], dtype=np.float32)
    best_i = int(np.argmax(scores))
    return float(THRS[best_i]), float(scores[best_i]), scores

# ----------------------------
# Load Y + predictions
# ----------------------------
try:
    Y_full, top_terms = _load_targets_topk()
    w = _load_ia_weights(top_terms)
except Exception as e:
    print('Cannot run contributor analysis (missing targets/IA):', repr(e))
    raise

models = {
    'logreg': _load_pred('oof_pred_logreg'),
    'gbdt': _load_pred('oof_pred_gbdt'),
    'dnn': _load_pred('oof_pred_dnn'),
    'knn': _load_pred('oof_pred_knn'),
}
models = {k: v for k, v in models.items() if v is not None}
if not models:
    print('No OOF prediction matrices found. Expected files like features/level1_preds/oof_pred_logreg.npy')
    raise SystemExit(0)

# Shape sanity
n = int(Y_full.shape[0])
for k, v in models.items():
    if v.shape[0] != n:
        raise ValueError(f'Row mismatch for {k}: preds={v.shape[0]} vs y={n}')
    if v.shape[1] != Y_full.shape[1]:
        raise ValueError(f'Col mismatch for {k}: preds={v.shape[1]} vs y={Y_full.shape[1]}')

# Subsample to keep it interactive
Y_s, pred_list = _subsample_rows(Y_full, *[models[k] for k in models.keys()])
models_s = dict(zip(models.keys(), pred_list))

# ----------------------------
# 1) Individual model scores
# ----------------------------
rows = []
curves = {}
for name, p in models_s.items():
    best_thr, best_score, curve = _best_over_thrs(Y_s, p, w)
    rows.append({'model': name, 'best_thr': best_thr, 'best_ia_f1': best_score})
    curves[name] = curve
df_ind = pd.DataFrame(rows).sort_values('best_ia_f1', ascending=False)
print('\nIndividual model best IA-F1 (ALL terms; sampled):')
display(df_ind)

if plt is not None:
    plt.figure(figsize=(10, 4))
    for name in df_ind['model'].tolist():
        plt.plot(THRS, curves[name], label=name)
    plt.title('OOF (sampled): IA-F1 vs threshold per model')
    plt.xlabel('threshold')
    plt.ylabel('IA-F1')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()

# ----------------------------
# 2) Full mean-ensemble and leave-one-out deltas
# ----------------------------
names = list(models_s.keys())
full_pred = np.mean(np.stack([models_s[nm] for nm in names], axis=0), axis=0)
full_thr, full_best, full_curve = _best_over_thrs(Y_s, full_pred, w)
print(f"\nFull mean-ensemble best (sampled): IA-F1={full_best:.4f} @ thr={full_thr:.2f} using {names}")

loo = []
for drop in names:
    keep = [nm for nm in names if nm != drop]
    if not keep:
        continue
    pred = np.mean(np.stack([models_s[nm] for nm in keep], axis=0), axis=0)
    thr, best, _ = _best_over_thrs(Y_s, pred, w)
    loo.append({
        'drop': drop,
        'score_without': best,
        'delta_vs_full': best - full_best,
        'best_thr_without': thr,
    })
df_loo = pd.DataFrame(loo).sort_values('delta_vs_full')
print('\nLeave-one-out deltas (negative means the dropped model was helping):')
display(df_loo)

if plt is not None and len(df_loo) > 0:
    plt.figure(figsize=(10, 4))
    sns.barplot(data=df_loo, y='drop', x='delta_vs_full') if sns is not None else plt.barh(df_loo['drop'], df_loo['delta_vs_full'])
    plt.axvline(0.0, color='k', linewidth=1)
    plt.title('Contribution via leave-one-out: ΔIA-F1 vs full mean-ensemble (sampled)')
    plt.xlabel('Δ IA-F1 (positive => model hurts full mean)')
    plt.ylabel('dropped model')
    plt.grid(True, axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()

# ----------------------------
# 3) Greedy forward selection (what to add next)
# ----------------------------
remaining = set(names)
selected = []
history = []
base = None
base_thr = None
base_best = 0.0

for step in range(len(names)):
    best_choice = None
    best_score = -1.0
    best_thr = None
    for cand in sorted(remaining):
        trial = selected + [cand]
        pred = models_s[trial[0]] if len(trial) == 1 else np.mean(np.stack([models_s[nm] for nm in trial], axis=0), axis=0)
        thr, score, _ = _best_over_thrs(Y_s, pred, w)
        if score > best_score:
            best_score = score
            best_choice = cand
            best_thr = thr
    selected.append(best_choice)
    remaining.remove(best_choice)
    history.append({'step': step + 1, 'added': best_choice, 'best_ia_f1': best_score, 'best_thr': best_thr})

df_greedy = pd.DataFrame(history)
print('\nGreedy forward selection (mean-ensemble; sampled):')
display(df_greedy)

if plt is not None and len(df_greedy) > 0:
    plt.figure(figsize=(10, 4))
    plt.plot(df_greedy['step'], df_greedy['best_ia_f1'], marker='o')
    for _, r in df_greedy.iterrows():
        plt.text(float(r['step']) + 0.05, float(r['best_ia_f1']), str(r['added']))
    plt.title('Greedy add: best achievable IA-F1 as models are added (sampled)')
    plt.xlabel('number of models in mean-ensemble')
    plt.ylabel('best IA-F1')
    plt.grid(True, alpha=0.3)
    plt.show()

print("\nSo what?")
print("- Models with strongly negative leave-one-out deltas are likely your best contributors.")
print("- Models with positive deltas may be uncalibrated/noisy; consider re-weighting, per-model temperature, or dropping.")
print("- If greedy selection stalls early, extra models aren’t adding signal under simple averaging (stacker should still help).")